# Part One
Import required tools for collecting and cleaning the data: <br>
* Some tools are more commonly used
* Some tools are specific to the Alpha Vantage API

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv('secret')

In [2]:
import json
import requests
import collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import scipy.stats as sp
import pickle

In [3]:
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.timeseries import TimeSeries

import asyncio
from alpha_vantage.async_support.timeseries import TimeSeries

# Part Two
Obtain forex data from the Alpha Vantage API: <br>
* I am obtaining pricing data for the past 100 days for pairings of the top ten most traded currencies <br>
* For each pairing I am taking only the data required into a pandas dataframe, adding a column for "to" and "from" symbol to keep track of each currency, and then concatenating the dataframes into a large dataframe for easier use

In [4]:
app = ForeignExchange(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')

Create empty dataframe to concatenate to:

In [5]:
all_pairs = pd.DataFrame()

Obtain data from API:

In [6]:
usdeur = app.get_currency_exchange_daily('USD', 'EUR')

* Create dataframe from returned data
* Create new columns for "to" and "from" symbols
* Rename columns to clean up
* Show tail of dataframe in preparation of next step

In [7]:
usdeur1 = usdeur[0]
usdeur1['to symbol'] = 'EUR'
usdeur1['from symbol'] = 'USD'
usdeur1.rename(columns={'1. open': 'open'}, inplace=True)
usdeur1.rename(columns={'2. high': 'high'}, inplace=True)
usdeur1.rename(columns={'3. low': 'low'}, inplace=True)
usdeur1.rename(columns={'4. close': 'close'}, inplace=True)

usdeur1.tail()

,open,high,low,close,to symbol,from symbol
date,,,,,,
2022-06-17,0.9473,0.9570,0.9467,0.9526,EUR,USD
2022-06-16,0.9571,0.9631,0.9432,0.9470,EUR,USD
2022-06-15,0.9597,0.9648,0.9516,0.9570,EUR,USD
2022-06-14,0.9604,0.9613,0.9537,0.9597,EUR,USD
2022-06-13,0.9507,0.9612,0.9503,0.9604,EUR,USD


* Convert index to datetime
* Reorder index to ascending order
* Show head of dataframe to confirm that the reorder was successful

In [8]:
pd.to_datetime(usdeur1.index)
usdeur1 = usdeur1.sort_index()
usdeur1.head()

,open,high,low,close,to symbol,from symbol
date,,,,,,
2022-06-13,0.9507,0.9612,0.9503,0.9604,EUR,USD
2022-06-14,0.9604,0.9613,0.9537,0.9597,EUR,USD
2022-06-15,0.9597,0.9648,0.9516,0.9570,EUR,USD
2022-06-16,0.9571,0.9631,0.9432,0.9470,EUR,USD
2022-06-17,0.9473,0.9570,0.9467,0.9526,EUR,USD


* Create a time column in preparation for time series analysis
* Reorder columns to be more easily understood

In [9]:
usdeur1['time'] = np.arange(len(usdeur1.index))
usdeur1 = usdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'time']]

usdeur1.head()

,from symbol,to symbol,open,high,low,close,time
date,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,0
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,1
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,2
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,3
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,4


* Add lag features using values from close column as the closing price is the target to predict

In [10]:
usdeur1['lag_1'] = usdeur1['close'].shift(1)
usdeur1['lag_2'] = usdeur1['close'].shift(2)
usdeur1['lag_3'] = usdeur1['close'].shift(3)
usdeur1['lag_4'] = usdeur1['close'].shift(4)
usdeur1['lag_5'] = usdeur1['close'].shift(5)
usdeur1['lag_6'] = usdeur1['close'].shift(6)
usdeur1['lag_7'] = usdeur1['close'].shift(7)

usdeur1 = usdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'time']]
usdeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,time
date,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,1
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,2
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,3
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,4
2022-06-20,USD,EUR,0.9528,0.9544,0.9482,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,5
2022-06-21,USD,EUR,0.9510,0.9513,0.9448,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,6
2022-06-22,USD,EUR,0.9489,0.9547,0.9426,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,7
2022-06-23,USD,EUR,0.9460,0.9536,0.9449,0.9499,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,8


* Add rolling mean column using values from close column
* Also known as rolling window

In [11]:
usdeur1['rolling_mean'] = usdeur1['close'].rolling(window=7).mean()
usdeur1 = usdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'time']]
usdeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,time
date,,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,2
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,3
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,4
2022-06-20,USD,EUR,0.9528,0.9544,0.9482,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,5
2022-06-21,USD,EUR,0.9510,0.9513,0.9448,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,0.953786,6
2022-06-22,USD,EUR,0.9489,0.9547,0.9426,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,0.951757,7
2022-06-23,USD,EUR,0.9460,0.9536,0.9449,0.9499,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.950357,8


* Add expanding mean column using values from close column
* Also known as expanding window

In [12]:
usdeur1['expanding_mean'] = usdeur1['close'].expanding(2).mean()
usdeur1 = usdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]
usdeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.960050,1
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,0.959033,2
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,0.956025,3
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,0.955340,4
2022-06-20,USD,EUR,0.9528,0.9544,0.9482,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,0.954617,5
2022-06-21,USD,EUR,0.9510,0.9513,0.9448,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,0.953786,0.953786,6
2022-06-22,USD,EUR,0.9489,0.9547,0.9426,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,0.951757,0.952838,7
2022-06-23,USD,EUR,0.9460,0.9536,0.9449,0.9499,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.950357,0.952511,8


Concatanate individual dataframe to all_pairs dataframe:

In [13]:
all_pairs = pd.concat([all_pairs, usdeur1])

Save as pickle for future use:

In [14]:
usdeur1.to_pickle('usdeur1.pkl')

Now repeat for all currency pairs:

In [15]:
usdjpy = app.get_currency_exchange_daily('USD', 'jpy')
usdjpy1 = usdjpy[0]
usdjpy1['to symbol'] = 'JPY'
usdjpy1['from symbol'] = 'USD'

usdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
usdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
usdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
usdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdjpy1.index)
usdjpy1 = usdjpy1.sort_index()

usdjpy1['time'] = np.arange(len(usdjpy1.index))

usdjpy1['lag_1'] = usdjpy1['close'].shift(1)
usdjpy1['lag_2'] = usdjpy1['close'].shift(2)
usdjpy1['lag_3'] = usdjpy1['close'].shift(3)
usdjpy1['lag_4'] = usdjpy1['close'].shift(4)
usdjpy1['lag_5'] = usdjpy1['close'].shift(5)
usdjpy1['lag_6'] = usdjpy1['close'].shift(6)
usdjpy1['lag_7'] = usdjpy1['close'].shift(7)

usdjpy1['rolling_mean'] = usdjpy1['close'].rolling(window=7).mean()
usdjpy1['expanding_mean'] = usdjpy1['close'].expanding(2).mean()

usdjpy1 = usdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdjpy1])

usdjpy1.to_pickle('usdjpy1.pkl')

usdjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,JPY,134.400,135.197,133.56,134.399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,JPY,134.409,135.600,133.85,135.565,134.399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.982000,1
2022-06-15,USD,JPY,135.565,135.579,133.48,133.810,135.565,134.399,NaN,NaN,NaN,NaN,NaN,NaN,134.591333,2
2022-06-16,USD,JPY,133.815,134.678,131.46,132.288,133.810,135.565,134.399,NaN,NaN,NaN,NaN,NaN,134.015500,3
2022-06-17,USD,JPY,132.293,135.425,132.15,134.960,132.288,133.810,135.565,134.399,NaN,NaN,NaN,NaN,134.204400,4
2022-06-20,USD,JPY,134.993,135.441,134.50,135.091,134.960,132.288,133.810,135.565,134.399,NaN,NaN,NaN,134.352167,5
2022-06-21,USD,JPY,135.095,136.708,134.89,136.654,135.091,134.960,132.288,133.810,135.565,134.399,NaN,134.681000,134.681000,6
2022-06-22,USD,JPY,136.651,136.713,135.65,136.121,136.654,135.091,134.960,132.288,133.810,135.565,134.399,134.927000,134.861000,7
2022-06-23,USD,JPY,136.123,136.200,134.24,134.927,136.121,136.654,135.091,134.960,132.288,133.810,135.565,134.835857,134.868333,8


In [16]:
usdgbp = app.get_currency_exchange_daily('USD', 'gbp')
usdgbp1 = usdgbp[0]
usdgbp1['to symbol'] = 'GBP'
usdgbp1['from symbol'] = 'USD'

usdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
usdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
usdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
usdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdgbp1.index)
usdgbp1 = usdgbp1.sort_index()

usdgbp1['time'] = np.arange(len(usdgbp1.index))

usdgbp1['lag_1'] = usdgbp1['close'].shift(1)
usdgbp1['lag_2'] = usdgbp1['close'].shift(2)
usdgbp1['lag_3'] = usdgbp1['close'].shift(3)
usdgbp1['lag_4'] = usdgbp1['close'].shift(4)
usdgbp1['lag_5'] = usdgbp1['close'].shift(5)
usdgbp1['lag_6'] = usdgbp1['close'].shift(6)
usdgbp1['lag_7'] = usdgbp1['close'].shift(7)

usdgbp1['rolling_mean'] = usdgbp1['close'].rolling(window=7).mean()
usdgbp1['expanding_mean'] = usdgbp1['close'].expanding(2).mean()

usdgbp1 = usdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]


all_pairs = pd.concat([all_pairs, usdgbp1])

usdgbp1.to_pickle('usdgbp1.pkl')

usdgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,GBP,0.81218,0.82568,0.81218,0.82399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,GBP,0.82396,0.83773,0.81952,0.83347,0.82399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.828730,1
2022-06-15,USD,GBP,0.83340,0.83389,0.81957,0.82153,0.83347,0.82399,NaN,NaN,NaN,NaN,NaN,NaN,0.826330,2
2022-06-16,USD,GBP,0.82144,0.82990,0.80647,0.80942,0.82153,0.83347,0.82399,NaN,NaN,NaN,NaN,NaN,0.822102,3
2022-06-17,USD,GBP,0.80938,0.82139,0.80901,0.81764,0.80942,0.82153,0.83347,0.82399,NaN,NaN,NaN,NaN,0.821210,4
2022-06-20,USD,GBP,0.81819,0.81969,0.81445,0.81634,0.81764,0.80942,0.82153,0.83347,0.82399,NaN,NaN,NaN,0.820398,5
2022-06-21,USD,GBP,0.81632,0.81697,0.81180,0.81432,0.81634,0.81764,0.80942,0.82153,0.83347,0.82399,NaN,0.819530,0.819530,6
2022-06-22,USD,GBP,0.81428,0.82187,0.81218,0.81536,0.81432,0.81634,0.81764,0.80942,0.82153,0.83347,0.82399,0.818297,0.819009,7
2022-06-23,USD,GBP,0.81520,0.82163,0.81340,0.81585,0.81536,0.81432,0.81634,0.81764,0.80942,0.82153,0.83347,0.815780,0.818658,8


In [17]:
usdaud = app.get_currency_exchange_daily('USD', 'aud')
usdaud1 = usdaud[0]
usdaud1['to symbol'] = 'AUD'
usdaud1['from symbol'] = 'USD'

usdaud1.rename(columns={'1. open': 'open'}, inplace=True)
usdaud1.rename(columns={'2. high': 'high'}, inplace=True)
usdaud1.rename(columns={'3. low': 'low'}, inplace=True)
usdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdaud1.index)
usdaud1 = usdaud1.sort_index()

usdaud1['time'] = np.arange(len(usdaud1.index))

usdaud1['lag_1'] = usdaud1['close'].shift(1)
usdaud1['lag_2'] = usdaud1['close'].shift(2)
usdaud1['lag_3'] = usdaud1['close'].shift(3)
usdaud1['lag_4'] = usdaud1['close'].shift(4)
usdaud1['lag_5'] = usdaud1['close'].shift(5)
usdaud1['lag_6'] = usdaud1['close'].shift(6)
usdaud1['lag_7'] = usdaud1['close'].shift(7)

usdaud1['rolling_mean'] = usdaud1['close'].rolling(window=7).mean()
usdaud1['expanding_mean'] = usdaud1['close'].expanding(2).mean()

usdaud1 = usdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdaud1])

usdaud1.to_pickle('usdaud1.pkl')

usdaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,AUD,1.42074,1.44640,1.42064,1.44269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,AUD,1.44261,1.45932,1.43450,1.45390,1.44269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.448295,1
2022-06-15,USD,AUD,1.45433,1.45444,1.42351,1.42680,1.45390,1.44269,NaN,NaN,NaN,NaN,NaN,NaN,1.441130,2
2022-06-16,USD,AUD,1.42684,1.43968,1.41440,1.41850,1.42680,1.45390,1.44269,NaN,NaN,NaN,NaN,NaN,1.435472,3
2022-06-17,USD,AUD,1.41842,1.44955,1.41770,1.44180,1.41850,1.42680,1.45390,1.44269,NaN,NaN,NaN,NaN,1.436738,4
2022-06-20,USD,AUD,1.44053,1.44331,1.42890,1.43732,1.44180,1.41850,1.42680,1.45390,1.44269,NaN,NaN,NaN,1.436835,5
2022-06-21,USD,AUD,1.43742,1.44167,1.42984,1.43392,1.43732,1.44180,1.41850,1.42680,1.45390,1.44269,NaN,1.436419,1.436419,6
2022-06-22,USD,AUD,1.43373,1.45300,1.43373,1.44360,1.43392,1.43732,1.44180,1.41850,1.42680,1.45390,1.44269,1.436549,1.437316,7
2022-06-23,USD,AUD,1.44319,1.45541,1.44298,1.44936,1.44360,1.43392,1.43732,1.44180,1.41850,1.42680,1.45390,1.435900,1.438654,8


In [18]:
usdchf = app.get_currency_exchange_daily('USD', 'chf')
usdchf1 = usdchf[0]
usdchf1['to symbol'] = 'CHF'
usdchf1['from symbol'] = 'USD'

usdchf1.rename(columns={'1. open': 'open'}, inplace=True)
usdchf1.rename(columns={'2. high': 'high'}, inplace=True)
usdchf1.rename(columns={'3. low': 'low'}, inplace=True)
usdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdchf1.index)
usdchf1 = usdchf1.sort_index()

usdchf1['time'] = np.arange(len(usdchf1.index))

usdchf1['lag_1'] = usdchf1['close'].shift(1)
usdchf1['lag_2'] = usdchf1['close'].shift(2)
usdchf1['lag_3'] = usdchf1['close'].shift(3)
usdchf1['lag_4'] = usdchf1['close'].shift(4)
usdchf1['lag_5'] = usdchf1['close'].shift(5)
usdchf1['lag_6'] = usdchf1['close'].shift(6)
usdchf1['lag_7'] = usdchf1['close'].shift(7)

usdchf1['rolling_mean'] = usdchf1['close'].rolling(window=7).mean()
usdchf1['expanding_mean'] = usdchf1['close'].expanding(2).mean()

usdchf1 = usdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdchf1])

usdchf1.to_pickle('usdchf1.pkl')

usdchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,CHF,0.98761,0.99944,0.98699,0.99711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,CHF,0.99723,1.00371,0.98700,1.00137,0.99711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999240,1
2022-06-15,USD,CHF,1.00116,1.00499,0.99280,0.99358,1.00137,0.99711,NaN,NaN,NaN,NaN,NaN,NaN,0.997353,2
2022-06-16,USD,CHF,0.99369,0.99893,0.96279,0.96627,0.99358,1.00137,0.99711,NaN,NaN,NaN,NaN,NaN,0.989583,3
2022-06-17,USD,CHF,0.96609,0.97327,0.96160,0.96970,0.96627,0.99358,1.00137,0.99711,NaN,NaN,NaN,NaN,0.985606,4
2022-06-20,USD,CHF,0.96969,0.97140,0.96190,0.96670,0.96970,0.96627,0.99358,1.00137,0.99711,NaN,NaN,NaN,0.982455,5
2022-06-21,USD,CHF,0.96661,0.96838,0.96260,0.96519,0.96670,0.96970,0.96627,0.99358,1.00137,0.99711,NaN,0.979989,0.979989,6
2022-06-22,USD,CHF,0.96528,0.96895,0.95780,0.96094,0.96519,0.96670,0.96970,0.96627,0.99358,1.00137,0.99711,0.974821,0.977608,7
2022-06-23,USD,CHF,0.96089,0.96785,0.95640,0.96088,0.96094,0.96519,0.96670,0.96970,0.96627,0.99358,1.00137,0.969037,0.975749,8


In [19]:
usdcad = app.get_currency_exchange_daily('USD', 'cad')
usdcad1 = usdcad[0]
usdcad1['to symbol'] = 'CAD'
usdcad1['from symbol'] = 'USD'

usdcad1.rename(columns={'1. open': 'open'}, inplace=True)
usdcad1.rename(columns={'2. high': 'high'}, inplace=True)
usdcad1.rename(columns={'3. low': 'low'}, inplace=True)
usdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdcad1.index)
usdcad1 = usdcad1.sort_index()

usdcad1['time'] = np.arange(len(usdcad1.index))

usdcad1['lag_1'] = usdcad1['close'].shift(1)
usdcad1['lag_2'] = usdcad1['close'].shift(2)
usdcad1['lag_3'] = usdcad1['close'].shift(3)
usdcad1['lag_4'] = usdcad1['close'].shift(4)
usdcad1['lag_5'] = usdcad1['close'].shift(5)
usdcad1['lag_6'] = usdcad1['close'].shift(6)
usdcad1['lag_7'] = usdcad1['close'].shift(7)

usdcad1['rolling_mean'] = usdcad1['close'].rolling(window=7).mean()
usdcad1['expanding_mean'] = usdcad1['close'].expanding(2).mean()

usdcad1 = usdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdcad1])

usdcad1.to_pickle('usdcad1.pkl')

usdcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,CAD,1.27764,1.29005,1.27764,1.28926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,CAD,1.28926,1.29749,1.28630,1.29557,1.28926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.292415,1
2022-06-15,USD,CAD,1.29549,1.29959,1.28600,1.28839,1.29557,1.28926,NaN,NaN,NaN,NaN,NaN,NaN,1.291073,2
2022-06-16,USD,CAD,1.28846,1.29703,1.28580,1.29476,1.28839,1.29557,1.28926,NaN,NaN,NaN,NaN,NaN,1.291995,3
2022-06-17,USD,CAD,1.29475,1.30788,1.29360,1.30260,1.29476,1.28839,1.29557,1.28926,NaN,NaN,NaN,NaN,1.294116,4
2022-06-20,USD,CAD,1.30251,1.30380,1.29750,1.29799,1.30260,1.29476,1.28839,1.29557,1.28926,NaN,NaN,NaN,1.294762,5
2022-06-21,USD,CAD,1.29800,1.29862,1.29030,1.29168,1.29799,1.30260,1.29476,1.28839,1.29557,1.28926,NaN,1.294321,1.294321,6
2022-06-22,USD,CAD,1.29173,1.29965,1.29100,1.29473,1.29168,1.29799,1.30260,1.29476,1.28839,1.29557,1.28926,1.295103,1.294372,7
2022-06-23,USD,CAD,1.29482,1.30174,1.29330,1.29940,1.29473,1.29168,1.29799,1.30260,1.29476,1.28839,1.29557,1.295650,1.294931,8


In [20]:
usdhkd = app.get_currency_exchange_daily('USD', 'hkd')
usdhkd1 = usdhkd[0]
usdhkd1['to symbol'] = 'HKD'
usdhkd1['from symbol'] = 'USD'

usdhkd1.rename(columns={'1. open': 'open'}, inplace=True)
usdhkd1.rename(columns={'2. high': 'high'}, inplace=True)
usdhkd1.rename(columns={'3. low': 'low'}, inplace=True)
usdhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdhkd1.index)
usdhkd1 = usdhkd1.sort_index()

usdhkd1['time'] = np.arange(len(usdhkd1.index))

usdhkd1['lag_1'] = usdhkd1['close'].shift(1)
usdhkd1['lag_2'] = usdhkd1['close'].shift(2)
usdhkd1['lag_3'] = usdhkd1['close'].shift(3)
usdhkd1['lag_4'] = usdhkd1['close'].shift(4)
usdhkd1['lag_5'] = usdhkd1['close'].shift(5)
usdhkd1['lag_6'] = usdhkd1['close'].shift(6)
usdhkd1['lag_7'] = usdhkd1['close'].shift(7)

usdhkd1['rolling_mean'] = usdhkd1['close'].rolling(window=7).mean()
usdhkd1['expanding_mean'] = usdhkd1['close'].expanding(2).mean()

usdhkd1 = usdhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdhkd1])

usdhkd1.to_pickle('usdhkd1.pkl')

usdhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,HKD,7.84900,7.85001,7.84896,7.84961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,HKD,7.84966,7.85001,7.84922,7.84975,7.84961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.849680,1
2022-06-15,USD,HKD,7.84986,7.85001,7.84903,7.84945,7.84975,7.84961,NaN,NaN,NaN,NaN,NaN,NaN,7.849603,2
2022-06-16,USD,HKD,7.84946,7.84997,7.84790,7.84940,7.84945,7.84975,7.84961,NaN,NaN,NaN,NaN,NaN,7.849552,3
2022-06-17,USD,HKD,7.84960,7.85010,7.84877,7.84940,7.84940,7.84945,7.84975,7.84961,NaN,NaN,NaN,NaN,7.849522,4
2022-06-20,USD,HKD,7.84937,7.85000,7.84900,7.84990,7.84940,7.84940,7.84945,7.84975,7.84961,NaN,NaN,NaN,7.849585,5
2022-06-21,USD,HKD,7.84984,7.85002,7.84933,7.85000,7.84990,7.84940,7.84940,7.84945,7.84975,7.84961,NaN,7.849644,7.849644,6
2022-06-22,USD,HKD,7.84989,7.85001,7.84920,7.84968,7.85000,7.84990,7.84940,7.84940,7.84945,7.84975,7.84961,7.849654,7.849649,7
2022-06-23,USD,HKD,7.84962,7.84990,7.84800,7.84886,7.84968,7.85000,7.84990,7.84940,7.84940,7.84945,7.84975,7.849527,7.849561,8


In [21]:
usdsek = app.get_currency_exchange_daily('USD', 'sek')
usdsek1 = usdsek[0]
usdsek1['to symbol'] = 'SEK'
usdsek1['from symbol'] = 'USD'

usdsek1.rename(columns={'1. open': 'open'}, inplace=True)
usdsek1.rename(columns={'2. high': 'high'}, inplace=True)
usdsek1.rename(columns={'3. low': 'low'}, inplace=True)
usdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdsek1.index)
usdsek1 = usdsek1.sort_index()

usdsek1['time'] = np.arange(len(usdsek1.index))

usdsek1['lag_1'] = usdsek1['close'].shift(1)
usdsek1['lag_2'] = usdsek1['close'].shift(2)
usdsek1['lag_3'] = usdsek1['close'].shift(3)
usdsek1['lag_4'] = usdsek1['close'].shift(4)
usdsek1['lag_5'] = usdsek1['close'].shift(5)
usdsek1['lag_6'] = usdsek1['close'].shift(6)
usdsek1['lag_7'] = usdsek1['close'].shift(7)

usdsek1['rolling_mean'] = usdsek1['close'].rolling(window=7).mean()
usdsek1['expanding_mean'] = usdsek1['close'].expanding(2).mean()

usdsek1 = usdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdsek1])

usdsek1.to_pickle('usdsek1.pkl')

usdsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,SEK,9.99660,10.20427,9.99660,10.19630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,SEK,10.19650,10.22667,10.10965,10.19890,10.19630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.197600,1
2022-06-15,USD,SEK,10.19920,10.30360,10.09680,10.17700,10.19890,10.19630,NaN,NaN,NaN,NaN,NaN,NaN,10.190733,2
2022-06-16,USD,SEK,10.17740,10.30876,10.09670,10.12930,10.17700,10.19890,10.19630,NaN,NaN,NaN,NaN,NaN,10.175375,3
2022-06-17,USD,SEK,10.13160,10.24461,10.12220,10.17006,10.12930,10.17700,10.19890,10.19630,NaN,NaN,NaN,NaN,10.174312,4
2022-06-20,USD,SEK,10.15400,10.19172,10.09290,10.11460,10.17006,10.12930,10.17700,10.19890,10.19630,NaN,NaN,NaN,10.164360,5
2022-06-21,USD,SEK,10.11460,10.12621,10.03350,10.09110,10.11460,10.17006,10.12930,10.17700,10.19890,10.1963,NaN,10.153894,10.153894,6
2022-06-22,USD,SEK,10.09348,10.19246,10.04150,10.09290,10.09110,10.11460,10.17006,10.12930,10.17700,10.1989,10.1963,10.139123,10.146270,7
2022-06-23,USD,SEK,10.09370,10.21038,10.07190,10.18190,10.09290,10.09110,10.11460,10.17006,10.12930,10.1770,10.1989,10.136694,10.150229,8


In [22]:
usdnzd = app.get_currency_exchange_daily('USD', 'nzd')
usdnzd1 = usdnzd[0]
usdnzd1['to symbol'] = 'NZD'
usdnzd1['from symbol'] = 'USD'

usdnzd1.rename(columns={'1. open': 'open'}, inplace=True)
usdnzd1.rename(columns={'2. high': 'high'}, inplace=True)
usdnzd1.rename(columns={'3. low': 'low'}, inplace=True)
usdnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(usdnzd1.index)
usdnzd1 = usdnzd1.sort_index()

usdnzd1['time'] = np.arange(len(usdnzd1.index))

usdnzd1['lag_1'] = usdnzd1['close'].shift(1)
usdnzd1['lag_2'] = usdnzd1['close'].shift(2)
usdnzd1['lag_3'] = usdnzd1['close'].shift(3)
usdnzd1['lag_4'] = usdnzd1['close'].shift(4)
usdnzd1['lag_5'] = usdnzd1['close'].shift(5)
usdnzd1['lag_6'] = usdnzd1['close'].shift(6)
usdnzd1['lag_7'] = usdnzd1['close'].shift(7)

usdnzd1['rolling_mean'] = usdnzd1['close'].rolling(window=7).mean()
usdnzd1['expanding_mean'] = usdnzd1['close'].expanding(2).mean()

usdnzd1 = usdnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, usdnzd1])

usdnzd1.to_pickle('usdnzd1.pkl')

usdnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,NZD,1.5756,1.6001,1.5744,1.5960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,NZD,1.5955,1.6132,1.5890,1.6080,1.5960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.602000,1
2022-06-15,USD,NZD,1.6078,1.6079,1.5842,1.5905,1.6080,1.5960,NaN,NaN,NaN,NaN,NaN,NaN,1.598167,2
2022-06-16,USD,NZD,1.5900,1.6036,1.5636,1.5710,1.5905,1.6080,1.5960,NaN,NaN,NaN,NaN,NaN,1.591375,3
2022-06-17,USD,NZD,1.5719,1.5940,1.5695,1.5834,1.5710,1.5905,1.6080,1.5960,NaN,NaN,NaN,NaN,1.589780,4
2022-06-20,USD,NZD,1.5838,1.5854,1.5710,1.5786,1.5834,1.5710,1.5905,1.6080,1.5960,NaN,NaN,NaN,1.587917,5
2022-06-21,USD,NZD,1.5785,1.5819,1.5713,1.5785,1.5786,1.5834,1.5710,1.5905,1.6080,1.5960,NaN,1.586571,1.586571,6
2022-06-22,USD,NZD,1.5786,1.6009,1.5786,1.5901,1.5785,1.5786,1.5834,1.5710,1.5905,1.6080,1.5960,1.585729,1.587012,7
2022-06-23,USD,NZD,1.5906,1.5998,1.5841,1.5920,1.5901,1.5785,1.5786,1.5834,1.5710,1.5905,1.6080,1.583443,1.587567,8


In [23]:
eurusd = app.get_currency_exchange_daily('EUR', 'USD')
eurusd1 = eurusd[0]
eurusd1['to symbol'] = 'USD'
eurusd1['from symbol'] = 'EUR'

eurusd1.rename(columns={'1. open': 'open'}, inplace=True)
eurusd1.rename(columns={'2. high': 'high'}, inplace=True)
eurusd1.rename(columns={'3. low': 'low'}, inplace=True)
eurusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurusd1.index)
eurusd1 = eurusd1.sort_index()

eurusd1['time'] = np.arange(len(eurusd1.index))

eurusd1['lag_1'] = eurusd1['close'].shift(1)
eurusd1['lag_2'] = eurusd1['close'].shift(2)
eurusd1['lag_3'] = eurusd1['close'].shift(3)
eurusd1['lag_4'] = eurusd1['close'].shift(4)
eurusd1['lag_5'] = eurusd1['close'].shift(5)
eurusd1['lag_6'] = eurusd1['close'].shift(6)
eurusd1['lag_7'] = eurusd1['close'].shift(7)

eurusd1['rolling_mean'] = eurusd1['close'].rolling(window=7).mean()
eurusd1['expanding_mean'] = eurusd1['close'].expanding(2).mean()

eurusd1 = eurusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurusd1])

eurusd1.to_pickle('eurusd1.pkl')

eurusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,USD,1.05142,1.05201,1.0398,1.04075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,USD,1.04084,1.04848,1.0395,1.04117,1.04075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.040960,1
2022-06-15,EUR,USD,1.04153,1.05078,1.0357,1.04451,1.04117,1.04075,NaN,NaN,NaN,NaN,NaN,NaN,1.042143,2
2022-06-16,EUR,USD,1.04462,1.06014,1.0379,1.05540,1.04451,1.04117,1.04075,NaN,NaN,NaN,NaN,NaN,1.045457,3
2022-06-17,EUR,USD,1.05515,1.05608,1.0443,1.04980,1.05540,1.04451,1.04117,1.04075,NaN,NaN,NaN,NaN,1.046326,4
2022-06-20,EUR,USD,1.04870,1.05458,1.0472,1.05117,1.04980,1.05540,1.04451,1.04117,1.04075,NaN,NaN,NaN,1.047133,5
2022-06-21,EUR,USD,1.05102,1.05825,1.0507,1.05337,1.05117,1.04980,1.05540,1.04451,1.04117,1.04075,NaN,1.048024,1.048024,6
2022-06-22,EUR,USD,1.05341,1.06059,1.0467,1.05638,1.05337,1.05117,1.04980,1.05540,1.04451,1.04117,1.04075,1.050257,1.049069,7
2022-06-23,EUR,USD,1.05636,1.05812,1.0481,1.05208,1.05638,1.05337,1.05117,1.04980,1.05540,1.04451,1.04117,1.051816,1.049403,8


In [24]:
eurjpy = app.get_currency_exchange_daily('EUR', 'JPY')
eurjpy1 = eurjpy[0]
eurjpy1['to symbol'] = 'JPY'
eurjpy1['from symbol'] = 'EUR'

eurjpy1.rename(columns={'1. open': 'open'}, inplace=True)
eurjpy1.rename(columns={'2. high': 'high'}, inplace=True)
eurjpy1.rename(columns={'3. low': 'low'}, inplace=True)
eurjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurjpy1.index)
eurjpy1 = eurjpy1.sort_index()

eurjpy1['time'] = np.arange(len(eurjpy1.index))

eurjpy1['lag_1'] = eurjpy1['close'].shift(1)
eurjpy1['lag_2'] = eurjpy1['close'].shift(2)
eurjpy1['lag_3'] = eurjpy1['close'].shift(3)
eurjpy1['lag_4'] = eurjpy1['close'].shift(4)
eurjpy1['lag_5'] = eurjpy1['close'].shift(5)
eurjpy1['lag_6'] = eurjpy1['close'].shift(6)
eurjpy1['lag_7'] = eurjpy1['close'].shift(7)

eurjpy1['rolling_mean'] = eurjpy1['close'].rolling(window=7).mean()
eurjpy1['expanding_mean'] = eurjpy1['close'].expanding(2).mean()

eurjpy1 = eurjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurjpy1])

eurjpy1.to_pickle('eurjpy1.pkl')

eurjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,JPY,141.282,141.781,139.360,139.897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,JPY,139.908,141.221,139.460,141.168,139.897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.532500,1
2022-06-15,EUR,JPY,141.178,141.489,139.500,139.800,141.168,139.897,NaN,NaN,NaN,NaN,NaN,NaN,140.288333,2
2022-06-16,EUR,JPY,139.781,140.602,137.820,139.610,139.800,141.168,139.897,NaN,NaN,NaN,NaN,NaN,140.118750,3
2022-06-17,EUR,JPY,139.616,141.972,139.347,141.715,139.610,139.800,141.168,139.897,NaN,NaN,NaN,NaN,140.438000,4
2022-06-20,EUR,JPY,141.565,142.355,141.420,142.004,141.715,139.610,139.800,141.168,139.897,NaN,NaN,NaN,140.699000,5
2022-06-21,EUR,JPY,141.990,143.999,141.920,143.965,142.004,141.715,139.610,139.800,141.168,139.897,NaN,141.165571,141.165571,6
2022-06-22,EUR,JPY,143.967,144.246,142.630,143.796,143.965,142.004,141.715,139.610,139.800,141.168,139.897,141.722571,141.494375,7
2022-06-23,EUR,JPY,143.802,143.991,141.360,141.971,143.796,143.965,142.004,141.715,139.610,139.800,141.168,141.837286,141.547333,8


In [25]:
eurgbp = app.get_currency_exchange_daily('EUR', 'GBP')
eurgbp1 = eurgbp[0]
eurgbp1['to symbol'] = 'GBP'
eurgbp1['from symbol'] = 'EUR'

eurgbp1.rename(columns={'1. open': 'open'}, inplace=True)
eurgbp1.rename(columns={'2. high': 'high'}, inplace=True)
eurgbp1.rename(columns={'3. low': 'low'}, inplace=True)
eurgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurgbp1.index)
eurgbp1 = eurgbp1.sort_index()

eurgbp1['time'] = np.arange(len(eurgbp1.index))

eurgbp1['lag_1'] = eurgbp1['close'].shift(1)
eurgbp1['lag_2'] = eurgbp1['close'].shift(2)
eurgbp1['lag_3'] = eurgbp1['close'].shift(3)
eurgbp1['lag_4'] = eurgbp1['close'].shift(4)
eurgbp1['lag_5'] = eurgbp1['close'].shift(5)
eurgbp1['lag_6'] = eurgbp1['close'].shift(6)
eurgbp1['lag_7'] = eurgbp1['close'].shift(7)

eurgbp1['rolling_mean'] = eurgbp1['close'].rolling(window=7).mean()
eurgbp1['expanding_mean'] = eurgbp1['close'].expanding(2).mean()

eurgbp1 = eurgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurgbp1])

eurgbp1.to_pickle('eurgbp1.pkl')

eurgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,GBP,0.85316,0.85940,0.8529,0.85750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,GBP,0.85760,0.87157,0.8551,0.86780,0.85750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.862650,1
2022-06-15,EUR,GBP,0.86792,0.87211,0.8563,0.85820,0.86780,0.85750,NaN,NaN,NaN,NaN,NaN,NaN,0.861167,2
2022-06-16,EUR,GBP,0.85801,0.86334,0.8510,0.85419,0.85820,0.86780,0.85750,NaN,NaN,NaN,NaN,NaN,0.859422,3
2022-06-17,EUR,GBP,0.85434,0.85952,0.8521,0.85886,0.85419,0.85820,0.86780,0.85750,NaN,NaN,NaN,NaN,0.859310,4
2022-06-20,EUR,GBP,0.85764,0.86119,0.8569,0.85820,0.85886,0.85419,0.85820,0.86780,0.85750,NaN,NaN,NaN,0.859125,5
2022-06-21,EUR,GBP,0.85792,0.86103,0.8566,0.85790,0.85820,0.85886,0.85419,0.85820,0.86780,0.85750,NaN,0.858950,0.858950,6
2022-06-22,EUR,GBP,0.85751,0.86248,0.8573,0.86109,0.85790,0.85820,0.85886,0.85419,0.85820,0.86780,0.8575,0.859463,0.859217,7
2022-06-23,EUR,GBP,0.86119,0.86410,0.8571,0.85830,0.86109,0.85790,0.85820,0.85886,0.85419,0.85820,0.8678,0.858106,0.859116,8


In [26]:
euraud = app.get_currency_exchange_daily('EUR', 'AUD')
euraud1 = euraud[0]
euraud1['to symbol'] = 'AUD'
euraud1['from symbol'] = 'EUR'

euraud1.rename(columns={'1. open': 'open'}, inplace=True)
euraud1.rename(columns={'2. high': 'high'}, inplace=True)
euraud1.rename(columns={'3. low': 'low'}, inplace=True)
euraud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(euraud1.index)
euraud1 = euraud1.sort_index()

euraud1['time'] = np.arange(len(euraud1.index))

euraud1['lag_1'] = euraud1['close'].shift(1)
euraud1['lag_2'] = euraud1['close'].shift(2)
euraud1['lag_3'] = euraud1['close'].shift(3)
euraud1['lag_4'] = euraud1['close'].shift(4)
euraud1['lag_5'] = euraud1['close'].shift(5)
euraud1['lag_6'] = euraud1['close'].shift(6)
euraud1['lag_7'] = euraud1['close'].shift(7)

euraud1['rolling_mean'] = euraud1['close'].rolling(window=7).mean()
euraud1['expanding_mean'] = euraud1['close'].expanding(2).mean()

euraud1 = euraud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, euraud1])

euraud1.to_pickle('euraud1.pkl')

euraud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,AUD,1.49218,1.50560,1.4901,1.50227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,AUD,1.50204,1.51851,1.4963,1.51454,1.50227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.508405,1
2022-06-15,EUR,AUD,1.51470,1.51899,1.4873,1.49053,1.51454,1.50227,NaN,NaN,NaN,NaN,NaN,NaN,1.502447,2
2022-06-16,EUR,AUD,1.49041,1.50869,1.4857,1.49710,1.49053,1.51454,1.50227,NaN,NaN,NaN,NaN,NaN,1.501110,3
2022-06-17,EUR,AUD,1.49736,1.51599,1.4952,1.51381,1.49710,1.49053,1.51454,1.50227,NaN,NaN,NaN,NaN,1.503650,4
2022-06-20,EUR,AUD,1.51226,1.51518,1.5042,1.51131,1.51381,1.49710,1.49053,1.51454,1.50227,NaN,NaN,NaN,1.504927,5
2022-06-21,EUR,AUD,1.51130,1.52104,1.5073,1.51095,1.51131,1.51381,1.49710,1.49053,1.51454,1.50227,NaN,1.505787,1.505787,6
2022-06-22,EUR,AUD,1.51092,1.52801,1.5101,1.52537,1.51095,1.51131,1.51381,1.49710,1.49053,1.51454,1.50227,1.509087,1.508235,7
2022-06-23,EUR,AUD,1.52523,1.53451,1.5199,1.52505,1.52537,1.51095,1.51131,1.51381,1.49710,1.49053,1.51454,1.510589,1.510103,8


In [27]:
eurchf = app.get_currency_exchange_daily('EUR', 'CHF')
eurchf1 = eurchf[0]
eurchf1['to symbol'] = 'CHF'
eurchf1['from symbol'] = 'EUR'

eurchf1.rename(columns={'1. open': 'open'}, inplace=True)
eurchf1.rename(columns={'2. high': 'high'}, inplace=True)
eurchf1.rename(columns={'3. low': 'low'}, inplace=True)
eurchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurchf1.index)
eurchf1 = eurchf1.sort_index()

eurchf1['time'] = np.arange(len(eurchf1.index))

eurchf1['lag_1'] = eurchf1['close'].shift(1)
eurchf1['lag_2'] = eurchf1['close'].shift(2)
eurchf1['lag_3'] = eurchf1['close'].shift(3)
eurchf1['lag_4'] = eurchf1['close'].shift(4)
eurchf1['lag_5'] = eurchf1['close'].shift(5)
eurchf1['lag_6'] = eurchf1['close'].shift(6)
eurchf1['lag_7'] = eurchf1['close'].shift(7)

eurchf1['rolling_mean'] = eurchf1['close'].rolling(window=7).mean()
eurchf1['expanding_mean'] = eurchf1['close'].expanding(2).mean()

eurchf1 = eurchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]


all_pairs = pd.concat([all_pairs, eurchf1])

eurchf1.to_pickle('eurchf1.pkl')

eurchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,CHF,1.03815,1.04095,1.03587,1.03870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,CHF,1.03813,1.04524,1.03370,1.04330,1.03870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.041000,1
2022-06-15,EUR,CHF,1.04295,1.04781,1.03766,1.03829,1.04330,1.03870,NaN,NaN,NaN,NaN,NaN,NaN,1.040097,2
2022-06-16,EUR,CHF,1.03818,1.04165,1.01270,1.01980,1.03829,1.04330,1.03870,NaN,NaN,NaN,NaN,NaN,1.035022,3
2022-06-17,EUR,CHF,1.02010,1.02202,1.00950,1.01785,1.01980,1.03829,1.04330,1.03870,NaN,NaN,NaN,NaN,1.031588,4
2022-06-20,EUR,CHF,1.01750,1.01944,1.01292,1.01640,1.01785,1.01980,1.03829,1.04330,1.03870,NaN,NaN,NaN,1.029057,5
2022-06-21,EUR,CHF,1.01655,1.02161,1.01612,1.01729,1.01640,1.01785,1.01980,1.03829,1.04330,1.03870,NaN,1.027376,1.027376,6
2022-06-22,EUR,CHF,1.01768,1.01886,1.01280,1.01537,1.01729,1.01640,1.01785,1.01980,1.03829,1.04330,1.03870,1.024043,1.025875,7
2022-06-23,EUR,CHF,1.01540,1.01655,1.00630,1.01103,1.01537,1.01729,1.01640,1.01785,1.01980,1.03829,1.04330,1.019433,1.024226,8


In [28]:
eurcad = app.get_currency_exchange_daily('EUR', 'CAD')
eurcad1 = eurcad[0]
eurcad1['to symbol'] = 'CAD'
eurcad1['from symbol'] = 'EUR'

eurcad1.rename(columns={'1. open': 'open'}, inplace=True)
eurcad1.rename(columns={'2. high': 'high'}, inplace=True)
eurcad1.rename(columns={'3. low': 'low'}, inplace=True)
eurcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurcad1.index)
eurcad1 = eurcad1.sort_index()

eurcad1['time'] = np.arange(len(eurcad1.index))

eurcad1['lag_1'] = eurcad1['close'].shift(1)
eurcad1['lag_2'] = eurcad1['close'].shift(2)
eurcad1['lag_3'] = eurcad1['close'].shift(3)
eurcad1['lag_4'] = eurcad1['close'].shift(4)
eurcad1['lag_5'] = eurcad1['close'].shift(5)
eurcad1['lag_6'] = eurcad1['close'].shift(6)
eurcad1['lag_7'] = eurcad1['close'].shift(7)

eurcad1['rolling_mean'] = eurcad1['close'].rolling(window=7).mean()
eurcad1['expanding_mean'] = eurcad1['close'].expanding(2).mean()

eurcad1 = eurcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurcad1])

eurcad1.to_pickle('eurcad1.pkl')

eurcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,CAD,1.34366,1.34664,1.34000,1.34176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,CAD,1.34186,1.35287,1.34060,1.34946,1.34176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345610,1
2022-06-15,EUR,CAD,1.34940,1.36066,1.34310,1.34595,1.34946,1.34176,NaN,NaN,NaN,NaN,NaN,NaN,1.345723,2
2022-06-16,EUR,CAD,1.34587,1.36762,1.34155,1.36656,1.34595,1.34946,1.34176,NaN,NaN,NaN,NaN,NaN,1.350933,3
2022-06-17,EUR,CAD,1.36642,1.36996,1.36061,1.36743,1.36656,1.34595,1.34946,1.34176,NaN,NaN,NaN,NaN,1.354232,4
2022-06-20,EUR,CAD,1.36584,1.37028,1.36302,1.36434,1.36743,1.36656,1.34595,1.34946,1.34176,NaN,NaN,NaN,1.355917,5
2022-06-21,EUR,CAD,1.36381,1.36813,1.35905,1.36072,1.36434,1.36743,1.36656,1.34595,1.34946,1.34176,NaN,1.356603,1.356603,6
2022-06-22,EUR,CAD,1.36075,1.37151,1.35961,1.36789,1.36072,1.36434,1.36743,1.36656,1.34595,1.34946,1.34176,1.360336,1.358014,7
2022-06-23,EUR,CAD,1.36781,1.37102,1.35894,1.36742,1.36789,1.36072,1.36434,1.36743,1.36656,1.34595,1.34946,1.362901,1.359059,8


In [29]:
eurhkd = app.get_currency_exchange_daily('EUR', 'HKD')
eurhkd1 = eurhkd[0]
eurhkd1['to symbol'] = 'HKD'
eurhkd1['from symbol'] = 'EUR'

eurhkd1.rename(columns={'1. open': 'open'}, inplace=True)
eurhkd1.rename(columns={'2. high': 'high'}, inplace=True)
eurhkd1.rename(columns={'3. low': 'low'}, inplace=True)
eurhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurhkd1.index)
eurhkd1 = eurhkd1.sort_index()

eurhkd1['time'] = np.arange(len(eurhkd1.index))

eurhkd1['lag_1'] = eurhkd1['close'].shift(1)
eurhkd1['lag_2'] = eurhkd1['close'].shift(2)
eurhkd1['lag_3'] = eurhkd1['close'].shift(3)
eurhkd1['lag_4'] = eurhkd1['close'].shift(4)
eurhkd1['lag_5'] = eurhkd1['close'].shift(5)
eurhkd1['lag_6'] = eurhkd1['close'].shift(6)
eurhkd1['lag_7'] = eurhkd1['close'].shift(7)

eurhkd1['rolling_mean'] = eurhkd1['close'].rolling(window=7).mean()
eurhkd1['expanding_mean'] = eurhkd1['close'].expanding(2).mean()

eurhkd1 = eurhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurhkd1])

eurhkd1.to_pickle('eurhkd1.pkl')

eurhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,HKD,8.25267,8.25718,8.16240,8.16800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,HKD,8.16982,8.23014,8.16020,8.17178,8.16800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.169890,1
2022-06-15,EUR,HKD,8.17474,8.24800,8.13093,8.19882,8.17178,8.16800,NaN,NaN,NaN,NaN,NaN,NaN,8.179533,2
2022-06-16,EUR,HKD,8.19874,8.32093,8.14770,8.28320,8.19882,8.17178,8.16800,NaN,NaN,NaN,NaN,NaN,8.205450,3
2022-06-17,EUR,HKD,8.27827,8.28837,8.19810,8.24150,8.28320,8.19882,8.17178,8.16800,NaN,NaN,NaN,NaN,8.212660,4
2022-06-20,EUR,HKD,8.23220,8.27793,8.22050,8.25020,8.24150,8.28320,8.19882,8.17178,8.16800,NaN,NaN,NaN,8.218917,5
2022-06-21,EUR,HKD,8.25080,8.30702,8.24810,8.26899,8.25020,8.24150,8.28320,8.19882,8.17178,8.16800,NaN,8.226070,8.226070,6
2022-06-22,EUR,HKD,8.26979,8.32502,8.21630,8.29110,8.26899,8.25020,8.24150,8.28320,8.19882,8.17178,8.16800,8.243656,8.234199,7
2022-06-23,EUR,HKD,8.29249,8.30503,8.22640,8.25730,8.29110,8.26899,8.25020,8.24150,8.28320,8.19882,8.17178,8.255873,8.236766,8


In [30]:
eursek = app.get_currency_exchange_daily('EUR', 'SEK')
eursek1 = eursek[0]
eursek1['to symbol'] = 'SEK'
eursek1['from symbol'] = 'EUR'

eursek1.rename(columns={'1. open': 'open'}, inplace=True)
eursek1.rename(columns={'2. high': 'high'}, inplace=True)
eursek1.rename(columns={'3. low': 'low'}, inplace=True)
eursek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eursek1.index)
eursek1 = eursek1.sort_index()

eursek1['time'] = np.arange(len(eursek1.index))

eursek1['lag_1'] = eursek1['close'].shift(1)
eursek1['lag_2'] = eursek1['close'].shift(2)
eursek1['lag_3'] = eursek1['close'].shift(3)
eursek1['lag_4'] = eursek1['close'].shift(4)
eursek1['lag_5'] = eursek1['close'].shift(5)
eursek1['lag_6'] = eursek1['close'].shift(6)
eursek1['lag_7'] = eursek1['close'].shift(7)

eursek1['rolling_mean'] = eursek1['close'].rolling(window=7).mean()
eursek1['expanding_mean'] = eursek1['close'].expanding(2).mean()

eursek1 = eursek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]


all_pairs = pd.concat([all_pairs, eursek1])

eursek1.to_pickle('eursek1.pkl')

eursek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,SEK,10.51197,10.63104,10.51154,10.61445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,SEK,10.61305,10.64339,10.55562,10.61622,10.61445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.615335,1
2022-06-15,EUR,SEK,10.61867,10.68250,10.60065,10.63275,10.61622,10.61445,NaN,NaN,NaN,NaN,NaN,NaN,10.621140,2
2022-06-16,EUR,SEK,10.62810,10.73964,10.61388,10.68575,10.63275,10.61622,10.61445,NaN,NaN,NaN,NaN,NaN,10.637293,3
2022-06-17,EUR,SEK,10.68568,10.71440,10.66180,10.68131,10.68575,10.63275,10.61622,10.61445,NaN,NaN,NaN,NaN,10.646096,4
2022-06-20,EUR,SEK,10.64946,10.69780,10.62730,10.63228,10.68131,10.68575,10.63275,10.61622,10.61445,NaN,NaN,NaN,10.643793,5
2022-06-21,EUR,SEK,10.63780,10.65800,10.61330,10.63432,10.63228,10.68131,10.68575,10.63275,10.61622,10.61445,NaN,10.642440,10.642440,6
2022-06-22,EUR,SEK,10.63714,10.69081,10.63270,10.66323,10.63432,10.63228,10.68131,10.68575,10.63275,10.61622,10.61445,10.649409,10.645039,7
2022-06-23,EUR,SEK,10.65690,10.72660,10.65501,10.71030,10.66323,10.63432,10.63228,10.68131,10.68575,10.63275,10.61622,10.662849,10.652290,8


In [31]:
eurnzd = app.get_currency_exchange_daily('EUR', 'NZD')
eurnzd1 = eurnzd[0]
eurnzd1['to symbol'] = 'NZD'
eurnzd1['from symbol'] = 'EUR'

eurnzd1.rename(columns={'1. open': 'open'}, inplace=True)
eurnzd1.rename(columns={'2. high': 'high'}, inplace=True)
eurnzd1.rename(columns={'3. low': 'low'}, inplace=True)
eurnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(eurnzd1.index)
eurnzd1 = eurnzd1.sort_index()

eurnzd1['time'] = np.arange(len(eurnzd1.index))

eurnzd1['lag_1'] = eurnzd1['close'].shift(1)
eurnzd1['lag_2'] = eurnzd1['close'].shift(2)
eurnzd1['lag_3'] = eurnzd1['close'].shift(3)
eurnzd1['lag_4'] = eurnzd1['close'].shift(4)
eurnzd1['lag_5'] = eurnzd1['close'].shift(5)
eurnzd1['lag_6'] = eurnzd1['close'].shift(6)
eurnzd1['lag_7'] = eurnzd1['close'].shift(7)

eurnzd1['rolling_mean'] = eurnzd1['close'].rolling(window=7).mean()
eurnzd1['expanding_mean'] = eurnzd1['close'].expanding(2).mean()

eurnzd1 = eurnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, eurnzd1])

eurnzd1.to_pickle('eurnzd1.pkl')

eurnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,EUR,NZD,1.65540,1.66666,1.65142,1.66090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,EUR,NZD,1.66134,1.67924,1.65833,1.67536,1.66090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.668130,1
2022-06-15,EUR,NZD,1.67536,1.68532,1.65540,1.66099,1.67536,1.66090,NaN,NaN,NaN,NaN,NaN,NaN,1.665750,2
2022-06-16,EUR,NZD,1.66158,1.66774,1.65119,1.65813,1.66099,1.67536,1.66090,NaN,NaN,NaN,NaN,NaN,1.663845,3
2022-06-17,EUR,NZD,1.65843,1.66775,1.65570,1.66252,1.65813,1.66099,1.67536,1.66090,NaN,NaN,NaN,NaN,1.663580,4
2022-06-20,EUR,NZD,1.66019,1.66459,1.65385,1.65987,1.66252,1.65813,1.66099,1.67536,1.66090,NaN,NaN,NaN,1.662962,5
2022-06-21,EUR,NZD,1.65994,1.66962,1.65853,1.66306,1.65987,1.66252,1.65813,1.66099,1.67536,1.66090,NaN,1.662976,1.662976,6
2022-06-22,EUR,NZD,1.66367,1.68529,1.66240,1.68025,1.66306,1.65987,1.66252,1.65813,1.66099,1.67536,1.66090,1.665740,1.665135,7
2022-06-23,EUR,NZD,1.68026,1.68908,1.67037,1.67595,1.68025,1.66306,1.65987,1.66252,1.65813,1.66099,1.67536,1.665824,1.666337,8


In [32]:
gbpusd = app.get_currency_exchange_daily('GBP', 'USD')
gbpusd1 = gbpusd[0]
gbpusd1['to symbol'] = 'USD'
gbpusd1['from symbol'] = 'GBP'

gbpusd1.rename(columns={'1. open': 'open'}, inplace=True)
gbpusd1.rename(columns={'2. high': 'high'}, inplace=True)
gbpusd1.rename(columns={'3. low': 'low'}, inplace=True)
gbpusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpusd1.index)
gbpusd1 = gbpusd1.sort_index()

gbpusd1['time'] = np.arange(len(gbpusd1.index))

gbpusd1['lag_1'] = gbpusd1['close'].shift(1)
gbpusd1['lag_2'] = gbpusd1['close'].shift(2)
gbpusd1['lag_3'] = gbpusd1['close'].shift(3)
gbpusd1['lag_4'] = gbpusd1['close'].shift(4)
gbpusd1['lag_5'] = gbpusd1['close'].shift(5)
gbpusd1['lag_6'] = gbpusd1['close'].shift(6)
gbpusd1['lag_7'] = gbpusd1['close'].shift(7)

gbpusd1['rolling_mean'] = gbpusd1['close'].rolling(window=7).mean()
gbpusd1['expanding_mean'] = gbpusd1['close'].expanding(2).mean()

gbpusd1 = gbpusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbpusd1])

gbpusd1.to_pickle('gbpusd1.pkl')

gbpusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,USD,1.23139,1.23170,1.2106,1.21345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,USD,1.21334,1.22086,1.1932,1.19953,1.21345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.206490,1
2022-06-15,GBP,USD,1.19965,1.22046,1.1988,1.21703,1.19953,1.21345,NaN,NaN,NaN,NaN,NaN,NaN,1.210003,2
2022-06-16,GBP,USD,1.21690,1.24064,1.2039,1.23512,1.21703,1.19953,1.21345,NaN,NaN,NaN,NaN,NaN,1.216282,3
2022-06-17,GBP,USD,1.23506,1.23605,1.2171,1.22220,1.23512,1.21703,1.19953,1.21345,NaN,NaN,NaN,NaN,1.217466,4
2022-06-20,GBP,USD,1.22227,1.22803,1.2197,1.22460,1.22220,1.23512,1.21703,1.19953,1.21345,NaN,NaN,NaN,1.218655,5
2022-06-21,GBP,USD,1.22478,1.23242,1.2239,1.22783,1.22460,1.22220,1.23512,1.21703,1.19953,1.21345,NaN,1.219966,1.219966,6
2022-06-22,GBP,USD,1.22777,1.23149,1.2159,1.22618,1.22783,1.22460,1.22220,1.23512,1.21703,1.19953,1.21345,1.221784,1.220743,7
2022-06-23,GBP,USD,1.22639,1.22946,1.2168,1.22575,1.22618,1.22783,1.22460,1.22220,1.23512,1.21703,1.19953,1.225530,1.221299,8


In [33]:
gbpeur = app.get_currency_exchange_daily('GBP', 'EUR')
gbpeur1 = gbpeur[0]
gbpeur1['to symbol'] = 'EUR'
gbpeur1['from symbol'] = 'GBP'

gbpeur1.rename(columns={'1. open': 'open'}, inplace=True)
gbpeur1.rename(columns={'2. high': 'high'}, inplace=True)
gbpeur1.rename(columns={'3. low': 'low'}, inplace=True)
gbpeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpeur1.index)
gbpeur1 = gbpeur1.sort_index()

gbpeur1['time'] = np.arange(len(gbpeur1.index))

gbpeur1['lag_1'] = gbpeur1['close'].shift(1)
gbpeur1['lag_2'] = gbpeur1['close'].shift(2)
gbpeur1['lag_3'] = gbpeur1['close'].shift(3)
gbpeur1['lag_4'] = gbpeur1['close'].shift(4)
gbpeur1['lag_5'] = gbpeur1['close'].shift(5)
gbpeur1['lag_6'] = gbpeur1['close'].shift(6)
gbpeur1['lag_7'] = gbpeur1['close'].shift(7)

gbpeur1['rolling_mean'] = gbpeur1['close'].rolling(window=7).mean()
gbpeur1['expanding_mean'] = gbpeur1['close'].expanding(2).mean()

gbpeur1 = gbpeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbpeur1])

gbpeur1.to_pickle('gbpeur1.pkl')

gbpeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,EUR,1.17070,1.17204,1.16330,1.16531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,EUR,1.16539,1.16887,1.14720,1.15140,1.16531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.158355,1
2022-06-15,GBP,EUR,1.15136,1.16733,1.14620,1.16460,1.15140,1.16531,NaN,NaN,NaN,NaN,NaN,NaN,1.160437,2
2022-06-16,GBP,EUR,1.16478,1.17467,1.15839,1.16970,1.16460,1.15140,1.16531,NaN,NaN,NaN,NaN,NaN,1.162753,3
2022-06-17,GBP,EUR,1.16980,1.17317,1.16310,1.16380,1.16970,1.16460,1.15140,1.16531,NaN,NaN,NaN,NaN,1.162962,4
2022-06-20,GBP,EUR,1.16490,1.16645,1.16090,1.16460,1.16380,1.16970,1.16460,1.15140,1.16531,NaN,NaN,NaN,1.163235,5
2022-06-21,GBP,EUR,1.16493,1.16700,1.16100,1.16510,1.16460,1.16380,1.16970,1.16460,1.15140,1.16531,NaN,1.163501,1.163501,6
2022-06-22,GBP,EUR,1.16500,1.16610,1.15900,1.16051,1.16510,1.16460,1.16380,1.16970,1.16460,1.15140,1.16531,1.162816,1.163128,7
2022-06-23,GBP,EUR,1.16037,1.16628,1.15680,1.16470,1.16051,1.16510,1.16460,1.16380,1.16970,1.16460,1.15140,1.164716,1.163302,8


In [34]:
gbpjpy = app.get_currency_exchange_daily('GBP', 'JPY')
gbpjpy1 = gbpjpy[0]
gbpjpy1['to symbol'] = 'JPY'
gbpjpy1['from symbol'] = 'GBP'

gbpjpy1.rename(columns={'1. open': 'open'}, inplace=True)
gbpjpy1.rename(columns={'2. high': 'high'}, inplace=True)
gbpjpy1.rename(columns={'3. low': 'low'}, inplace=True)
gbpjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpjpy1.index)
gbpjpy1 = gbpjpy1.sort_index()

gbpjpy1['time'] = np.arange(len(gbpjpy1.index))

gbpjpy1['lag_1'] = gbpjpy1['close'].shift(1)
gbpjpy1['lag_2'] = gbpjpy1['close'].shift(2)
gbpjpy1['lag_3'] = gbpjpy1['close'].shift(3)
gbpjpy1['lag_4'] = gbpjpy1['close'].shift(4)
gbpjpy1['lag_5'] = gbpjpy1['close'].shift(5)
gbpjpy1['lag_6'] = gbpjpy1['close'].shift(6)
gbpjpy1['lag_7'] = gbpjpy1['close'].shift(7)

gbpjpy1['rolling_mean'] = gbpjpy1['close'].rolling(window=7).mean()
gbpjpy1['expanding_mean'] = gbpjpy1['close'].expanding(2).mean()

gbpjpy1 = gbpjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]


all_pairs = pd.concat([all_pairs, gbpjpy1])

gbpjpy1.to_pickle('gbpjpy1.pkl')

gbpjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,JPY,165.542,166.127,162.230,163.093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,JPY,163.090,164.320,161.310,162.619,163.093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.856000,1
2022-06-15,GBP,JPY,162.639,163.158,161.700,162.874,162.619,163.093,NaN,NaN,NaN,NaN,NaN,NaN,162.862000,2
2022-06-16,GBP,JPY,162.874,163.802,159.960,163.416,162.874,162.619,163.093,NaN,NaN,NaN,NaN,NaN,163.000500,3
2022-06-17,GBP,JPY,163.436,166.222,162.919,164.995,163.416,162.874,162.619,163.093,NaN,NaN,NaN,NaN,163.399400,4
2022-06-20,GBP,JPY,164.995,165.654,164.450,165.447,164.995,163.416,162.874,162.619,163.093,NaN,NaN,NaN,163.740667,5
2022-06-21,GBP,JPY,165.446,167.849,165.310,167.808,165.447,164.995,163.416,162.874,162.619,163.093,NaN,164.321714,164.321714,6
2022-06-22,GBP,JPY,167.805,167.856,165.740,166.943,167.808,165.447,164.995,163.416,162.874,162.619,163.093,164.871714,164.649375,7
2022-06-23,GBP,JPY,166.952,167.042,164.630,165.398,166.943,167.808,165.447,164.995,163.416,162.874,162.619,165.268714,164.732556,8


In [35]:
gbpaud = app.get_currency_exchange_daily('GBP', 'AUD')
gbpaud1 = gbpaud[0]
gbpaud1['to symbol'] = 'AUD'
gbpaud1['from symbol'] = 'GBP'

gbpaud1.rename(columns={'1. open': 'open'}, inplace=True)
gbpaud1.rename(columns={'2. high': 'high'}, inplace=True)
gbpaud1.rename(columns={'3. low': 'low'}, inplace=True)
gbpaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpaud1.index)
gbpaud1 = gbpaud1.sort_index()

gbpaud1['time'] = np.arange(len(gbpaud1.index))

gbpaud1['lag_1'] = gbpaud1['close'].shift(1)
gbpaud1['lag_2'] = gbpaud1['close'].shift(2)
gbpaud1['lag_3'] = gbpaud1['close'].shift(3)
gbpaud1['lag_4'] = gbpaud1['close'].shift(4)
gbpaud1['lag_5'] = gbpaud1['close'].shift(5)
gbpaud1['lag_6'] = gbpaud1['close'].shift(6)
gbpaud1['lag_7'] = gbpaud1['close'].shift(7)

gbpaud1['rolling_mean'] = gbpaud1['close'].rolling(window=7).mean()
gbpaud1['expanding_mean'] = gbpaud1['close'].expanding(2).mean()

gbpaud1 = gbpaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbpaud1])

gbpaud1.to_pickle('gbpaud1.pkl')

gbpaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,AUD,1.74817,1.75507,1.74319,1.75104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,AUD,1.75139,1.75721,1.74140,1.74452,1.75104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.747780,1
2022-06-15,GBP,AUD,1.74453,1.74541,1.73248,1.73676,1.74452,1.75104,NaN,NaN,NaN,NaN,NaN,NaN,1.744107,2
2022-06-16,GBP,AUD,1.73656,1.76616,1.73072,1.75230,1.73676,1.74452,1.75104,NaN,NaN,NaN,NaN,NaN,1.746155,3
2022-06-17,GBP,AUD,1.75249,1.76582,1.74885,1.76306,1.75230,1.73676,1.74452,1.75104,NaN,NaN,NaN,NaN,1.749536,4
2022-06-20,GBP,AUD,1.76227,1.76349,1.74965,1.76079,1.76306,1.75230,1.73676,1.74452,1.75104,NaN,NaN,NaN,1.751412,5
2022-06-21,GBP,AUD,1.76081,1.76784,1.75546,1.76128,1.76079,1.76306,1.75230,1.73676,1.74452,1.75104,NaN,1.752821,1.752821,6
2022-06-22,GBP,AUD,1.76129,1.77818,1.76020,1.77049,1.76128,1.76079,1.76306,1.75230,1.73676,1.74452,1.75104,1.755600,1.755030,7
2022-06-23,GBP,AUD,1.77081,1.77924,1.76860,1.77688,1.77049,1.76128,1.76079,1.76306,1.75230,1.73676,1.74452,1.760223,1.757458,8


In [36]:
gbpchf = app.get_currency_exchange_daily('GBP', 'CHF')
gbpchf1 = gbpchf[0]
gbpchf1['to symbol'] = 'CHF'
gbpchf1['from symbol'] = 'GBP'

gbpchf1.rename(columns={'1. open': 'open'}, inplace=True)
gbpchf1.rename(columns={'2. high': 'high'}, inplace=True)
gbpchf1.rename(columns={'3. low': 'low'}, inplace=True)
gbpchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpchf1.index)
gbpchf1 = gbpchf1.sort_index()

gbpchf1['time'] = np.arange(len(gbpchf1.index))

gbpchf1['lag_1'] = gbpchf1['close'].shift(1)
gbpchf1['lag_2'] = gbpchf1['close'].shift(2)
gbpchf1['lag_3'] = gbpchf1['close'].shift(3)
gbpchf1['lag_4'] = gbpchf1['close'].shift(4)
gbpchf1['lag_5'] = gbpchf1['close'].shift(5)
gbpchf1['lag_6'] = gbpchf1['close'].shift(6)
gbpchf1['lag_7'] = gbpchf1['close'].shift(7)

gbpchf1['rolling_mean'] = gbpchf1['close'].rolling(window=7).mean()
gbpchf1['expanding_mean'] = gbpchf1['close'].expanding(2).mean()

gbpchf1 = gbpchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]


all_pairs = pd.concat([all_pairs, gbpchf1])

gbpchf1.to_pickle('gbpchf1.pkl')

gbpchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,CHF,1.21638,1.21740,1.2075,1.20986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,CHF,1.20974,1.21454,1.1962,1.20153,1.20986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.205695,1
2022-06-15,GBP,CHF,1.20147,1.21350,1.1994,1.20970,1.20153,1.20986,NaN,NaN,NaN,NaN,NaN,NaN,1.207030,2
2022-06-16,GBP,CHF,1.20940,1.21318,1.1800,1.19334,1.20970,1.20153,1.20986,NaN,NaN,NaN,NaN,NaN,1.203607,3
2022-06-17,GBP,CHF,1.19339,1.19536,1.1789,1.18515,1.19334,1.20970,1.20153,1.20986,NaN,NaN,NaN,NaN,1.199916,4
2022-06-20,GBP,CHF,1.18492,1.18697,1.1783,1.18378,1.18515,1.19334,1.20970,1.20153,1.20986,NaN,NaN,NaN,1.197227,5
2022-06-21,GBP,CHF,1.18379,1.18965,1.1815,1.18486,1.18378,1.18515,1.19334,1.20970,1.20153,1.20986,NaN,1.195460,1.195460,6
2022-06-22,GBP,CHF,1.18547,1.18633,1.1764,1.17859,1.18486,1.18378,1.18515,1.19334,1.20970,1.20153,1.20986,1.190993,1.193351,7
2022-06-23,GBP,CHF,1.17867,1.18151,1.1725,1.17808,1.17859,1.18486,1.18378,1.18515,1.19334,1.20970,1.20153,1.187643,1.191654,8


In [37]:
gbpcad = app.get_currency_exchange_daily('GBP', 'CAD')
gbpcad1 = gbpcad[0]
gbpcad1['to symbol'] = 'CAD'
gbpcad1['from symbol'] = 'GBP'

gbpcad1.rename(columns={'1. open': 'open'}, inplace=True)
gbpcad1.rename(columns={'2. high': 'high'}, inplace=True)
gbpcad1.rename(columns={'3. low': 'low'}, inplace=True)
gbpcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpcad1.index)
gbpcad1 = gbpcad1.sort_index()

gbpcad1['time'] = np.arange(len(gbpcad1.index))

gbpcad1['lag_1'] = gbpcad1['close'].shift(1)
gbpcad1['lag_2'] = gbpcad1['close'].shift(2)
gbpcad1['lag_3'] = gbpcad1['close'].shift(3)
gbpcad1['lag_4'] = gbpcad1['close'].shift(4)
gbpcad1['lag_5'] = gbpcad1['close'].shift(5)
gbpcad1['lag_6'] = gbpcad1['close'].shift(6)
gbpcad1['lag_7'] = gbpcad1['close'].shift(7)

gbpcad1['rolling_mean'] = gbpcad1['close'].rolling(window=7).mean()
gbpcad1['expanding_mean'] = gbpcad1['close'].expanding(2).mean()

gbpcad1 = gbpcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbpcad1])

gbpcad1.to_pickle('gbpcad1.pkl')

gbpcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,CAD,1.57401,1.57526,1.56011,1.56443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,CAD,1.56431,1.57290,1.54819,1.55426,1.56443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.559345,1
2022-06-15,GBP,CAD,1.55427,1.57268,1.55332,1.56789,1.55426,1.56443,NaN,NaN,NaN,NaN,NaN,NaN,1.562193,2
2022-06-16,GBP,CAD,1.56809,1.60073,1.55819,1.59936,1.56789,1.55426,1.56443,NaN,NaN,NaN,NaN,NaN,1.571485,3
2022-06-17,GBP,CAD,1.59923,1.60034,1.58925,1.59242,1.59936,1.56789,1.55426,1.56443,NaN,NaN,NaN,NaN,1.575672,4
2022-06-20,GBP,CAD,1.59191,1.59527,1.58777,1.58994,1.59242,1.59936,1.56789,1.55426,1.56443,NaN,NaN,NaN,1.578050,5
2022-06-21,GBP,CAD,1.58951,1.59131,1.58414,1.58598,1.58994,1.59242,1.59936,1.56789,1.55426,1.56443,NaN,1.579183,1.579183,6
2022-06-22,GBP,CAD,1.58612,1.59275,1.57982,1.58764,1.58598,1.58994,1.59242,1.59936,1.56789,1.55426,1.56443,1.582499,1.580240,7
2022-06-23,GBP,CAD,1.58785,1.59431,1.57906,1.59262,1.58764,1.58598,1.58994,1.59242,1.59936,1.56789,1.55426,1.587979,1.581616,8


In [38]:
gbphkd = app.get_currency_exchange_daily('GBP', 'HKD')
gbphkd1 = gbphkd[0]
gbphkd1['to symbol'] = 'HKD'
gbphkd1['from symbol'] = 'GBP'

gbphkd1.rename(columns={'1. open': 'open'}, inplace=True)
gbphkd1.rename(columns={'2. high': 'high'}, inplace=True)
gbphkd1.rename(columns={'3. low': 'low'}, inplace=True)
gbphkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbphkd1.index)
gbphkd1 = gbphkd1.sort_index()

gbphkd1['time'] = np.arange(len(gbphkd1.index))

gbphkd1['lag_1'] = gbphkd1['close'].shift(1)
gbphkd1['lag_2'] = gbphkd1['close'].shift(2)
gbphkd1['lag_3'] = gbphkd1['close'].shift(3)
gbphkd1['lag_4'] = gbphkd1['close'].shift(4)
gbphkd1['lag_5'] = gbphkd1['close'].shift(5)
gbphkd1['lag_6'] = gbphkd1['close'].shift(6)
gbphkd1['lag_7'] = gbphkd1['close'].shift(7)

gbphkd1['rolling_mean'] = gbphkd1['close'].rolling(window=7).mean()
gbphkd1['expanding_mean'] = gbphkd1['close'].expanding(2).mean()

gbphkd1 = gbphkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbphkd1])

gbphkd1.to_pickle('gbphkd1.pkl')

gbphkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,HKD,9.660,9.660,9.505,9.523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,HKD,9.526,9.573,9.372,9.416,9.523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.469500,1
2022-06-15,GBP,HKD,9.416,9.573,9.412,9.552,9.416,9.523,NaN,NaN,NaN,NaN,NaN,NaN,9.497000,2
2022-06-16,GBP,HKD,9.554,9.730,9.456,9.694,9.552,9.416,9.523,NaN,NaN,NaN,NaN,NaN,9.546250,3
2022-06-17,GBP,HKD,9.694,9.699,9.555,9.594,9.694,9.552,9.416,9.523,NaN,NaN,NaN,NaN,9.555800,4
2022-06-20,GBP,HKD,9.592,9.634,9.576,9.613,9.594,9.694,9.552,9.416,9.523,NaN,NaN,NaN,9.565333,5
2022-06-21,GBP,HKD,9.613,9.665,9.608,9.638,9.613,9.594,9.694,9.552,9.416,9.523,NaN,9.575714,9.575714,6
2022-06-22,GBP,HKD,9.637,9.664,9.549,9.626,9.638,9.613,9.594,9.694,9.552,9.416,9.523,9.590429,9.582000,7
2022-06-23,GBP,HKD,9.626,9.647,9.550,9.620,9.626,9.638,9.613,9.594,9.694,9.552,9.416,9.619571,9.586222,8


In [39]:
gbpsek = app.get_currency_exchange_daily('GBP', 'SEK')
gbpsek1 = gbpsek[0]
gbpsek1['to symbol'] = 'SEK'
gbpsek1['from symbol'] = 'GBP'

gbpsek1.rename(columns={'1. open': 'open'}, inplace=True)
gbpsek1.rename(columns={'2. high': 'high'}, inplace=True)
gbpsek1.rename(columns={'3. low': 'low'}, inplace=True)
gbpsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(gbpsek1.index)
gbpsek1 = gbpsek1.sort_index()

gbpsek1['time'] = np.arange(len(gbpsek1.index))

gbpsek1['lag_1'] = gbpsek1['close'].shift(1)
gbpsek1['lag_2'] = gbpsek1['close'].shift(2)
gbpsek1['lag_3'] = gbpsek1['close'].shift(3)
gbpsek1['lag_4'] = gbpsek1['close'].shift(4)
gbpsek1['lag_5'] = gbpsek1['close'].shift(5)
gbpsek1['lag_6'] = gbpsek1['close'].shift(6)
gbpsek1['lag_7'] = gbpsek1['close'].shift(7)

gbpsek1['rolling_mean'] = gbpsek1['close'].rolling(window=7).mean()
gbpsek1['expanding_mean'] = gbpsek1['close'].expanding(2).mean()

gbpsek1 = gbpsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, gbpsek1])

gbpsek1.to_pickle('gbpsek1.pkl')

gbpsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,GBP,SEK,12.30600,12.39390,12.29650,12.37100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,GBP,SEK,12.37093,12.39395,12.19190,12.22900,12.37100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.300000,1
2022-06-15,GBP,SEK,12.22493,12.43076,12.16852,12.38500,12.22900,12.37100,NaN,NaN,NaN,NaN,NaN,NaN,12.328333,2
2022-06-16,GBP,SEK,12.37579,12.58680,12.34797,12.50842,12.38500,12.22900,12.37100,NaN,NaN,NaN,NaN,NaN,12.373355,3
2022-06-17,GBP,SEK,12.50293,12.54820,12.41575,12.43410,12.50842,12.38500,12.22900,12.37100,NaN,NaN,NaN,NaN,12.385504,4
2022-06-20,GBP,SEK,12.40900,12.45200,12.35140,12.38540,12.43410,12.50842,12.38500,12.22900,12.37100,NaN,NaN,NaN,12.385487,5
2022-06-21,GBP,SEK,12.38770,12.40860,12.34130,12.39060,12.38540,12.43410,12.50842,12.38500,12.22900,12.37100,NaN,12.386217,12.386217,6
2022-06-22,GBP,SEK,12.39388,12.45190,12.34180,12.37020,12.39060,12.38540,12.43410,12.50842,12.38500,12.22900,12.371,12.386103,12.384215,7
2022-06-23,GBP,SEK,12.37120,12.49947,12.34190,12.48220,12.37020,12.39060,12.38540,12.43410,12.50842,12.38500,12.229,12.422274,12.395102,8


In [40]:
audusd = app.get_currency_exchange_daily('AUD', 'USD')
audusd1 = audusd[0]
audusd1['to symbol'] = 'USD'
audusd1['from symbol'] = 'AUD'

audusd1.rename(columns={'1. open': 'open'}, inplace=True)
audusd1.rename(columns={'2. high': 'high'}, inplace=True)
audusd1.rename(columns={'3. low': 'low'}, inplace=True)
audusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audusd1.index)
audusd1 = audusd1.sort_index()

audusd1['time'] = np.arange(len(audusd1.index))

audusd1['lag_1'] = audusd1['close'].shift(1)
audusd1['lag_2'] = audusd1['close'].shift(2)
audusd1['lag_3'] = audusd1['close'].shift(3)
audusd1['lag_4'] = audusd1['close'].shift(4)
audusd1['lag_5'] = audusd1['close'].shift(5)
audusd1['lag_6'] = audusd1['close'].shift(6)
audusd1['lag_7'] = audusd1['close'].shift(7)

audusd1['rolling_mean'] = audusd1['close'].rolling(window=7).mean()
audusd1['expanding_mean'] = audusd1['close'].expanding(2).mean()

audusd1 = audusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audusd1])

audusd1.to_pickle('audusd1.pkl')

audusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,USD,0.70377,0.70410,0.6908,0.69259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,USD,0.69263,0.69704,0.6848,0.68737,0.69259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.689980,1
2022-06-15,AUD,USD,0.68737,0.70245,0.6870,0.70049,0.68737,0.69259,NaN,NaN,NaN,NaN,NaN,NaN,0.693483,2
2022-06-16,AUD,USD,0.70052,0.70692,0.6940,0.70470,0.70049,0.68737,0.69259,NaN,NaN,NaN,NaN,NaN,0.696287,3
2022-06-17,AUD,USD,0.70471,0.70513,0.6894,0.69310,0.70470,0.70049,0.68737,0.69259,NaN,NaN,NaN,NaN,0.695650,4
2022-06-20,AUD,USD,0.69325,0.69960,0.6923,0.69520,0.69310,0.70470,0.70049,0.68737,0.69259,NaN,NaN,NaN,0.695575,5
2022-06-21,AUD,USD,0.69503,0.69937,0.6931,0.69650,0.69520,0.69310,0.70470,0.70049,0.68737,0.69259,NaN,0.695707,0.695707,6
2022-06-22,AUD,USD,0.69650,0.69724,0.6877,0.69235,0.69650,0.69520,0.69310,0.70470,0.70049,0.68737,0.69259,0.695673,0.695288,7
2022-06-23,AUD,USD,0.69242,0.69279,0.6865,0.68953,0.69235,0.69650,0.69520,0.69310,0.70470,0.70049,0.68737,0.695981,0.694648,8


In [41]:
audeur = app.get_currency_exchange_daily('AUD', 'EUR')
audeur1 = audeur[0]
audeur1['to symbol'] = 'EUR'
audeur1['from symbol'] = 'AUD'

audeur1.rename(columns={'1. open': 'open'}, inplace=True)
audeur1.rename(columns={'2. high': 'high'}, inplace=True)
audeur1.rename(columns={'3. low': 'low'}, inplace=True)
audeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audeur1.index)
audeur1 = audeur1.sort_index()

audeur1['time'] = np.arange(len(audeur1.index))

audeur1['lag_1'] = audeur1['close'].shift(1)
audeur1['lag_2'] = audeur1['close'].shift(2)
audeur1['lag_3'] = audeur1['close'].shift(3)
audeur1['lag_4'] = audeur1['close'].shift(4)
audeur1['lag_5'] = audeur1['close'].shift(5)
audeur1['lag_6'] = audeur1['close'].shift(6)
audeur1['lag_7'] = audeur1['close'].shift(7)

audeur1['rolling_mean'] = audeur1['close'].rolling(window=7).mean()
audeur1['expanding_mean'] = audeur1['close'].expanding(2).mean()

audeur1 = audeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audeur1])

audeur1.to_pickle('audeur1.pkl')

audeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,EUR,0.66861,0.67066,0.66405,0.66530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,EUR,0.66509,0.66780,0.65851,0.65969,0.66530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.662495,1
2022-06-15,AUD,EUR,0.65970,0.67188,0.65823,0.67050,0.65969,0.66530,NaN,NaN,NaN,NaN,NaN,NaN,0.665163,2
2022-06-16,AUD,EUR,0.67029,0.67265,0.66280,0.66740,0.67050,0.65969,0.66530,NaN,NaN,NaN,NaN,NaN,0.665722,3
2022-06-17,AUD,EUR,0.66730,0.66830,0.65949,0.66020,0.66740,0.67050,0.65969,0.66530,NaN,NaN,NaN,NaN,0.664618,4
2022-06-20,AUD,EUR,0.65906,0.66436,0.65906,0.66140,0.66020,0.66740,0.67050,0.65969,0.66530,NaN,NaN,NaN,0.664082,5
2022-06-21,AUD,EUR,0.66094,0.66300,0.65739,0.66108,0.66140,0.66020,0.66740,0.67050,0.65969,0.66530,NaN,0.663653,0.663653,6
2022-06-22,AUD,EUR,0.66131,0.66171,0.65431,0.65518,0.66108,0.66140,0.66020,0.66740,0.67050,0.65969,0.66530,0.662207,0.662594,7
2022-06-23,AUD,EUR,0.65519,0.65773,0.65154,0.65499,0.65518,0.66108,0.66140,0.66020,0.66740,0.67050,0.65969,0.661536,0.661749,8


In [42]:
audjpy = app.get_currency_exchange_daily('AUD', 'JPY')
audjpy1 = audjpy[0]
audjpy1['to symbol'] = 'JPY'
audjpy1['from symbol'] = 'AUD'

audjpy1.rename(columns={'1. open': 'open'}, inplace=True)
audjpy1.rename(columns={'2. high': 'high'}, inplace=True)
audjpy1.rename(columns={'3. low': 'low'}, inplace=True)
audjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audjpy1.index)
audjpy1 = audjpy1.sort_index()

audjpy1['time'] = np.arange(len(audjpy1.index))

audjpy1['lag_1'] = audjpy1['close'].shift(1)
audjpy1['lag_2'] = audjpy1['close'].shift(2)
audjpy1['lag_3'] = audjpy1['close'].shift(3)
audjpy1['lag_4'] = audjpy1['close'].shift(4)
audjpy1['lag_5'] = audjpy1['close'].shift(5)
audjpy1['lag_6'] = audjpy1['close'].shift(6)
audjpy1['lag_7'] = audjpy1['close'].shift(7)

audjpy1['rolling_mean'] = audjpy1['close'].rolling(window=7).mean()
audjpy1['expanding_mean'] = audjpy1['close'].expanding(2).mean()

audjpy1 = audjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audjpy1])

audjpy1.to_pickle('audjpy1.pkl')

audjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,JPY,94.605,94.986,92.600,93.081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,JPY,93.079,93.894,92.410,93.186,93.081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.133500,1
2022-06-15,AUD,JPY,93.186,93.932,92.930,93.759,93.186,93.081,NaN,NaN,NaN,NaN,NaN,NaN,93.342000,2
2022-06-16,AUD,JPY,93.751,94.538,91.950,93.216,93.759,93.186,93.081,NaN,NaN,NaN,NaN,NaN,93.310500,3
2022-06-17,AUD,JPY,93.215,94.569,92.944,93.580,93.216,93.759,93.186,93.081,NaN,NaN,NaN,NaN,93.364400,4
2022-06-20,AUD,JPY,93.585,94.355,93.360,93.904,93.580,93.216,93.759,93.186,93.081,NaN,NaN,NaN,93.454333,5
2022-06-21,AUD,JPY,93.894,95.322,93.770,95.236,93.904,93.580,93.216,93.759,93.186,93.081,NaN,93.708857,93.708857,6
2022-06-22,AUD,JPY,95.201,95.290,93.570,94.256,95.236,93.904,93.580,93.216,93.759,93.186,93.081,93.876714,93.777250,7
2022-06-23,AUD,JPY,94.255,94.333,92.620,93.052,94.256,95.236,93.904,93.580,93.216,93.759,93.186,93.857571,93.696667,8


In [43]:
audgbp = app.get_currency_exchange_daily('AUD', 'GBP')
audgbp1 = audgbp[0]
audgbp1['to symbol'] = 'GBP'
audgbp1['from symbol'] = 'AUD'

audgbp1.rename(columns={'1. open': 'open'}, inplace=True)
audgbp1.rename(columns={'2. high': 'high'}, inplace=True)
audgbp1.rename(columns={'3. low': 'low'}, inplace=True)
audgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audgbp1.index)
audgbp1 = audgbp1.sort_index()

audgbp1['time'] = np.arange(len(audgbp1.index))

audgbp1['lag_1'] = audgbp1['close'].shift(1)
audgbp1['lag_2'] = audgbp1['close'].shift(2)
audgbp1['lag_3'] = audgbp1['close'].shift(3)
audgbp1['lag_4'] = audgbp1['close'].shift(4)
audgbp1['lag_5'] = audgbp1['close'].shift(5)
audgbp1['lag_6'] = audgbp1['close'].shift(6)
audgbp1['lag_7'] = audgbp1['close'].shift(7)

audgbp1['rolling_mean'] = audgbp1['close'].rolling(window=7).mean()
audgbp1['expanding_mean'] = audgbp1['close'].expanding(2).mean()

audgbp1 = audgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audgbp1])

audgbp1.to_pickle('audgbp1.pkl')

audgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,GBP,0.57029,0.57340,0.56817,0.57049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,GBP,0.57050,0.57399,0.56880,0.57261,0.57049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571550,1
2022-06-15,AUD,GBP,0.57258,0.57661,0.57125,0.57526,0.57261,0.57049,NaN,NaN,NaN,NaN,NaN,NaN,0.572787,2
2022-06-16,AUD,GBP,0.57512,0.57751,0.56580,0.56844,0.57526,0.57261,0.57049,NaN,NaN,NaN,NaN,NaN,0.571700,3
2022-06-17,AUD,GBP,0.56853,0.57155,0.56600,0.56670,0.56844,0.57526,0.57261,0.57049,NaN,NaN,NaN,NaN,0.570700,4
2022-06-20,AUD,GBP,0.56565,0.57131,0.56557,0.56710,0.56670,0.56844,0.57526,0.57261,0.57049,NaN,NaN,NaN,0.570100,5
2022-06-21,AUD,GBP,0.56748,0.56940,0.56530,0.56649,0.56710,0.56670,0.56844,0.57526,0.57261,0.57049,NaN,0.569584,0.569584,6
2022-06-22,AUD,GBP,0.56658,0.56762,0.56210,0.56365,0.56649,0.56710,0.56670,0.56844,0.57526,0.57261,0.57049,0.568607,0.568843,7
2022-06-23,AUD,GBP,0.56403,0.56515,0.56040,0.56200,0.56365,0.56649,0.56710,0.56670,0.56844,0.57526,0.57261,0.567091,0.568082,8


In [44]:
audchf = app.get_currency_exchange_daily('AUD', 'CHF')
audchf1 = audchf[0]
audchf1['to symbol'] = 'CHF'
audchf1['from symbol'] = 'AUD'

audchf1.rename(columns={'1. open': 'open'}, inplace=True)
audchf1.rename(columns={'2. high': 'high'}, inplace=True)
audchf1.rename(columns={'3. low': 'low'}, inplace=True)
audchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audchf1.index)
audchf1 = audchf1.sort_index()

audchf1['time'] = np.arange(len(audchf1.index))

audchf1['lag_1'] = audchf1['close'].shift(1)
audchf1['lag_2'] = audchf1['close'].shift(2)
audchf1['lag_3'] = audchf1['close'].shift(3)
audchf1['lag_4'] = audchf1['close'].shift(4)
audchf1['lag_5'] = audchf1['close'].shift(5)
audchf1['lag_6'] = audchf1['close'].shift(6)
audchf1['lag_7'] = audchf1['close'].shift(7)

audchf1['rolling_mean'] = audchf1['close'].rolling(window=7).mean()
audchf1['expanding_mean'] = audchf1['close'].expanding(2).mean()

audchf1 = audchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audchf1])

audchf1.to_pickle('audchf1.pkl')

audchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,CHF,0.69535,0.69604,0.6887,0.69034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,CHF,0.69052,0.69394,0.6832,0.68853,0.69034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.689435,1
2022-06-15,AUD,CHF,0.68867,0.69847,0.6871,0.69609,0.68853,0.69034,NaN,NaN,NaN,NaN,NaN,NaN,0.691653,2
2022-06-16,AUD,CHF,0.69628,0.70022,0.6736,0.68109,0.69609,0.68853,0.69034,NaN,NaN,NaN,NaN,NaN,0.689013,3
2022-06-17,AUD,CHF,0.68047,0.68190,0.6697,0.67173,0.68109,0.69609,0.68853,0.69034,NaN,NaN,NaN,NaN,0.685556,4
2022-06-20,AUD,CHF,0.67232,0.67559,0.6695,0.67180,0.67173,0.68109,0.69609,0.68853,0.69034,NaN,NaN,NaN,0.683263,5
2022-06-21,AUD,CHF,0.67224,0.67507,0.6702,0.67291,0.67180,0.67173,0.68109,0.69609,0.68853,0.69034,NaN,0.681784,0.681784,6
2022-06-22,AUD,CHF,0.67288,0.67364,0.6636,0.66540,0.67291,0.67180,0.67173,0.68109,0.69609,0.68853,0.69034,0.678221,0.679736,7
2022-06-23,AUD,CHF,0.66545,0.66623,0.6596,0.66288,0.66540,0.67291,0.67180,0.67173,0.68109,0.69609,0.68853,0.674557,0.677863,8


In [45]:
audcad = app.get_currency_exchange_daily('AUD', 'CAD')
audcad1 = audcad[0]
audcad1['to symbol'] = 'CAD'
audcad1['from symbol'] = 'AUD'

audcad1.rename(columns={'1. open': 'open'}, inplace=True)
audcad1.rename(columns={'2. high': 'high'}, inplace=True)
audcad1.rename(columns={'3. low': 'low'}, inplace=True)
audcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audcad1.index)
audcad1 = audcad1.sort_index()

audcad1['time'] = np.arange(len(audcad1.index))

audcad1['lag_1'] = audcad1['close'].shift(1)
audcad1['lag_2'] = audcad1['close'].shift(2)
audcad1['lag_3'] = audcad1['close'].shift(3)
audcad1['lag_4'] = audcad1['close'].shift(4)
audcad1['lag_5'] = audcad1['close'].shift(5)
audcad1['lag_6'] = audcad1['close'].shift(6)
audcad1['lag_7'] = audcad1['close'].shift(7)

audcad1['rolling_mean'] = audcad1['close'].rolling(window=7).mean()
audcad1['expanding_mean'] = audcad1['close'].expanding(2).mean()

audcad1 = audcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audcad1])

audcad1.to_pickle('audcad1.pkl')

audcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,CAD,0.89955,0.90094,0.89016,0.89312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,CAD,0.89307,0.89733,0.88747,0.89045,0.89312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.891785,1
2022-06-15,AUD,CAD,0.89055,0.90559,0.88955,0.90253,0.89045,0.89312,NaN,NaN,NaN,NaN,NaN,NaN,0.895367,2
2022-06-16,AUD,CAD,0.90263,0.91311,0.89894,0.91226,0.90253,0.89045,0.89312,NaN,NaN,NaN,NaN,NaN,0.899590,3
2022-06-17,AUD,CAD,0.91242,0.91286,0.90096,0.90233,0.91226,0.90253,0.89045,0.89312,NaN,NaN,NaN,NaN,0.900138,4
2022-06-20,AUD,CAD,0.90290,0.90882,0.90083,0.90231,0.90233,0.91226,0.90253,0.89045,0.89312,NaN,NaN,NaN,0.900500,5
2022-06-21,AUD,CAD,0.90215,0.90550,0.89798,0.90004,0.90231,0.90233,0.91226,0.90253,0.89045,0.89312,NaN,0.900434,0.900434,6
2022-06-22,AUD,CAD,0.89999,0.90080,0.89299,0.89634,0.90004,0.90231,0.90233,0.91226,0.90253,0.89045,0.89312,0.900894,0.899922,7
2022-06-23,AUD,CAD,0.89643,0.89699,0.89124,0.89592,0.89634,0.90004,0.90231,0.90233,0.91226,0.90253,0.89045,0.901676,0.899478,8


In [46]:
audhkd = app.get_currency_exchange_daily('AUD', 'HKD')
audhkd1 = audhkd[0]
audhkd1['to symbol'] = 'HKD'
audhkd1['from symbol'] = 'AUD'

audhkd1.rename(columns={'1. open': 'open'}, inplace=True)
audhkd1.rename(columns={'2. high': 'high'}, inplace=True)
audhkd1.rename(columns={'3. low': 'low'}, inplace=True)
audhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audhkd1.index)
audhkd1 = audhkd1.sort_index()

audhkd1['time'] = np.arange(len(audhkd1.index))

audhkd1['lag_1'] = audhkd1['close'].shift(1)
audhkd1['lag_2'] = audhkd1['close'].shift(2)
audhkd1['lag_3'] = audhkd1['close'].shift(3)
audhkd1['lag_4'] = audhkd1['close'].shift(4)
audhkd1['lag_5'] = audhkd1['close'].shift(5)
audhkd1['lag_6'] = audhkd1['close'].shift(6)
audhkd1['lag_7'] = audhkd1['close'].shift(7)

audhkd1['rolling_mean'] = audhkd1['close'].rolling(window=7).mean()
audhkd1['expanding_mean'] = audhkd1['close'].expanding(2).mean()

audhkd1 = audhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audhkd1])

audhkd1.to_pickle('audhkd1.pkl')

audhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,HKD,5.5220,5.5220,5.4210,5.4362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,HKD,5.4369,5.4711,5.3755,5.3959,5.4362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.416050,1
2022-06-15,AUD,HKD,5.3951,5.5128,5.3920,5.4990,5.3959,5.4362,NaN,NaN,NaN,NaN,NaN,NaN,5.443700,2
2022-06-16,AUD,HKD,5.4990,5.5484,5.4375,5.5306,5.4990,5.3959,5.4362,NaN,NaN,NaN,NaN,NaN,5.465425,3
2022-06-17,AUD,HKD,5.5306,5.5339,5.4040,5.4402,5.5306,5.4990,5.3959,5.4362,NaN,NaN,NaN,NaN,5.460380,4
2022-06-20,AUD,HKD,5.4414,5.4911,5.4230,5.4565,5.4402,5.5306,5.4990,5.3959,5.4362,NaN,NaN,NaN,5.459733,5
2022-06-21,AUD,HKD,5.4565,5.4893,5.4305,5.4699,5.4565,5.4402,5.5306,5.4990,5.3959,5.4362,NaN,5.461186,5.461186,6
2022-06-22,AUD,HKD,5.4706,5.4727,5.3875,5.4337,5.4699,5.4565,5.4402,5.5306,5.4990,5.3959,5.4362,5.460829,5.457750,7
2022-06-23,AUD,HKD,5.4355,5.4375,5.3780,5.4108,5.4337,5.4699,5.4565,5.4402,5.5306,5.4990,5.3959,5.462957,5.452533,8


In [47]:
audsek = app.get_currency_exchange_daily('AUD', 'SEK')
audsek1 = audsek[0]
audsek1['to symbol'] = 'SEK'
audsek1['from symbol'] = 'AUD'

audsek1.rename(columns={'1. open': 'open'}, inplace=True)
audsek1.rename(columns={'2. high': 'high'}, inplace=True)
audsek1.rename(columns={'3. low': 'low'}, inplace=True)
audsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audsek1.index)
audsek1 = audsek1.sort_index()

audsek1['time'] = np.arange(len(audsek1.index))

audsek1['lag_1'] = audsek1['close'].shift(1)
audsek1['lag_2'] = audsek1['close'].shift(2)
audsek1['lag_3'] = audsek1['close'].shift(3)
audsek1['lag_4'] = audsek1['close'].shift(4)
audsek1['lag_5'] = audsek1['close'].shift(5)
audsek1['lag_6'] = audsek1['close'].shift(6)
audsek1['lag_7'] = audsek1['close'].shift(7)

audsek1['rolling_mean'] = audsek1['close'].rolling(window=7).mean()
audsek1['expanding_mean'] = audsek1['close'].expanding(2).mean()

audsek1 = audsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audsek1])

audsek1.to_pickle('audsek1.pkl')

audsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,SEK,7.0367,7.0898,7.0240,7.0559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,SEK,7.0553,7.0792,6.9805,7.0010,7.0559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.028450,1
2022-06-15,AUD,SEK,6.9990,7.1537,6.9887,7.1156,7.0010,7.0559,NaN,NaN,NaN,NaN,NaN,NaN,7.057500,2
2022-06-16,AUD,SEK,7.1165,7.1811,7.0963,7.1243,7.1156,7.0010,7.0559,NaN,NaN,NaN,NaN,NaN,7.074200,3
2022-06-17,AUD,SEK,7.1229,7.1528,7.0376,7.0502,7.1243,7.1156,7.0010,7.0559,NaN,NaN,NaN,NaN,7.069400,4
2022-06-20,AUD,SEK,7.0422,7.1006,7.0236,7.0287,7.0502,7.1243,7.1156,7.0010,7.0559,NaN,NaN,NaN,7.062617,5
2022-06-21,AUD,SEK,7.0258,7.0556,6.9914,7.0292,7.0287,7.0502,7.1243,7.1156,7.0010,7.0559,NaN,7.057843,7.057843,6
2022-06-22,AUD,SEK,7.0315,7.0472,6.9692,6.9745,7.0292,7.0287,7.0502,7.1243,7.1156,7.0010,7.0559,7.046214,7.047425,7
2022-06-23,AUD,SEK,6.9742,7.0474,6.9473,7.0140,6.9745,7.0292,7.0287,7.0502,7.1243,7.1156,7.0010,7.048071,7.043711,8


In [48]:
audnzd = app.get_currency_exchange_daily('AUD', 'NZD')
audnzd1 = audnzd[0]
audnzd1['to symbol'] = 'NZD'
audnzd1['from symbol'] = 'AUD'

audnzd1.rename(columns={'1. open': 'open'}, inplace=True)
audnzd1.rename(columns={'2. high': 'high'}, inplace=True)
audnzd1.rename(columns={'3. low': 'low'}, inplace=True)
audnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(audnzd1.index)
audnzd1 = audnzd1.sort_index()

audnzd1['time'] = np.arange(len(audnzd1.index))

audnzd1['lag_1'] = audnzd1['close'].shift(1)
audnzd1['lag_2'] = audnzd1['close'].shift(2)
audnzd1['lag_3'] = audnzd1['close'].shift(3)
audnzd1['lag_4'] = audnzd1['close'].shift(4)
audnzd1['lag_5'] = audnzd1['close'].shift(5)
audnzd1['lag_6'] = audnzd1['close'].shift(6)
audnzd1['lag_7'] = audnzd1['close'].shift(7)

audnzd1['rolling_mean'] = audnzd1['close'].rolling(window=7).mean()
audnzd1['expanding_mean'] = audnzd1['close'].expanding(2).mean()

audnzd1 = audnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, audnzd1])

audnzd1.to_pickle('audnzd1.pkl')

audnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,AUD,NZD,1.10848,1.10986,1.10295,1.10544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,AUD,NZD,1.10544,1.10981,1.10229,1.10535,1.10544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.105395,1
2022-06-15,AUD,NZD,1.10510,1.11491,1.10462,1.11415,1.10535,1.10544,NaN,NaN,NaN,NaN,NaN,NaN,1.108313,2
2022-06-16,AUD,NZD,1.11432,1.11684,1.10069,1.10757,1.11415,1.10535,1.10544,NaN,NaN,NaN,NaN,NaN,1.108127,3
2022-06-17,AUD,NZD,1.10737,1.10889,1.09545,1.09770,1.10757,1.11415,1.10535,1.10544,NaN,NaN,NaN,NaN,1.106042,4
2022-06-20,AUD,NZD,1.09755,1.10067,1.09364,1.09795,1.09770,1.10757,1.11415,1.10535,1.10544,NaN,NaN,NaN,1.104693,5
2022-06-21,AUD,NZD,1.09773,1.10155,1.09626,1.10055,1.09795,1.09770,1.10757,1.11415,1.10535,1.10544,NaN,1.104101,1.104101,6
2022-06-22,AUD,NZD,1.09998,1.10511,1.09717,1.10098,1.10055,1.09795,1.09770,1.10757,1.11415,1.10535,1.10544,1.103464,1.103711,7
2022-06-23,AUD,NZD,1.10136,1.10307,1.09505,1.09833,1.10098,1.10055,1.09795,1.09770,1.10757,1.11415,1.10535,1.102461,1.103113,8


In [49]:
chfusd = app.get_currency_exchange_daily('CHF', 'USD')
chfusd1 = chfusd[0]
chfusd1['to symbol'] = 'USD'
chfusd1['from symbol'] = 'CHF'

chfusd1.rename(columns={'1. open': 'open'}, inplace=True)
chfusd1.rename(columns={'2. high': 'high'}, inplace=True)
chfusd1.rename(columns={'3. low': 'low'}, inplace=True)
chfusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfusd1.index)
chfusd1 = chfusd1.sort_index()

chfusd1['time'] = np.arange(len(chfusd1.index))

chfusd1['lag_1'] = chfusd1['close'].shift(1)
chfusd1['lag_2'] = chfusd1['close'].shift(2)
chfusd1['lag_3'] = chfusd1['close'].shift(3)
chfusd1['lag_4'] = chfusd1['close'].shift(4)
chfusd1['lag_5'] = chfusd1['close'].shift(5)
chfusd1['lag_6'] = chfusd1['close'].shift(6)
chfusd1['lag_7'] = chfusd1['close'].shift(7)

chfusd1['rolling_mean'] = chfusd1['close'].rolling(window=7).mean()
chfusd1['expanding_mean'] = chfusd1['close'].expanding(2).mean()

chfusd1 = chfusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfusd1])

chfusd1.to_pickle('chfusd1.pkl')

chfusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,USD,1.01083,1.01189,1.00030,1.00140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,USD,1.00093,1.01250,0.99577,0.99610,1.00140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998750,1
2022-06-15,CHF,USD,0.99761,1.00670,0.99488,1.00530,0.99610,1.00140,NaN,NaN,NaN,NaN,NaN,NaN,1.000933,2
2022-06-16,CHF,USD,1.00449,1.03794,1.00080,1.03470,1.00530,0.99610,1.00140,NaN,NaN,NaN,NaN,NaN,1.009375,3
2022-06-17,CHF,USD,1.03386,1.03924,1.02736,1.03090,1.03470,1.00530,0.99610,1.00140,NaN,NaN,NaN,NaN,1.013680,4
2022-06-20,CHF,USD,1.02935,1.03905,1.02880,1.03204,1.03090,1.03470,1.00530,0.99610,1.0014,NaN,NaN,NaN,1.016740,5
2022-06-21,CHF,USD,1.03265,1.03802,1.03240,1.03380,1.03204,1.03090,1.03470,1.00530,0.9961,1.0014,NaN,1.019177,1.019177,6
2022-06-22,CHF,USD,1.03450,1.04356,1.03130,1.03913,1.03380,1.03204,1.03090,1.03470,1.0053,0.9961,1.0014,1.024567,1.021671,7
2022-06-23,CHF,USD,1.03925,1.04463,1.03280,1.03970,1.03913,1.03380,1.03204,1.03090,1.0347,1.0053,0.9961,1.030796,1.023674,8


In [50]:
chfeur = app.get_currency_exchange_daily('CHF', 'EUR')
chfeur1 = chfeur[0]
chfeur1['to symbol'] = 'EUR'
chfeur1['from symbol'] = 'CHF'

chfeur1.rename(columns={'1. open': 'open'}, inplace=True)
chfeur1.rename(columns={'2. high': 'high'}, inplace=True)
chfeur1.rename(columns={'3. low': 'low'}, inplace=True)
chfeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfeur1.index)
chfeur1 = chfeur1.sort_index()

chfeur1['time'] = np.arange(len(chfeur1.index))

chfeur1['lag_1'] = chfeur1['close'].shift(1)
chfeur1['lag_2'] = chfeur1['close'].shift(2)
chfeur1['lag_3'] = chfeur1['close'].shift(3)
chfeur1['lag_4'] = chfeur1['close'].shift(4)
chfeur1['lag_5'] = chfeur1['close'].shift(5)
chfeur1['lag_6'] = chfeur1['close'].shift(6)
chfeur1['lag_7'] = chfeur1['close'].shift(7)

chfeur1['rolling_mean'] = chfeur1['close'].rolling(window=7).mean()
chfeur1['expanding_mean'] = chfeur1['close'].expanding(2).mean()

chfeur1 = chfeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfeur1])

chfeur1.to_pickle('chfeur1.pkl')

chfeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,EUR,0.9623,0.9644,0.9603,0.9623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,EUR,0.9625,0.9664,0.9562,0.9576,0.9623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.959950,1
2022-06-15,CHF,EUR,0.9582,0.9627,0.9539,0.9624,0.9576,0.9623,NaN,NaN,NaN,NaN,NaN,NaN,0.960767,2
2022-06-16,CHF,EUR,0.9620,0.9863,0.9596,0.9795,0.9624,0.9576,0.9623,NaN,NaN,NaN,NaN,NaN,0.965450,3
2022-06-17,CHF,EUR,0.9803,0.9895,0.9780,0.9815,0.9795,0.9624,0.9576,0.9623,NaN,NaN,NaN,NaN,0.968660,4
2022-06-20,CHF,EUR,0.9823,0.9863,0.9806,0.9829,0.9815,0.9795,0.9624,0.9576,0.9623,NaN,NaN,NaN,0.971033,5
2022-06-21,CHF,EUR,0.9829,0.9831,0.9783,0.9820,0.9829,0.9815,0.9795,0.9624,0.9576,0.9623,NaN,0.972600,0.972600,6
2022-06-22,CHF,EUR,0.9821,0.9862,0.9811,0.9840,0.9820,0.9829,0.9815,0.9795,0.9624,0.9576,0.9623,0.975700,0.974025,7
2022-06-23,CHF,EUR,0.9838,0.9924,0.9832,0.9880,0.9840,0.9820,0.9829,0.9815,0.9795,0.9624,0.9576,0.980043,0.975578,8


In [51]:
chfjpy = app.get_currency_exchange_daily('CHF', 'JPY')
chfjpy1 = chfjpy[0]
chfjpy1['to symbol'] = 'JPY'
chfjpy1['from symbol'] = 'CHF'

chfjpy1.rename(columns={'1. open': 'open'}, inplace=True)
chfjpy1.rename(columns={'2. high': 'high'}, inplace=True)
chfjpy1.rename(columns={'3. low': 'low'}, inplace=True)
chfjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfjpy1.index)
chfjpy1 = chfjpy1.sort_index()

chfjpy1['time'] = np.arange(len(chfjpy1.index))

chfjpy1['lag_1'] = chfjpy1['close'].shift(1)
chfjpy1['lag_2'] = chfjpy1['close'].shift(2)
chfjpy1['lag_3'] = chfjpy1['close'].shift(3)
chfjpy1['lag_4'] = chfjpy1['close'].shift(4)
chfjpy1['lag_5'] = chfjpy1['close'].shift(5)
chfjpy1['lag_6'] = chfjpy1['close'].shift(6)
chfjpy1['lag_7'] = chfjpy1['close'].shift(7)

chfjpy1['rolling_mean'] = chfjpy1['close'].rolling(window=7).mean()
chfjpy1['expanding_mean'] = chfjpy1['close'].expanding(2).mean()

chfjpy1 = chfjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfjpy1])

chfjpy1.to_pickle('chfjpy1.pkl')

chfjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,JPY,136.028,136.622,134.154,134.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,JPY,134.700,135.916,134.393,135.280,134.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.990000,1
2022-06-15,CHF,JPY,135.330,135.415,134.013,134.620,135.280,134.700,NaN,NaN,NaN,NaN,NaN,NaN,134.866667,2
2022-06-16,CHF,JPY,134.620,137.197,134.493,136.900,134.620,135.280,134.700,NaN,NaN,NaN,NaN,NaN,135.375000,3
2022-06-17,CHF,JPY,136.930,139.991,136.639,139.161,136.900,134.620,135.280,134.700,NaN,NaN,NaN,NaN,136.132200,4
2022-06-20,CHF,JPY,139.090,140.187,139.090,139.640,139.161,136.900,134.620,135.280,134.700,NaN,NaN,NaN,136.716833,5
2022-06-21,CHF,JPY,139.599,141.592,139.506,141.387,139.640,139.161,136.900,134.620,135.280,134.70,NaN,137.384000,137.384000,6
2022-06-22,CHF,JPY,141.382,141.886,140.610,141.620,141.387,139.640,139.161,136.900,134.620,135.28,134.70,138.372571,137.913500,7
2022-06-23,CHF,JPY,141.596,141.732,139.840,140.328,141.620,141.387,139.640,139.161,136.900,134.62,135.28,139.093714,138.181778,8


In [52]:
chfgbp = app.get_currency_exchange_daily('CHF', 'GBP')
chfgbp1 = chfgbp[0]
chfgbp1['to symbol'] = 'GBP'
chfgbp1['from symbol'] = 'CHF'

chfgbp1.rename(columns={'1. open': 'open'}, inplace=True)
chfgbp1.rename(columns={'2. high': 'high'}, inplace=True)
chfgbp1.rename(columns={'3. low': 'low'}, inplace=True)
chfgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfgbp1.index)
chfgbp1 = chfgbp1.sort_index()

chfgbp1['time'] = np.arange(len(chfgbp1.index))

chfgbp1['lag_1'] = chfgbp1['close'].shift(1)
chfgbp1['lag_2'] = chfgbp1['close'].shift(2)
chfgbp1['lag_3'] = chfgbp1['close'].shift(3)
chfgbp1['lag_4'] = chfgbp1['close'].shift(4)
chfgbp1['lag_5'] = chfgbp1['close'].shift(5)
chfgbp1['lag_6'] = chfgbp1['close'].shift(6)
chfgbp1['lag_7'] = chfgbp1['close'].shift(7)

chfgbp1['rolling_mean'] = chfgbp1['close'].rolling(window=7).mean()
chfgbp1['expanding_mean'] = chfgbp1['close'].expanding(2).mean()

chfgbp1 = chfgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfgbp1])

chfgbp1.to_pickle('chfgbp1.pkl')

chfgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,GBP,0.8216,0.8271,0.8212,0.8254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,GBP,0.8254,0.8351,0.8231,0.8313,0.8254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.828350,1
2022-06-15,CHF,GBP,0.8319,0.8328,0.8237,0.8259,0.8313,0.8254,NaN,NaN,NaN,NaN,NaN,NaN,0.827533,2
2022-06-16,CHF,GBP,0.8255,0.8460,0.8239,0.8370,0.8259,0.8313,0.8254,NaN,NaN,NaN,NaN,NaN,0.829900,3
2022-06-17,CHF,GBP,0.8376,0.8473,0.8363,0.8429,0.8370,0.8259,0.8313,0.8254,NaN,NaN,NaN,NaN,0.832500,4
2022-06-20,CHF,GBP,0.8428,0.8478,0.8419,0.8435,0.8429,0.8370,0.8259,0.8313,0.8254,NaN,NaN,NaN,0.834333,5
2022-06-21,CHF,GBP,0.8434,0.8452,0.8404,0.8424,0.8435,0.8429,0.8370,0.8259,0.8313,0.8254,NaN,0.835486,0.835486,6
2022-06-22,CHF,GBP,0.8425,0.8488,0.8424,0.8476,0.8424,0.8435,0.8429,0.8370,0.8259,0.8313,0.8254,0.838657,0.837000,7
2022-06-23,CHF,GBP,0.8475,0.8516,0.8459,0.8479,0.8476,0.8424,0.8435,0.8429,0.8370,0.8259,0.8313,0.841029,0.838211,8


In [53]:
chfaud = app.get_currency_exchange_daily('CHF', 'AUD')
chfaud1 = chfaud[0]
chfaud1['to symbol'] = 'AUD'
chfaud1['from symbol'] = 'CHF'

chfaud1.rename(columns={'1. open': 'open'}, inplace=True)
chfaud1.rename(columns={'2. high': 'high'}, inplace=True)
chfaud1.rename(columns={'3. low': 'low'}, inplace=True)
chfaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfaud1.index)
chfaud1 = chfaud1.sort_index()
chfaud1
chfaud1['time'] = np.arange(len(chfaud1.index))

chfaud1['lag_1'] = chfaud1['close'].shift(1)
chfaud1['lag_2'] = chfaud1['close'].shift(2)
chfaud1['lag_3'] = chfaud1['close'].shift(3)
chfaud1['lag_4'] = chfaud1['close'].shift(4)
chfaud1['lag_5'] = chfaud1['close'].shift(5)
chfaud1['lag_6'] = chfaud1['close'].shift(6)
chfaud1['lag_7'] = chfaud1['close'].shift(7)

chfaud1['rolling_mean'] = chfaud1['close'].rolling(window=7).mean()
chfaud1['expanding_mean'] = chfaud1['close'].expanding(2).mean()

chfaud1 = chfaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfaud1])

chfaud1.to_pickle('chfaud1.pkl')

chfaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,AUD,1.43398,1.44830,1.43324,1.44630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,AUD,1.44175,1.46131,1.44082,1.45100,1.44630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.448650,1
2022-06-15,CHF,AUD,1.44728,1.45290,1.43092,1.43520,1.45100,1.44630,NaN,NaN,NaN,NaN,NaN,NaN,1.444167,2
2022-06-16,CHF,AUD,1.43074,1.48225,1.42750,1.46760,1.43520,1.45100,1.44630,NaN,NaN,NaN,NaN,NaN,1.450025,3
2022-06-17,CHF,AUD,1.46840,1.49067,1.46323,1.48650,1.46760,1.43520,1.45100,1.4463,NaN,NaN,NaN,NaN,1.457320,4
2022-06-20,CHF,AUD,1.48560,1.49110,1.47759,1.48600,1.48650,1.46760,1.43520,1.4510,1.4463,NaN,NaN,NaN,1.462100,5
2022-06-21,CHF,AUD,1.48550,1.48940,1.48008,1.48028,1.48600,1.48650,1.46760,1.4352,1.4510,1.4463,NaN,1.464697,1.464697,6
2022-06-22,CHF,AUD,1.48450,1.50370,1.48030,1.50160,1.48028,1.48600,1.48650,1.4676,1.4352,1.4510,1.4463,1.472597,1.469310,7
2022-06-23,CHF,AUD,1.49784,1.51360,1.49784,1.50760,1.50160,1.48028,1.48600,1.4865,1.4676,1.4352,1.4510,1.480683,1.473564,8


In [54]:
chfcad = app.get_currency_exchange_daily('CHF', 'CAD')
chfcad1 = chfcad[0]
chfcad1['to symbol'] = 'CAD'
chfcad1['from symbol'] = 'CHF'

chfcad1.rename(columns={'1. open': 'open'}, inplace=True)
chfcad1.rename(columns={'2. high': 'high'}, inplace=True)
chfcad1.rename(columns={'3. low': 'low'}, inplace=True)
chfcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfcad1.index)
chfcad1 = chfcad1.sort_index()

chfcad1['time'] = np.arange(len(chfcad1.index))

chfcad1['lag_1'] = chfcad1['close'].shift(1)
chfcad1['lag_2'] = chfcad1['close'].shift(2)
chfcad1['lag_3'] = chfcad1['close'].shift(3)
chfcad1['lag_4'] = chfcad1['close'].shift(4)
chfcad1['lag_5'] = chfcad1['close'].shift(5)
chfcad1['lag_6'] = chfcad1['close'].shift(6)
chfcad1['lag_7'] = chfcad1['close'].shift(7)

chfcad1['rolling_mean'] = chfcad1['close'].rolling(window=7).mean()
chfcad1['expanding_mean'] = chfcad1['close'].expanding(2).mean()

chfcad1 = chfcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfcad1])

chfcad1.to_pickle('chfcad1.pkl')

chfcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,CAD,1.2933,1.2960,1.2881,1.2912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,CAD,1.2916,1.3060,1.2894,1.2917,1.2912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.291450,1
2022-06-15,CHF,CAD,1.2925,1.2980,1.2908,1.2952,1.2917,1.2912,NaN,NaN,NaN,NaN,NaN,NaN,1.292700,2
2022-06-16,CHF,CAD,1.2951,1.3404,1.2903,1.3389,1.2952,1.2917,1.2912,NaN,NaN,NaN,NaN,NaN,1.304250,3
2022-06-17,CHF,CAD,1.3392,1.3507,1.3330,1.3423,1.3389,1.2952,1.2917,1.2912,NaN,NaN,NaN,NaN,1.311860,4
2022-06-20,CHF,CAD,1.3421,1.3493,1.3390,1.3412,1.3423,1.3389,1.2952,1.2917,1.2912,NaN,NaN,NaN,1.316750,5
2022-06-21,CHF,CAD,1.3410,1.3419,1.3346,1.3362,1.3412,1.3423,1.3389,1.2952,1.2917,1.2912,NaN,1.319529,1.319529,6
2022-06-22,CHF,CAD,1.3364,1.3485,1.3364,1.3459,1.3362,1.3412,1.3423,1.3389,1.2952,1.2917,1.2912,1.327343,1.322825,7
2022-06-23,CHF,CAD,1.3460,1.3535,1.3401,1.3514,1.3459,1.3362,1.3412,1.3423,1.3389,1.2952,1.2917,1.335871,1.326000,8


In [56]:
chfhkd = app.get_currency_exchange_daily('CHF', 'HKD')
chfhkd1 = chfhkd[0]
chfhkd1['to symbol'] = 'HKD'
chfhkd1['from symbol'] = 'CHF'

chfhkd1.rename(columns={'1. open': 'open'}, inplace=True)
chfhkd1.rename(columns={'2. high': 'high'}, inplace=True)
chfhkd1.rename(columns={'3. low': 'low'}, inplace=True)
chfhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfhkd1.index)
chfhkd1 = chfhkd1.sort_index()

chfhkd1['time'] = np.arange(len(chfhkd1.index))

chfhkd1['lag_1'] = chfhkd1['close'].shift(1)
chfhkd1['lag_2'] = chfhkd1['close'].shift(2)
chfhkd1['lag_3'] = chfhkd1['close'].shift(3)
chfhkd1['lag_4'] = chfhkd1['close'].shift(4)
chfhkd1['lag_5'] = chfhkd1['close'].shift(5)
chfhkd1['lag_6'] = chfhkd1['close'].shift(6)
chfhkd1['lag_7'] = chfhkd1['close'].shift(7)

chfhkd1['rolling_mean'] = chfhkd1['close'].rolling(window=7).mean()
chfhkd1['expanding_mean'] = chfhkd1['close'].expanding(2).mean()

chfhkd1 = chfhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfhkd1])

chfhkd1.to_pickle('chfhkd1.pkl')

chfhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,HKD,7.9434,7.9455,7.8485,7.8672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,HKD,7.8691,7.9486,7.8115,7.8339,7.8672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.850550,1
2022-06-15,CHF,HKD,7.8391,7.9033,7.8080,7.8955,7.8339,7.8672,NaN,NaN,NaN,NaN,NaN,NaN,7.865533,2
2022-06-16,CHF,HKD,7.8933,8.1482,7.8395,8.1207,7.8955,7.8339,7.8672,NaN,NaN,NaN,NaN,NaN,7.929325,3
2022-06-17,CHF,HKD,8.1257,8.1587,8.0520,8.0930,8.1207,7.8955,7.8339,7.8672,NaN,NaN,NaN,NaN,7.962060,4
2022-06-20,CHF,HKD,8.0906,8.1565,8.0700,8.1152,8.0930,8.1207,7.8955,7.8339,7.8672,NaN,NaN,NaN,7.987583,5
2022-06-21,CHF,HKD,8.1135,8.1488,8.0885,8.1262,8.1152,8.0930,8.1207,7.8955,7.8339,7.8672,NaN,8.007386,8.007386,6
2022-06-22,CHF,HKD,8.1255,8.1892,8.0830,8.1652,8.1262,8.1152,8.0930,8.1207,7.8955,7.8339,7.8672,8.049957,8.027112,7
2022-06-23,CHF,HKD,8.1643,8.1994,8.0935,8.1635,8.1652,8.1262,8.1152,8.0930,8.1207,7.8955,7.8339,8.097043,8.042267,8


In [57]:
chfsek = app.get_currency_exchange_daily('CHF', 'SEK')
chfsek1 = chfsek[0]
chfsek1['to symbol'] = 'SEK'
chfsek1['from symbol'] = 'CHF'

chfsek1.rename(columns={'1. open': 'open'}, inplace=True)
chfsek1.rename(columns={'2. high': 'high'}, inplace=True)
chfsek1.rename(columns={'3. low': 'low'}, inplace=True)
chfsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfsek1.index)
chfsek1 = chfsek1.sort_index()

chfsek1['time'] = np.arange(len(chfsek1.index))

chfsek1['lag_1'] = chfsek1['close'].shift(1)
chfsek1['lag_2'] = chfsek1['close'].shift(2)
chfsek1['lag_3'] = chfsek1['close'].shift(3)
chfsek1['lag_4'] = chfsek1['close'].shift(4)
chfsek1['lag_5'] = chfsek1['close'].shift(5)
chfsek1['lag_6'] = chfsek1['close'].shift(6)
chfsek1['lag_7'] = chfsek1['close'].shift(7)

chfsek1['rolling_mean'] = chfsek1['close'].rolling(window=7).mean()
chfsek1['expanding_mean'] = chfsek1['close'].expanding(2).mean()

chfsek1 = chfsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfsek1])

chfsek1.to_pickle('chfsek1.pkl')

chfsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,SEK,10.1161,10.24200,10.10990,10.22140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,SEK,10.2099,10.26410,10.14570,10.18050,10.22140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.200950,1
2022-06-15,CHF,SEK,10.1733,10.26750,10.11950,10.23330,10.18050,10.22140,NaN,NaN,NaN,NaN,NaN,NaN,10.211733,2
2022-06-16,CHF,SEK,10.2329,10.56660,10.17490,10.47480,10.23330,10.18050,10.2214,NaN,NaN,NaN,NaN,NaN,10.277500,3
2022-06-17,CHF,SEK,10.4771,10.57384,10.45250,10.49080,10.47480,10.23330,10.1805,10.2214,NaN,NaN,NaN,NaN,10.320160,4
2022-06-20,CHF,SEK,10.4615,10.53530,10.43160,10.45350,10.49080,10.47480,10.2333,10.1805,10.2214,NaN,NaN,NaN,10.342383,5
2022-06-21,CHF,SEK,10.4466,10.46800,10.38800,10.43310,10.45350,10.49080,10.4748,10.2333,10.1805,10.2214,NaN,10.355343,10.355343,6
2022-06-22,CHF,SEK,10.4429,10.53560,10.43987,10.47498,10.43310,10.45350,10.4908,10.4748,10.2333,10.1805,10.2214,10.391569,10.370297,7
2022-06-23,CHF,SEK,10.4968,10.64340,10.47168,10.58890,10.47498,10.43310,10.4535,10.4908,10.4748,10.2333,10.1805,10.449911,10.394587,8


In [58]:
chfnzd = app.get_currency_exchange_daily('CHF', 'NZD')
chfnzd1 = chfnzd[0]
chfnzd1['to symbol'] = 'NZD'
chfnzd1['from symbol'] = 'CHF'

chfnzd1.rename(columns={'1. open': 'open'}, inplace=True)
chfnzd1.rename(columns={'2. high': 'high'}, inplace=True)
chfnzd1.rename(columns={'3. low': 'low'}, inplace=True)
chfnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(chfnzd1.index)
chfnzd1 = chfnzd1.sort_index()

chfnzd1['time'] = np.arange(len(chfnzd1.index))

chfnzd1['lag_1'] = chfnzd1['close'].shift(1)
chfnzd1['lag_2'] = chfnzd1['close'].shift(2)
chfnzd1['lag_3'] = chfnzd1['close'].shift(3)
chfnzd1['lag_4'] = chfnzd1['close'].shift(4)
chfnzd1['lag_5'] = chfnzd1['close'].shift(5)
chfnzd1['lag_6'] = chfnzd1['close'].shift(6)
chfnzd1['lag_7'] = chfnzd1['close'].shift(7)

chfnzd1['rolling_mean'] = chfnzd1['close'].rolling(window=7).mean()
chfnzd1['expanding_mean'] = chfnzd1['close'].expanding(2).mean()

chfnzd1 = chfnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, chfnzd1])

chfnzd1.to_pickle('chfnzd1.pkl')

chfnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CHF,NZD,1.5945,1.6038,1.5919,1.5996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CHF,NZD,1.5995,1.6164,1.5960,1.6048,1.5996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.602200,1
2022-06-15,CHF,NZD,1.6057,1.6093,1.5930,1.5998,1.6048,1.5996,NaN,NaN,NaN,NaN,NaN,NaN,1.601400,2
2022-06-16,CHF,NZD,1.5989,1.6394,1.5930,1.6254,1.5998,1.6048,1.5996,NaN,NaN,NaN,NaN,NaN,1.607400,3
2022-06-17,CHF,NZD,1.6273,1.6441,1.6232,1.6326,1.6254,1.5998,1.6048,1.5996,NaN,NaN,NaN,NaN,1.612440,4
2022-06-20,CHF,NZD,1.6325,1.6380,1.6265,1.6321,1.6326,1.6254,1.5998,1.6048,1.5996,NaN,NaN,NaN,1.615717,5
2022-06-21,CHF,NZD,1.6316,1.6354,1.6260,1.6342,1.6321,1.6326,1.6254,1.5998,1.6048,1.5996,NaN,1.618357,1.618357,6
2022-06-22,CHF,NZD,1.6341,1.6589,1.6341,1.6540,1.6342,1.6321,1.6326,1.6254,1.5998,1.6048,1.5996,1.626129,1.622812,7
2022-06-23,CHF,NZD,1.6544,1.6619,1.6494,1.6558,1.6540,1.6342,1.6321,1.6326,1.6254,1.5998,1.6048,1.633414,1.626478,8


In [59]:
cadusd = app.get_currency_exchange_daily('CAD', 'USD')
cadusd1 = cadusd[0]
cadusd1['to symbol'] = 'USD'
cadusd1['from symbol'] = 'CAD'

cadusd1.rename(columns={'1. open': 'open'}, inplace=True)
cadusd1.rename(columns={'2. high': 'high'}, inplace=True)
cadusd1.rename(columns={'3. low': 'low'}, inplace=True)
cadusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadusd1.index)
cadusd1 = cadusd1.sort_index()

cadusd1['time'] = np.arange(len(cadusd1.index))

cadusd1['lag_1'] = cadusd1['close'].shift(1)
cadusd1['lag_2'] = cadusd1['close'].shift(2)
cadusd1['lag_3'] = cadusd1['close'].shift(3)
cadusd1['lag_4'] = cadusd1['close'].shift(4)
cadusd1['lag_5'] = cadusd1['close'].shift(5)
cadusd1['lag_6'] = cadusd1['close'].shift(6)
cadusd1['lag_7'] = cadusd1['close'].shift(7)

cadusd1['rolling_mean'] = cadusd1['close'].rolling(window=7).mean()
cadusd1['expanding_mean'] = cadusd1['close'].expanding(2).mean()

cadusd1 = cadusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadusd1])

chfnzd1.to_pickle('chfnzd1.pkl')

cadusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,USD,0.78187,0.78187,0.77460,0.77511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,USD,0.77514,0.77709,0.77020,0.77125,0.77511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.773180,1
2022-06-15,CAD,USD,0.77149,0.77728,0.76910,0.77571,0.77125,0.77511,NaN,NaN,NaN,NaN,NaN,NaN,0.774023,2
2022-06-16,CAD,USD,0.77573,0.77741,0.77060,0.77170,0.77571,0.77125,0.77511,NaN,NaN,NaN,NaN,NaN,0.773443,3
2022-06-17,CAD,USD,0.77194,0.77268,0.76410,0.76700,0.77170,0.77571,0.77125,0.77511,NaN,NaN,NaN,NaN,0.772154,4
2022-06-20,CAD,USD,0.76618,0.77037,0.76618,0.76998,0.76700,0.77170,0.77571,0.77125,0.77511,NaN,NaN,NaN,0.771792,5
2022-06-21,CAD,USD,0.76999,0.77472,0.76950,0.77378,0.76998,0.76700,0.77170,0.77571,0.77125,0.77511,NaN,0.772076,0.772076,6
2022-06-22,CAD,USD,0.77380,0.77431,0.76900,0.77201,0.77378,0.76998,0.76700,0.77170,0.77571,0.77125,0.77511,0.771633,0.772068,7
2022-06-23,CAD,USD,0.77180,0.77286,0.76770,0.76885,0.77201,0.77378,0.76998,0.76700,0.77170,0.77571,0.77125,0.771290,0.771710,8


In [60]:
cadeur = app.get_currency_exchange_daily('CAD', 'EUR')
cadeur1 = cadeur[0]
cadeur1['to symbol'] = 'EUR'
cadeur1['from symbol'] = 'CAD'

cadeur1.rename(columns={'1. open': 'open'}, inplace=True)
cadeur1.rename(columns={'2. high': 'high'}, inplace=True)
cadeur1.rename(columns={'3. low': 'low'}, inplace=True)
cadeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadeur1.index)
cadeur1 = cadeur1.sort_index()

cadeur1['time'] = np.arange(len(cadeur1.index))

cadeur1['lag_1'] = cadeur1['close'].shift(1)
cadeur1['lag_2'] = cadeur1['close'].shift(2)
cadeur1['lag_3'] = cadeur1['close'].shift(3)
cadeur1['lag_4'] = cadeur1['close'].shift(4)
cadeur1['lag_5'] = cadeur1['close'].shift(5)
cadeur1['lag_6'] = cadeur1['close'].shift(6)
cadeur1['lag_7'] = cadeur1['close'].shift(7)

cadeur1['rolling_mean'] = cadeur1['close'].rolling(window=7).mean()
cadeur1['expanding_mean'] = cadeur1['close'].expanding(2).mean()

cadeur1 = cadeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadeur1])

cadeur1.to_pickle('cadeur1.pkl')

cadeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,EUR,0.7436,0.7458,0.7425,0.7446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,EUR,0.7448,0.7453,0.7391,0.7407,0.7446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.742650,1
2022-06-15,CAD,EUR,0.7409,0.7433,0.7350,0.7427,0.7407,0.7446,NaN,NaN,NaN,NaN,NaN,NaN,0.742667,2
2022-06-16,CAD,EUR,0.7423,0.7446,0.7312,0.7314,0.7427,0.7407,0.7446,NaN,NaN,NaN,NaN,NaN,0.739850,3
2022-06-17,CAD,EUR,0.7316,0.7343,0.7297,0.7308,0.7314,0.7427,0.7407,0.7446,NaN,NaN,NaN,NaN,0.738040,4
2022-06-20,CAD,EUR,0.7321,0.7330,0.7298,0.7325,0.7308,0.7314,0.7427,0.7407,0.7446,NaN,NaN,NaN,0.737117,5
2022-06-21,CAD,EUR,0.7324,0.7349,0.7309,0.7344,0.7325,0.7308,0.7314,0.7427,0.7407,0.7446,NaN,0.736729,0.736729,6
2022-06-22,CAD,EUR,0.7345,0.7349,0.7291,0.7307,0.7344,0.7325,0.7308,0.7314,0.7427,0.7407,0.7446,0.734743,0.735975,7
2022-06-23,CAD,EUR,0.7304,0.7352,0.7292,0.7308,0.7307,0.7344,0.7325,0.7308,0.7314,0.7427,0.7407,0.733329,0.735400,8


In [61]:
cadjpy = app.get_currency_exchange_daily('CAD', 'JPY')
cadjpy1 = cadjpy[0]
cadjpy1['to symbol'] = 'JPY'
cadjpy1['from symbol'] = 'CAD'

cadjpy1.rename(columns={'1. open': 'open'}, inplace=True)
cadjpy1.rename(columns={'2. high': 'high'}, inplace=True)
cadjpy1.rename(columns={'3. low': 'low'}, inplace=True)
cadjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadjpy1.index)
cadjpy1 = cadjpy1.sort_index()

cadjpy1['time'] = np.arange(len(cadjpy1.index))

cadjpy1['lag_1'] = cadjpy1['close'].shift(1)
cadjpy1['lag_2'] = cadjpy1['close'].shift(2)
cadjpy1['lag_3'] = cadjpy1['close'].shift(3)
cadjpy1['lag_4'] = cadjpy1['close'].shift(4)
cadjpy1['lag_5'] = cadjpy1['close'].shift(5)
cadjpy1['lag_6'] = cadjpy1['close'].shift(6)
cadjpy1['lag_7'] = cadjpy1['close'].shift(7)

cadjpy1['rolling_mean'] = cadjpy1['close'].rolling(window=7).mean()
cadjpy1['expanding_mean'] = cadjpy1['close'].expanding(2).mean()

cadjpy1 = cadjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadjpy1])

cadjpy1.to_pickle('cadjpy1.pkl')

cadjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,JPY,105.123,105.515,103.820,104.212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,JPY,104.213,104.703,103.673,104.597,104.212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.404500,1
2022-06-15,CAD,JPY,104.600,104.672,103.451,103.825,104.597,104.212,NaN,NaN,NaN,NaN,NaN,NaN,104.211333,2
2022-06-16,CAD,JPY,103.842,104.540,101.646,102.144,103.825,104.597,104.212,NaN,NaN,NaN,NaN,NaN,103.694500,3
2022-06-17,CAD,JPY,102.163,104.015,102.009,103.575,102.144,103.825,104.597,104.212,NaN,NaN,NaN,NaN,103.670600,4
2022-06-20,CAD,JPY,103.601,104.151,103.460,104.055,103.575,102.144,103.825,104.597,104.212,NaN,NaN,NaN,103.734667,5
2022-06-21,CAD,JPY,104.056,105.845,103.912,105.763,104.055,103.575,102.144,103.825,104.597,104.212,NaN,104.024429,104.024429,6
2022-06-22,CAD,JPY,105.759,105.813,104.588,105.116,105.763,104.055,103.575,102.144,103.825,104.597,104.212,104.153571,104.160875,7
2022-06-23,CAD,JPY,105.114,105.203,103.520,103.776,105.116,105.763,104.055,103.575,102.144,103.825,104.597,104.036286,104.118111,8


In [62]:
cadgbp = app.get_currency_exchange_daily('CAD', 'GBP')
cadgbp1 = cadgbp[0]
cadgbp1['to symbol'] = 'GBP'
cadgbp1['from symbol'] = 'CAD'

cadgbp1.rename(columns={'1. open': 'open'}, inplace=True)
cadgbp1.rename(columns={'2. high': 'high'}, inplace=True)
cadgbp1.rename(columns={'3. low': 'low'}, inplace=True)
cadgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadgbp1.index)
cadgbp1 = cadgbp1.sort_index()

cadgbp1['time'] = np.arange(len(cadgbp1.index))

cadgbp1['lag_1'] = cadgbp1['close'].shift(1)
cadgbp1['lag_2'] = cadgbp1['close'].shift(2)
cadgbp1['lag_3'] = cadgbp1['close'].shift(3)
cadgbp1['lag_4'] = cadgbp1['close'].shift(4)
cadgbp1['lag_5'] = cadgbp1['close'].shift(5)
cadgbp1['lag_6'] = cadgbp1['close'].shift(6)
cadgbp1['lag_7'] = cadgbp1['close'].shift(7)

cadgbp1['rolling_mean'] = cadgbp1['close'].rolling(window=7).mean()
cadgbp1['expanding_mean'] = cadgbp1['close'].expanding(2).mean()

cadgbp1 = cadgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadgbp1])

cadgbp1.to_pickle('cadgbp1.pkl')

cadgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,GBP,0.6347,0.6402,0.6343,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,GBP,0.6384,0.6450,0.6354,0.6427,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.640600,1
2022-06-15,CAD,GBP,0.6430,0.6431,0.6354,0.6371,0.6427,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,0.639433,2
2022-06-16,CAD,GBP,0.6368,0.6407,0.6241,0.6246,0.6371,0.6427,0.6385,NaN,NaN,NaN,NaN,NaN,0.635725,3
2022-06-17,CAD,GBP,0.6248,0.6286,0.6244,0.6274,0.6246,0.6371,0.6427,0.6385,NaN,NaN,NaN,NaN,0.634060,4
2022-06-20,CAD,GBP,0.6274,0.6292,0.6263,0.6284,0.6274,0.6246,0.6371,0.6427,0.6385,NaN,NaN,NaN,0.633117,5
2022-06-21,CAD,GBP,0.6283,0.6305,0.6279,0.6298,0.6284,0.6274,0.6246,0.6371,0.6427,0.6385,NaN,0.632643,0.632643,6
2022-06-22,CAD,GBP,0.6298,0.6323,0.6274,0.6292,0.6298,0.6284,0.6274,0.6246,0.6371,0.6427,0.6385,0.631314,0.632212,7
2022-06-23,CAD,GBP,0.6290,0.6326,0.6267,0.6269,0.6292,0.6298,0.6284,0.6274,0.6246,0.6371,0.6427,0.629057,0.631622,8


In [63]:
cadaud = app.get_currency_exchange_daily('CAD', 'AUD')
cadaud1 = cadaud[0]
cadaud1['to symbol'] = 'AUD'
cadaud1['from symbol'] = 'CAD'

cadaud1.rename(columns={'1. open': 'open'}, inplace=True)
cadaud1.rename(columns={'2. high': 'high'}, inplace=True)
cadaud1.rename(columns={'3. low': 'low'}, inplace=True)
cadaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadaud1.index)
cadaud1 = cadaud1.sort_index()

cadaud1['time'] = np.arange(len(cadaud1.index))

cadaud1['lag_1'] = cadaud1['close'].shift(1)
cadaud1['lag_2'] = cadaud1['close'].shift(2)
cadaud1['lag_3'] = cadaud1['close'].shift(3)
cadaud1['lag_4'] = cadaud1['close'].shift(4)
cadaud1['lag_5'] = cadaud1['close'].shift(5)
cadaud1['lag_6'] = cadaud1['close'].shift(6)
cadaud1['lag_7'] = cadaud1['close'].shift(7)

cadaud1['rolling_mean'] = cadaud1['close'].rolling(window=7).mean()
cadaud1['expanding_mean'] = cadaud1['close'].expanding(2).mean()

cadaud1 = cadaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadaud1])

cadaud1.to_pickle('cadaud1.pkl')

cadaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,AUD,1.1110,1.1218,1.1098,1.1188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,AUD,1.1188,1.1249,1.1139,1.1220,1.1188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.120400,1
2022-06-15,CAD,AUD,1.1224,1.1226,1.1041,1.1071,1.1220,1.1188,NaN,NaN,NaN,NaN,NaN,NaN,1.115967,2
2022-06-16,CAD,AUD,1.1070,1.1107,1.0946,1.0953,1.1071,1.1220,1.1188,NaN,NaN,NaN,NaN,NaN,1.110800,3
2022-06-17,CAD,AUD,1.0955,1.1082,1.0948,1.1065,1.0953,1.1071,1.1220,1.1188,NaN,NaN,NaN,NaN,1.109940,4
2022-06-20,CAD,AUD,1.1060,1.1083,1.0997,1.1071,1.1065,1.0953,1.1071,1.1220,1.1188,NaN,NaN,NaN,1.109467,5
2022-06-21,CAD,AUD,1.1068,1.1119,1.1039,1.1101,1.1071,1.1065,1.0953,1.1071,1.1220,1.1188,NaN,1.109557,1.109557,6
2022-06-22,CAD,AUD,1.1099,1.1181,1.1096,1.1147,1.1101,1.1071,1.1065,1.0953,1.1071,1.1220,1.1188,1.108971,1.110200,7
2022-06-23,CAD,AUD,1.1144,1.1203,1.1143,1.1148,1.1147,1.1101,1.1071,1.1065,1.0953,1.1071,1.1220,1.107943,1.110711,8


In [64]:
cadchf = app.get_currency_exchange_daily('CAD', 'CHF')
cadchf1 = cadchf[0]
cadchf1['to symbol'] = 'CHF'
cadchf1['from symbol'] = 'CAD'

cadchf1.rename(columns={'1. open': 'open'}, inplace=True)
cadchf1.rename(columns={'2. high': 'high'}, inplace=True)
cadchf1.rename(columns={'3. low': 'low'}, inplace=True)
cadchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadchf1.index)
cadchf1 = cadchf1.sort_index()

cadchf1['time'] = np.arange(len(cadchf1.index))

cadchf1['lag_1'] = cadchf1['close'].shift(1)
cadchf1['lag_2'] = cadchf1['close'].shift(2)
cadchf1['lag_3'] = cadchf1['close'].shift(3)
cadchf1['lag_4'] = cadchf1['close'].shift(4)
cadchf1['lag_5'] = cadchf1['close'].shift(5)
cadchf1['lag_6'] = cadchf1['close'].shift(6)
cadchf1['lag_7'] = cadchf1['close'].shift(7)

cadchf1['rolling_mean'] = cadchf1['close'].rolling(window=7).mean()
cadchf1['expanding_mean'] = cadchf1['close'].expanding(2).mean()

cadchf1 = cadchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadchf1])

cadchf1.to_pickle('cadchf1.pkl')

cadchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,CHF,0.77248,0.77616,0.77000,0.77319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,CHF,0.77308,0.77526,0.76410,0.77290,0.77319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.773045,1
2022-06-15,CAD,CHF,0.77297,0.77421,0.76870,0.77097,0.77290,0.77319,NaN,NaN,NaN,NaN,NaN,NaN,0.772353,2
2022-06-16,CAD,CHF,0.77108,0.77453,0.74425,0.74622,0.77097,0.77290,0.77319,NaN,NaN,NaN,NaN,NaN,0.765820,3
2022-06-17,CAD,CHF,0.74557,0.74978,0.73860,0.74369,0.74622,0.77097,0.77290,0.77319,NaN,NaN,NaN,NaN,0.761394,4
2022-06-20,CAD,CHF,0.74425,0.74627,0.73960,0.74439,0.74369,0.74622,0.77097,0.77290,0.77319,NaN,NaN,NaN,0.758560,5
2022-06-21,CAD,CHF,0.74483,0.74874,0.74380,0.74721,0.74439,0.74369,0.74622,0.77097,0.77290,0.77319,NaN,0.756939,0.756939,6
2022-06-22,CAD,CHF,0.74769,0.74778,0.74000,0.74203,0.74721,0.74439,0.74369,0.74622,0.77097,0.77290,0.77319,0.752487,0.755075,7
2022-06-23,CAD,CHF,0.74207,0.74567,0.73680,0.73925,0.74203,0.74721,0.74439,0.74369,0.74622,0.77097,0.77290,0.747680,0.753317,8


In [65]:
cadhkd = app.get_currency_exchange_daily('CAD', 'HKD')
cadhkd1 = cadhkd[0]
cadhkd1['to symbol'] = 'HKD'
cadhkd1['from symbol'] = 'CAD'

cadhkd1.rename(columns={'1. open': 'open'}, inplace=True)
cadhkd1.rename(columns={'2. high': 'high'}, inplace=True)
cadhkd1.rename(columns={'3. low': 'low'}, inplace=True)
cadhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadhkd1.index)
cadhkd1 = cadhkd1.sort_index()

cadhkd1['time'] = np.arange(len(cadhkd1.index))

cadhkd1['lag_1'] = cadhkd1['close'].shift(1)
cadhkd1['lag_2'] = cadhkd1['close'].shift(2)
cadhkd1['lag_3'] = cadhkd1['close'].shift(3)
cadhkd1['lag_4'] = cadhkd1['close'].shift(4)
cadhkd1['lag_5'] = cadhkd1['close'].shift(5)
cadhkd1['lag_6'] = cadhkd1['close'].shift(6)
cadhkd1['lag_7'] = cadhkd1['close'].shift(7)

cadhkd1['rolling_mean'] = cadhkd1['close'].rolling(window=7).mean()
cadhkd1['expanding_mean'] = cadhkd1['close'].expanding(2).mean()

cadhkd1 = cadhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadhkd1])

cadhkd1.to_pickle('cadhkd1.pkl')

cadhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,HKD,6.13837,6.13845,6.0780,6.08584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,HKD,6.08689,6.10067,6.0370,6.05753,6.08584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.071685,1
2022-06-15,CAD,HKD,6.05795,6.10213,6.0300,6.09010,6.05753,6.08584,NaN,NaN,NaN,NaN,NaN,NaN,6.077823,2
2022-06-16,CAD,HKD,6.09101,6.10282,6.0295,6.06064,6.09010,6.05753,6.08584,NaN,NaN,NaN,NaN,NaN,6.073527,3
2022-06-17,CAD,HKD,6.06106,6.06600,5.9870,6.02370,6.06064,6.09010,6.05753,6.08584,NaN,NaN,NaN,NaN,6.063562,4
2022-06-20,CAD,HKD,6.02071,6.04781,6.0100,6.04550,6.02370,6.06064,6.09010,6.05753,6.08584,NaN,NaN,NaN,6.060552,5
2022-06-21,CAD,HKD,6.04641,6.08204,6.0290,6.07510,6.04550,6.02370,6.06064,6.09010,6.05753,6.08584,NaN,6.062630,6.062630,6
2022-06-22,CAD,HKD,6.07595,6.07897,6.0240,6.06154,6.07510,6.04550,6.02370,6.06064,6.09010,6.05753,6.08584,6.059159,6.062494,7
2022-06-23,CAD,HKD,6.06172,6.06712,6.0160,6.03610,6.06154,6.07510,6.04550,6.02370,6.06064,6.09010,6.05753,6.056097,6.059561,8


In [66]:
cadsek = app.get_currency_exchange_daily('CAD', 'SEK')
cadsek1 = cadsek[0]
cadsek1['to symbol'] = 'SEK'
cadsek1['from symbol'] = 'CAD'

cadsek1.rename(columns={'1. open': 'open'}, inplace=True)
cadsek1.rename(columns={'2. high': 'high'}, inplace=True)
cadsek1.rename(columns={'3. low': 'low'}, inplace=True)
cadsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadsek1.index)
cadsek1 = cadsek1.sort_index()

cadsek1['time'] = np.arange(len(cadsek1.index))

cadsek1['lag_1'] = cadsek1['close'].shift(1)
cadsek1['lag_2'] = cadsek1['close'].shift(2)
cadsek1['lag_3'] = cadsek1['close'].shift(3)
cadsek1['lag_4'] = cadsek1['close'].shift(4)
cadsek1['lag_5'] = cadsek1['close'].shift(5)
cadsek1['lag_6'] = cadsek1['close'].shift(6)
cadsek1['lag_7'] = cadsek1['close'].shift(7)

cadsek1['rolling_mean'] = cadsek1['close'].rolling(window=7).mean()
cadsek1['expanding_mean'] = cadsek1['close'].expanding(2).mean()

cadsek1 = cadsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadsek1])

cadsek1.to_pickle('cadsek1.pkl')

cadsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,SEK,7.8207,7.9239,7.8073,7.8976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,SEK,7.8979,7.9136,7.8337,7.8597,7.8976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.878650,1
2022-06-15,CAD,SEK,7.8498,7.9277,7.7916,7.8793,7.8597,7.8976,NaN,NaN,NaN,NaN,NaN,NaN,7.878867,2
2022-06-16,CAD,SEK,7.8787,7.9608,7.8070,7.8070,7.8793,7.8597,7.8976,NaN,NaN,NaN,NaN,NaN,7.860900,3
2022-06-17,CAD,SEK,7.8142,7.8587,7.7904,7.8052,7.8070,7.8793,7.8597,7.8976,NaN,NaN,NaN,NaN,7.849760,4
2022-06-20,CAD,SEK,7.7828,7.8285,7.7592,7.7862,7.8052,7.8070,7.8793,7.8597,7.8976,NaN,NaN,NaN,7.839167,5
2022-06-21,CAD,SEK,7.7907,7.8264,7.7632,7.7910,7.7862,7.8052,7.8070,7.8793,7.8597,7.8976,NaN,7.832286,7.832286,6
2022-06-22,CAD,SEK,7.7970,7.8487,7.7623,7.7749,7.7910,7.7862,7.8052,7.8070,7.8793,7.8597,7.8976,7.814757,7.825113,7
2022-06-23,CAD,SEK,7.7852,7.8732,7.7727,7.8186,7.7749,7.7910,7.7862,7.8052,7.8070,7.8793,7.8597,7.808886,7.824389,8


In [67]:
cadnzd = app.get_currency_exchange_daily('CAD', 'NZD')
cadnzd1 = cadnzd[0]
cadnzd1['to symbol'] = 'NZD'
cadnzd1['from symbol'] = 'CAD'

cadnzd1.rename(columns={'1. open': 'open'}, inplace=True)
cadnzd1.rename(columns={'2. high': 'high'}, inplace=True)
cadnzd1.rename(columns={'3. low': 'low'}, inplace=True)
cadnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(cadnzd1.index)
cadnzd1 = cadnzd1.sort_index()

cadnzd1['time'] = np.arange(len(cadnzd1.index))

cadnzd1['lag_1'] = cadnzd1['close'].shift(1)
cadnzd1['lag_2'] = cadnzd1['close'].shift(2)
cadnzd1['lag_3'] = cadnzd1['close'].shift(3)
cadnzd1['lag_4'] = cadnzd1['close'].shift(4)
cadnzd1['lag_5'] = cadnzd1['close'].shift(5)
cadnzd1['lag_6'] = cadnzd1['close'].shift(6)
cadnzd1['lag_7'] = cadnzd1['close'].shift(7)

cadnzd1['rolling_mean'] = cadnzd1['close'].rolling(window=7).mean()
cadnzd1['expanding_mean'] = cadnzd1['close'].expanding(2).mean()

cadnzd1 = cadnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, cadnzd1])

cadnzd1.to_pickle('cadnzd1.pkl')

cadnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,CAD,NZD,1.2311,1.2403,1.2287,1.2367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,CAD,NZD,1.2366,1.2441,1.2329,1.2402,1.2367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.238450,1
2022-06-15,CAD,NZD,1.2405,1.2407,1.2295,1.2333,1.2402,1.2367,NaN,NaN,NaN,NaN,NaN,NaN,1.236733,2
2022-06-16,CAD,NZD,1.2328,1.2368,1.2102,1.2122,1.2333,1.2402,1.2367,NaN,NaN,NaN,NaN,NaN,1.230600,3
2022-06-17,CAD,NZD,1.2134,1.2189,1.2115,1.2145,1.2122,1.2333,1.2402,1.2367,NaN,NaN,NaN,NaN,1.227380,4
2022-06-20,CAD,NZD,1.2150,1.2163,1.2079,1.2152,1.2145,1.2122,1.2333,1.2402,1.2367,NaN,NaN,NaN,1.225350,5
2022-06-21,CAD,NZD,1.2150,1.2221,1.2143,1.2210,1.2152,1.2145,1.2122,1.2333,1.2402,1.2367,NaN,1.224729,1.224729,6
2022-06-22,CAD,NZD,1.2212,1.2317,1.2211,1.2271,1.2210,1.2152,1.2145,1.2122,1.2333,1.2402,1.2367,1.223357,1.225025,7
2022-06-23,CAD,NZD,1.2274,1.2320,1.2231,1.2236,1.2271,1.2210,1.2152,1.2145,1.2122,1.2333,1.2402,1.220986,1.224867,8


In [68]:
hkdusd = app.get_currency_exchange_daily('HKD', 'USD')
hkdusd1 = hkdusd[0]
hkdusd1['to symbol'] = 'USD'
hkdusd1['from symbol'] = 'HKD'

hkdusd1.rename(columns={'1. open': 'open'}, inplace=True)
hkdusd1.rename(columns={'2. high': 'high'}, inplace=True)
hkdusd1.rename(columns={'3. low': 'low'}, inplace=True)
hkdusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdusd1.index)
hkdusd1 = hkdusd1.sort_index()

hkdusd1['time'] = np.arange(len(hkdusd1.index))

hkdusd1['lag_1'] = hkdusd1['close'].shift(1)
hkdusd1['lag_2'] = hkdusd1['close'].shift(2)
hkdusd1['lag_3'] = hkdusd1['close'].shift(3)
hkdusd1['lag_4'] = hkdusd1['close'].shift(4)
hkdusd1['lag_5'] = hkdusd1['close'].shift(5)
hkdusd1['lag_6'] = hkdusd1['close'].shift(6)
hkdusd1['lag_7'] = hkdusd1['close'].shift(7)

hkdusd1['rolling_mean'] = hkdusd1['close'].rolling(window=7).mean()
hkdusd1['expanding_mean'] = hkdusd1['close'].expanding(2).mean()

hkdusd1 = hkdusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdusd1])

hkdusd1.to_pickle('hkdusd1.pkl')

hkdusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,USD,0.12734,0.12735,0.12733,0.12734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,USD,0.12734,0.12734,0.12733,0.12734,0.12734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127340,1
2022-06-15,HKD,USD,0.12734,0.12734,0.12733,0.12734,0.12734,0.12734,NaN,NaN,NaN,NaN,NaN,NaN,0.127340,2
2022-06-16,HKD,USD,0.12734,0.12736,0.12733,0.12735,0.12734,0.12734,0.12734,NaN,NaN,NaN,NaN,NaN,0.127342,3
2022-06-17,HKD,USD,0.12735,0.12735,0.12733,0.12734,0.12735,0.12734,0.12734,0.12734,NaN,NaN,NaN,NaN,0.127342,4
2022-06-20,HKD,USD,0.12734,0.12734,0.12733,0.12734,0.12734,0.12735,0.12734,0.12734,0.12734,NaN,NaN,NaN,0.127342,5
2022-06-21,HKD,USD,0.12734,0.12734,0.12733,0.12734,0.12734,0.12734,0.12735,0.12734,0.12734,0.12734,NaN,0.127341,0.127341,6
2022-06-22,HKD,USD,0.12734,0.12734,0.12733,0.12733,0.12734,0.12734,0.12734,0.12735,0.12734,0.12734,0.12734,0.127340,0.127340,7
2022-06-23,HKD,USD,0.12734,0.12736,0.12734,0.12735,0.12733,0.12734,0.12734,0.12734,0.12735,0.12734,0.12734,0.127341,0.127341,8


In [69]:
hkdeur = app.get_currency_exchange_daily('HKD', 'EUR')
hkdeur1 = hkdeur[0]
hkdeur1['to symbol'] = 'EUR'
hkdeur1['from symbol'] = 'HKD'

hkdeur1.rename(columns={'1. open': 'open'}, inplace=True)
hkdeur1.rename(columns={'2. high': 'high'}, inplace=True)
hkdeur1.rename(columns={'3. low': 'low'}, inplace=True)
hkdeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdeur1.index)
hkdeur1 = hkdeur1.sort_index()

hkdeur1['time'] = np.arange(len(hkdeur1.index))

hkdeur1['lag_1'] = hkdeur1['close'].shift(1)
hkdeur1['lag_2'] = hkdeur1['close'].shift(2)
hkdeur1['lag_3'] = hkdeur1['close'].shift(3)
hkdeur1['lag_4'] = hkdeur1['close'].shift(4)
hkdeur1['lag_5'] = hkdeur1['close'].shift(5)
hkdeur1['lag_6'] = hkdeur1['close'].shift(6)
hkdeur1['lag_7'] = hkdeur1['close'].shift(7)

hkdeur1['rolling_mean'] = hkdeur1['close'].rolling(window=7).mean()
hkdeur1['expanding_mean'] = hkdeur1['close'].expanding(2).mean()

hkdeur1 = hkdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdeur1])

hkdeur1.to_pickle('hkdeur1.pkl')

hkdeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,EUR,0.12115,0.12239,0.12109,0.12234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,EUR,0.12232,0.12245,0.12149,0.12224,0.12234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122290,1
2022-06-15,HKD,EUR,0.12224,0.12288,0.12130,0.12192,0.12224,0.12234,NaN,NaN,NaN,NaN,NaN,NaN,0.122167,2
2022-06-16,HKD,EUR,0.12188,0.12263,0.12017,0.12066,0.12192,0.12224,0.12234,NaN,NaN,NaN,NaN,NaN,0.121790,3
2022-06-17,HKD,EUR,0.12065,0.12188,0.12059,0.12128,0.12066,0.12192,0.12224,0.12234,NaN,NaN,NaN,NaN,0.121688,4
2022-06-20,HKD,EUR,0.12149,0.12154,0.12079,0.12112,0.12128,0.12066,0.12192,0.12224,0.12234,NaN,NaN,NaN,0.121593,5
2022-06-21,HKD,EUR,0.12113,0.12116,0.12035,0.12085,0.12112,0.12128,0.12066,0.12192,0.12224,0.12234,NaN,0.121487,0.121487,6
2022-06-22,HKD,EUR,0.12088,0.12157,0.12011,0.12052,0.12085,0.12112,0.12128,0.12066,0.12192,0.12224,0.12234,0.121227,0.121366,7
2022-06-23,HKD,EUR,0.12048,0.12146,0.12037,0.12103,0.12052,0.12085,0.12112,0.12128,0.12066,0.12192,0.12224,0.121054,0.121329,8


In [70]:
hkdjpy = app.get_currency_exchange_daily('HKD', 'JPY')
hkdjpy1 = hkdjpy[0]
hkdjpy1['to symbol'] = 'JPY'
hkdjpy1['from symbol'] = 'HKD'

hkdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
hkdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
hkdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
hkdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdjpy1.index)
hkdjpy1 = hkdjpy1.sort_index()

hkdjpy1['time'] = np.arange(len(hkdjpy1.index))

hkdjpy1['lag_1'] = hkdjpy1['close'].shift(1)
hkdjpy1['lag_2'] = hkdjpy1['close'].shift(2)
hkdjpy1['lag_3'] = hkdjpy1['close'].shift(3)
hkdjpy1['lag_4'] = hkdjpy1['close'].shift(4)
hkdjpy1['lag_5'] = hkdjpy1['close'].shift(5)
hkdjpy1['lag_6'] = hkdjpy1['close'].shift(6)
hkdjpy1['lag_7'] = hkdjpy1['close'].shift(7)

hkdjpy1['rolling_mean'] = hkdjpy1['close'].rolling(window=7).mean()
hkdjpy1['expanding_mean'] = hkdjpy1['close'].expanding(2).mean()

hkdjpy1 = hkdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdjpy1])

hkdjpy1.to_pickle('hkdjpy1.pkl')

hkdjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,JPY,17.12080,17.22206,17.01690,17.12148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,JPY,17.12269,17.27140,17.05339,17.26742,17.12148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.194450,1
2022-06-15,HKD,JPY,17.26907,17.27082,17.00600,17.04740,17.26742,17.12148,NaN,NaN,NaN,NaN,NaN,NaN,17.145433,2
2022-06-16,HKD,JPY,17.04999,17.15725,16.75130,16.85246,17.04740,17.26742,17.12148,NaN,NaN,NaN,NaN,NaN,17.072190,3
2022-06-17,HKD,JPY,16.85669,17.25091,16.83845,17.19400,16.85246,17.04740,17.26742,17.12148,NaN,NaN,NaN,NaN,17.096552,4
2022-06-20,HKD,JPY,17.19290,17.25353,17.13779,17.20685,17.19400,16.85246,17.04740,17.26742,17.12148,NaN,NaN,NaN,17.114935,5
2022-06-21,HKD,JPY,17.20953,17.41418,17.18660,17.40846,17.20685,17.19400,16.85246,17.04740,17.26742,17.12148,NaN,17.156867,17.156867,6
2022-06-22,HKD,JPY,17.40897,17.41506,17.28434,17.33960,17.40846,17.20685,17.19400,16.85246,17.04740,17.26742,17.12148,17.188027,17.179709,7
2022-06-23,HKD,JPY,17.34087,17.35018,17.10530,17.18920,17.33960,17.40846,17.20685,17.19400,16.85246,17.04740,17.26742,17.176853,17.180763,8


In [71]:
hkdgbp = app.get_currency_exchange_daily('HKD', 'GBP')
hkdgbp1 = hkdgbp[0]
hkdgbp1['to symbol'] = 'GBP'
hkdgbp1['from symbol'] = 'HKD'

hkdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
hkdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
hkdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
hkdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdgbp1.index)
hkdgbp1 = hkdgbp1.sort_index()

hkdgbp1['time'] = np.arange(len(hkdgbp1.index))

hkdgbp1['lag_1'] = hkdgbp1['close'].shift(1)
hkdgbp1['lag_2'] = hkdgbp1['close'].shift(2)
hkdgbp1['lag_3'] = hkdgbp1['close'].shift(3)
hkdgbp1['lag_4'] = hkdgbp1['close'].shift(4)
hkdgbp1['lag_5'] = hkdgbp1['close'].shift(5)
hkdgbp1['lag_6'] = hkdgbp1['close'].shift(6)
hkdgbp1['lag_7'] = hkdgbp1['close'].shift(7)

hkdgbp1['rolling_mean'] = hkdgbp1['close'].rolling(window=7).mean()
hkdgbp1['expanding_mean'] = hkdgbp1['close'].expanding(2).mean()

hkdgbp1 = hkdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdgbp1])

hkdgbp1.to_pickle('hkdgbp1.pkl')

hkdgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,GBP,0.10344,0.10509,0.10344,0.10493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,GBP,0.10489,0.10662,0.10440,0.10611,0.10493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105520,1
2022-06-15,HKD,GBP,0.10612,0.10618,0.10437,0.10461,0.10611,0.10493,NaN,NaN,NaN,NaN,NaN,NaN,0.105217,2
2022-06-16,HKD,GBP,0.10459,0.10567,0.10270,0.10308,0.10461,0.10611,0.10493,NaN,NaN,NaN,NaN,NaN,0.104682,3
2022-06-17,HKD,GBP,0.10308,0.10457,0.10303,0.10416,0.10308,0.10461,0.10611,0.10493,NaN,NaN,NaN,NaN,0.104578,4
2022-06-20,HKD,GBP,0.10423,0.10436,0.10372,0.10395,0.10416,0.10308,0.10461,0.10611,0.10493,NaN,NaN,NaN,0.104473,5
2022-06-21,HKD,GBP,0.10396,0.10399,0.10337,0.10367,0.10395,0.10416,0.10308,0.10461,0.10611,0.10493,NaN,0.104359,0.104359,6
2022-06-22,HKD,GBP,0.10372,0.10463,0.10342,0.10381,0.10367,0.10395,0.10416,0.10308,0.10461,0.10611,0.10493,0.104199,0.104290,7
2022-06-23,HKD,GBP,0.10381,0.10464,0.10361,0.10387,0.10381,0.10367,0.10395,0.10416,0.10308,0.10461,0.10611,0.103879,0.104243,8


In [72]:
hkdaud = app.get_currency_exchange_daily('HKD', 'AUD')
hkdaud1 = hkdaud[0]
hkdaud1['to symbol'] = 'AUD'
hkdaud1['from symbol'] = 'HKD'

hkdaud1.rename(columns={'1. open': 'open'}, inplace=True)
hkdaud1.rename(columns={'2. high': 'high'}, inplace=True)
hkdaud1.rename(columns={'3. low': 'low'}, inplace=True)
hkdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdaud1.index)
hkdaud1 = hkdaud1.sort_index()

hkdaud1['time'] = np.arange(len(hkdaud1.index))

hkdaud1['lag_1'] = hkdaud1['close'].shift(1)
hkdaud1['lag_2'] = hkdaud1['close'].shift(2)
hkdaud1['lag_3'] = hkdaud1['close'].shift(3)
hkdaud1['lag_4'] = hkdaud1['close'].shift(4)
hkdaud1['lag_5'] = hkdaud1['close'].shift(5)
hkdaud1['lag_6'] = hkdaud1['close'].shift(6)
hkdaud1['lag_7'] = hkdaud1['close'].shift(7)

hkdaud1['rolling_mean'] = hkdaud1['close'].rolling(window=7).mean()
hkdaud1['expanding_mean'] = hkdaud1['close'].expanding(2).mean()

hkdaud1 = hkdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdaud1])

hkdaud1.to_pickle('hkdaud1.pkl')

hkdaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,AUD,0.18104,0.18418,0.18104,0.18385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,AUD,0.18380,0.18581,0.18276,0.18522,0.18385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.184535,1
2022-06-15,HKD,AUD,0.18523,0.18525,0.18139,0.18177,0.18522,0.18385,NaN,NaN,NaN,NaN,NaN,NaN,0.183613,2
2022-06-16,HKD,AUD,0.18179,0.18333,0.18021,0.18072,0.18177,0.18522,0.18385,NaN,NaN,NaN,NaN,NaN,0.182890,3
2022-06-17,HKD,AUD,0.18071,0.18456,0.18063,0.18368,0.18072,0.18177,0.18522,0.18385,NaN,NaN,NaN,NaN,0.183048,4
2022-06-20,HKD,AUD,0.18362,0.18386,0.18204,0.18312,0.18368,0.18072,0.18177,0.18522,0.18385,NaN,NaN,NaN,0.183060,5
2022-06-21,HKD,AUD,0.18309,0.18357,0.18216,0.18271,0.18312,0.18368,0.18072,0.18177,0.18522,0.18385,NaN,0.183010,0.183010,6
2022-06-22,HKD,AUD,0.18270,0.18507,0.18266,0.18390,0.18271,0.18312,0.18368,0.18072,0.18177,0.18522,0.18385,0.183017,0.183121,7
2022-06-23,HKD,AUD,0.18388,0.18539,0.18384,0.18468,0.18390,0.18271,0.18312,0.18368,0.18072,0.18177,0.18522,0.182940,0.183294,8


In [73]:
hkdchf = app.get_currency_exchange_daily('HKD', 'CHF')
hkdchf1 = hkdchf[0]
hkdchf1['to symbol'] = 'CHF'
hkdchf1['from symbol'] = 'HKD'

hkdchf1.rename(columns={'1. open': 'open'}, inplace=True)
hkdchf1.rename(columns={'2. high': 'high'}, inplace=True)
hkdchf1.rename(columns={'3. low': 'low'}, inplace=True)
hkdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdchf1.index)
hkdchf1 = hkdchf1.sort_index()

hkdchf1['time'] = np.arange(len(hkdchf1.index))

hkdchf1['lag_1'] = hkdchf1['close'].shift(1)
hkdchf1['lag_2'] = hkdchf1['close'].shift(2)
hkdchf1['lag_3'] = hkdchf1['close'].shift(3)
hkdchf1['lag_4'] = hkdchf1['close'].shift(4)
hkdchf1['lag_5'] = hkdchf1['close'].shift(5)
hkdchf1['lag_6'] = hkdchf1['close'].shift(6)
hkdchf1['lag_7'] = hkdchf1['close'].shift(7)

hkdchf1['rolling_mean'] = hkdchf1['close'].rolling(window=7).mean()
hkdchf1['expanding_mean'] = hkdchf1['close'].expanding(2).mean()

hkdchf1 = hkdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdchf1])

hkdchf1.to_pickle('hkdchf1.pkl')

hkdchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,CHF,0.12587,0.12731,0.12584,0.12701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,CHF,0.12701,0.12785,0.12578,0.12752,0.12701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127265,1
2022-06-15,HKD,CHF,0.12749,0.12800,0.12648,0.12659,0.12752,0.12701,NaN,NaN,NaN,NaN,NaN,NaN,0.127040,2
2022-06-16,HKD,CHF,0.12653,0.12723,0.12271,0.12301,0.12659,0.12752,0.12701,NaN,NaN,NaN,NaN,NaN,0.126032,3
2022-06-17,HKD,CHF,0.12295,0.12396,0.12254,0.12354,0.12301,0.12659,0.12752,0.12701,NaN,NaN,NaN,NaN,0.125534,4
2022-06-20,HKD,CHF,0.12358,0.12373,0.12257,0.12313,0.12354,0.12301,0.12659,0.12752,0.12701,NaN,NaN,NaN,0.125133,5
2022-06-21,HKD,CHF,0.12316,0.12334,0.12270,0.12301,0.12313,0.12354,0.12301,0.12659,0.12752,0.12701,NaN,0.124830,0.124830,6
2022-06-22,HKD,CHF,0.12299,0.12342,0.12208,0.12243,0.12301,0.12313,0.12354,0.12301,0.12659,0.12752,0.12701,0.124176,0.124530,7
2022-06-23,HKD,CHF,0.12241,0.12329,0.12191,0.12245,0.12243,0.12301,0.12313,0.12354,0.12301,0.12659,0.12752,0.123451,0.124299,8


In [74]:
hkdcad = app.get_currency_exchange_daily('HKD', 'CAD')
hkdcad1 = hkdcad[0]
hkdcad1['to symbol'] = 'CAD'
hkdcad1['from symbol'] = 'HKD'

hkdcad1.rename(columns={'1. open': 'open'}, inplace=True)
hkdcad1.rename(columns={'2. high': 'high'}, inplace=True)
hkdcad1.rename(columns={'3. low': 'low'}, inplace=True)
hkdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdcad1.index)
hkdcad1 = hkdcad1.sort_index()

hkdcad1['time'] = np.arange(len(hkdcad1.index))

hkdcad1['lag_1'] = hkdcad1['close'].shift(1)
hkdcad1['lag_2'] = hkdcad1['close'].shift(2)
hkdcad1['lag_3'] = hkdcad1['close'].shift(3)
hkdcad1['lag_4'] = hkdcad1['close'].shift(4)
hkdcad1['lag_5'] = hkdcad1['close'].shift(5)
hkdcad1['lag_6'] = hkdcad1['close'].shift(6)
hkdcad1['lag_7'] = hkdcad1['close'].shift(7)

hkdcad1['rolling_mean'] = hkdcad1['close'].rolling(window=7).mean()
hkdcad1['expanding_mean'] = hkdcad1['close'].expanding(2).mean()

hkdcad1 = hkdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdcad1])

hkdcad1.to_pickle('hkdcad1.pkl')

hkdcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,CAD,0.16288,0.16429,0.16288,0.16423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,CAD,0.16420,0.16523,0.16390,0.16501,0.16423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.164620,1
2022-06-15,HKD,CAD,0.16495,0.16543,0.16383,0.16412,0.16501,0.16423,NaN,NaN,NaN,NaN,NaN,NaN,0.164453,2
2022-06-16,HKD,CAD,0.16415,0.16515,0.16380,0.16492,0.16412,0.16501,0.16423,NaN,NaN,NaN,NaN,NaN,0.164570,3
2022-06-17,HKD,CAD,0.16487,0.16652,0.16480,0.16592,0.16492,0.16412,0.16501,0.16423,NaN,NaN,NaN,NaN,0.164840,4
2022-06-20,HKD,CAD,0.16594,0.16594,0.16530,0.16532,0.16592,0.16492,0.16412,0.16501,0.16423,NaN,NaN,NaN,0.164920,5
2022-06-21,HKD,CAD,0.16534,0.16540,0.16437,0.16451,0.16532,0.16592,0.16492,0.16412,0.16501,0.16423,NaN,0.164861,0.164861,6
2022-06-22,HKD,CAD,0.16454,0.16551,0.16447,0.16490,0.16451,0.16532,0.16592,0.16492,0.16412,0.16501,0.16423,0.164957,0.164866,7
2022-06-23,HKD,CAD,0.16492,0.16576,0.16478,0.16559,0.16490,0.16451,0.16532,0.16592,0.16492,0.16412,0.16501,0.165040,0.164947,8


In [75]:
hkdsek = app.get_currency_exchange_daily('HKD', 'SEK')
hkdsek1 = hkdsek[0]
hkdsek1['to symbol'] = 'SEK'
hkdsek1['from symbol'] = 'HKD'

hkdsek1.rename(columns={'1. open': 'open'}, inplace=True)
hkdsek1.rename(columns={'2. high': 'high'}, inplace=True)
hkdsek1.rename(columns={'3. low': 'low'}, inplace=True)
hkdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdsek1.index)
hkdsek1 = hkdsek1.sort_index()

hkdsek1['time'] = np.arange(len(hkdsek1.index))

hkdsek1['lag_1'] = hkdsek1['close'].shift(1)
hkdsek1['lag_2'] = hkdsek1['close'].shift(2)
hkdsek1['lag_3'] = hkdsek1['close'].shift(3)
hkdsek1['lag_4'] = hkdsek1['close'].shift(4)
hkdsek1['lag_5'] = hkdsek1['close'].shift(5)
hkdsek1['lag_6'] = hkdsek1['close'].shift(6)
hkdsek1['lag_7'] = hkdsek1['close'].shift(7)

hkdsek1['rolling_mean'] = hkdsek1['close'].rolling(window=7).mean()
hkdsek1['expanding_mean'] = hkdsek1['close'].expanding(2).mean()

hkdsek1 = hkdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdsek1])

hkdsek1.to_pickle('hkdsek1.pkl')

hkdsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,SEK,1.2733,1.2990,1.2727,1.2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,SEK,1.2978,1.3015,1.2860,1.2975,1.2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.297550,1
2022-06-15,HKD,SEK,1.2974,1.3106,1.2855,1.2937,1.2975,1.2976,NaN,NaN,NaN,NaN,NaN,NaN,1.296267,2
2022-06-16,HKD,SEK,1.2936,1.3121,1.2858,1.2881,1.2937,1.2975,1.2976,NaN,NaN,NaN,NaN,NaN,1.294225,3
2022-06-17,HKD,SEK,1.2885,1.3036,1.2880,1.2948,1.2881,1.2937,1.2975,1.2976,NaN,NaN,NaN,NaN,1.294340,4
2022-06-20,HKD,SEK,1.2934,1.2975,1.2839,1.2879,1.2948,1.2881,1.2937,1.2975,1.2976,NaN,NaN,NaN,1.293267,5
2022-06-21,HKD,SEK,1.2876,1.2991,1.2775,1.2853,1.2879,1.2948,1.2881,1.2937,1.2975,1.2976,NaN,1.292129,1.292129,6
2022-06-22,HKD,SEK,1.2853,1.2974,1.2785,1.2827,1.2853,1.2879,1.2948,1.2881,1.2937,1.2975,1.2976,1.290000,1.290950,7
2022-06-23,HKD,SEK,1.2824,1.2997,1.2824,1.2952,1.2827,1.2853,1.2879,1.2948,1.2881,1.2937,1.2975,1.289671,1.291422,8


In [76]:
hkdnzd = app.get_currency_exchange_daily('HKD', 'NZD')
hkdnzd1 = hkdnzd[0]
hkdnzd1['to symbol'] = 'NZD'
hkdnzd1['from symbol'] = 'HKD'

hkdnzd1.rename(columns={'1. open': 'open'}, inplace=True)
hkdnzd1.rename(columns={'2. high': 'high'}, inplace=True)
hkdnzd1.rename(columns={'3. low': 'low'}, inplace=True)
hkdnzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(hkdnzd1.index)
hkdnzd1 = hkdnzd1.sort_index()

hkdnzd1['time'] = np.arange(len(hkdnzd1.index))

hkdnzd1['lag_1'] = hkdnzd1['close'].shift(1)
hkdnzd1['lag_2'] = hkdnzd1['close'].shift(2)
hkdnzd1['lag_3'] = hkdnzd1['close'].shift(3)
hkdnzd1['lag_4'] = hkdnzd1['close'].shift(4)
hkdnzd1['lag_5'] = hkdnzd1['close'].shift(5)
hkdnzd1['lag_6'] = hkdnzd1['close'].shift(6)
hkdnzd1['lag_7'] = hkdnzd1['close'].shift(7)

hkdnzd1['rolling_mean'] = hkdnzd1['close'].rolling(window=7).mean()
hkdnzd1['expanding_mean'] = hkdnzd1['close'].expanding(2).mean()

hkdnzd1 = hkdnzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, hkdnzd1])

hkdnzd1.to_pickle('hkdnzd1.pkl')

hkdnzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,HKD,NZD,0.20054,0.20360,0.20038,0.20316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,HKD,NZD,0.20307,0.20529,0.20222,0.20466,0.20316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.203910,1
2022-06-15,HKD,NZD,0.20463,0.20467,0.20158,0.20244,0.20466,0.20316,NaN,NaN,NaN,NaN,NaN,NaN,0.203420,2
2022-06-16,HKD,NZD,0.20237,0.20410,0.19902,0.19995,0.20244,0.20466,0.20316,NaN,NaN,NaN,NaN,NaN,0.202552,3
2022-06-17,HKD,NZD,0.20007,0.20287,0.19981,0.20152,0.19995,0.20244,0.20466,0.20316,NaN,NaN,NaN,NaN,0.202346,4
2022-06-20,HKD,NZD,0.20159,0.20172,0.19994,0.20092,0.20152,0.19995,0.20244,0.20466,0.20316,NaN,NaN,NaN,0.202108,5
2022-06-21,HKD,NZD,0.20092,0.20137,0.19998,0.20090,0.20092,0.20152,0.19995,0.20244,0.20466,0.20316,NaN,0.201936,0.201936,6
2022-06-22,HKD,NZD,0.20095,0.20374,0.20095,0.20236,0.20090,0.20092,0.20152,0.19995,0.20244,0.20466,0.20316,0.201821,0.201989,7
2022-06-23,HKD,NZD,0.20241,0.20364,0.20167,0.20263,0.20236,0.20090,0.20092,0.20152,0.19995,0.20244,0.20466,0.201531,0.202060,8


In [77]:
sekusd = app.get_currency_exchange_daily('SEK', 'USD')
sekusd1 = sekusd[0]
sekusd1['to symbol'] = 'USD'
sekusd1['from symbol'] = 'SEK'

sekusd1.rename(columns={'1. open': 'open'}, inplace=True)
sekusd1.rename(columns={'2. high': 'high'}, inplace=True)
sekusd1.rename(columns={'3. low': 'low'}, inplace=True)
sekusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekusd1.index)
sekusd1 = sekusd1.sort_index()

sekusd1['time'] = np.arange(len(sekusd1.index))

sekusd1['lag_1'] = sekusd1['close'].shift(1)
sekusd1['lag_2'] = sekusd1['close'].shift(2)
sekusd1['lag_3'] = sekusd1['close'].shift(3)
sekusd1['lag_4'] = sekusd1['close'].shift(4)
sekusd1['lag_5'] = sekusd1['close'].shift(5)
sekusd1['lag_6'] = sekusd1['close'].shift(6)
sekusd1['lag_7'] = sekusd1['close'].shift(7)

sekusd1['rolling_mean'] = sekusd1['close'].rolling(window=7).mean()
sekusd1['expanding_mean'] = sekusd1['close'].expanding(2).mean()

sekusd1 = sekusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekusd1])

sekusd1.to_pickle('sekusd1.pkl')

sekusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,USD,0.0995,0.0995,0.0975,0.0976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,USD,0.0976,0.0983,0.0973,0.0976,0.0976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097600,1
2022-06-15,SEK,USD,0.0975,0.0984,0.0966,0.0977,0.0976,0.0976,NaN,NaN,NaN,NaN,NaN,NaN,0.097633,2
2022-06-16,SEK,USD,0.0978,0.0985,0.0966,0.0982,0.0977,0.0976,0.0976,NaN,NaN,NaN,NaN,NaN,0.097775,3
2022-06-17,SEK,USD,0.0982,0.0983,0.0972,0.0978,0.0982,0.0977,0.0976,0.0976,NaN,NaN,NaN,NaN,0.097780,4
2022-06-20,SEK,USD,0.0980,0.0985,0.0976,0.0983,0.0978,0.0982,0.0977,0.0976,0.0976,NaN,NaN,NaN,0.097867,5
2022-06-21,SEK,USD,0.0982,0.0991,0.0982,0.0985,0.0983,0.0978,0.0982,0.0977,0.0976,0.0976,NaN,0.097957,0.097957,6
2022-06-22,SEK,USD,0.0985,0.0990,0.0976,0.0986,0.0985,0.0983,0.0978,0.0982,0.0977,0.0976,0.0976,0.098100,0.098037,7
2022-06-23,SEK,USD,0.0986,0.0987,0.0974,0.0977,0.0986,0.0985,0.0983,0.0978,0.0982,0.0977,0.0976,0.098114,0.098000,8


In [78]:
sekeur = app.get_currency_exchange_daily('SEK', 'EUR')
sekeur1 = sekeur[0]
sekeur1['to symbol'] = 'EUR'
sekeur1['from symbol'] = 'SEK'

sekeur1.rename(columns={'1. open': 'open'}, inplace=True)
sekeur1.rename(columns={'2. high': 'high'}, inplace=True)
sekeur1.rename(columns={'3. low': 'low'}, inplace=True)
sekeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekeur1.index)
sekeur1 = sekeur1.sort_index()

sekeur1['time'] = np.arange(len(sekeur1.index))

sekeur1['lag_1'] = sekeur1['close'].shift(1)
sekeur1['lag_2'] = sekeur1['close'].shift(2)
sekeur1['lag_3'] = sekeur1['close'].shift(3)
sekeur1['lag_4'] = sekeur1['close'].shift(4)
sekeur1['lag_5'] = sekeur1['close'].shift(5)
sekeur1['lag_6'] = sekeur1['close'].shift(6)
sekeur1['lag_7'] = sekeur1['close'].shift(7)

sekeur1['rolling_mean'] = sekeur1['close'].rolling(window=7).mean()
sekeur1['expanding_mean'] = sekeur1['close'].expanding(2).mean()

sekeur1 = sekeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekeur1])

sekeur1.to_pickle('sekeur1.pkl')

sekeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,EUR,0.09461,0.09461,0.09357,0.09370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,EUR,0.09370,0.09415,0.09346,0.09362,0.09370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093660,1
2022-06-15,SEK,EUR,0.09365,0.09376,0.09312,0.09354,0.09362,0.09370,NaN,NaN,NaN,NaN,NaN,NaN,0.093620,2
2022-06-16,SEK,EUR,0.09352,0.09365,0.09263,0.09300,0.09354,0.09362,0.09370,NaN,NaN,NaN,NaN,NaN,0.093465,3
2022-06-17,SEK,EUR,0.09301,0.09325,0.09287,0.09310,0.09300,0.09354,0.09362,0.09370,NaN,NaN,NaN,NaN,0.093392,4
2022-06-20,SEK,EUR,0.09338,0.09354,0.09300,0.09353,0.09310,0.09300,0.09354,0.09362,0.09370,NaN,NaN,NaN,0.093415,5
2022-06-21,SEK,EUR,0.09350,0.09367,0.09335,0.09353,0.09353,0.09310,0.09300,0.09354,0.09362,0.09370,NaN,0.093431,0.093431,6
2022-06-22,SEK,EUR,0.09350,0.09351,0.09307,0.09326,0.09353,0.09353,0.09310,0.09300,0.09354,0.09362,0.09370,0.093369,0.093410,7
2022-06-23,SEK,EUR,0.09325,0.09330,0.09274,0.09281,0.09326,0.09353,0.09353,0.09310,0.09300,0.09354,0.09362,0.093253,0.093343,8


In [79]:
sekjpy = app.get_currency_exchange_daily('SEK', 'JPY')
sekjpy1 = sekjpy[0]
sekjpy1['to symbol'] = 'JPY'
sekjpy1['from symbol'] = 'SEK'

sekjpy1.rename(columns={'1. open': 'open'}, inplace=True)
sekjpy1.rename(columns={'2. high': 'high'}, inplace=True)
sekjpy1.rename(columns={'3. low': 'low'}, inplace=True)
sekjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekjpy1.index)
sekjpy1 = sekjpy1.sort_index()

sekjpy1['time'] = np.arange(len(sekjpy1.index))

sekjpy1['lag_1'] = sekjpy1['close'].shift(1)
sekjpy1['lag_2'] = sekjpy1['close'].shift(2)
sekjpy1['lag_3'] = sekjpy1['close'].shift(3)
sekjpy1['lag_4'] = sekjpy1['close'].shift(4)
sekjpy1['lag_5'] = sekjpy1['close'].shift(5)
sekjpy1['lag_6'] = sekjpy1['close'].shift(6)
sekjpy1['lag_7'] = sekjpy1['close'].shift(7)

sekjpy1['rolling_mean'] = sekjpy1['close'].rolling(window=7).mean()
sekjpy1['expanding_mean'] = sekjpy1['close'].expanding(2).mean()

sekjpy1 = sekjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekjpy1])

sekjpy1.to_pickle('sekjpy1.pkl')

sekjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,JPY,13.3954,13.463,13.1002,13.1567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,JPY,13.1533,13.287,13.1100,13.2614,13.1567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.209050,1
2022-06-15,SEK,JPY,13.2653,13.322,13.0438,13.1430,13.2614,13.1567,NaN,NaN,NaN,NaN,NaN,NaN,13.187033,2
2022-06-16,SEK,JPY,13.1042,13.225,12.8757,13.0224,13.1430,13.2614,13.1567,NaN,NaN,NaN,NaN,NaN,13.145875,3
2022-06-17,SEK,JPY,13.0185,13.282,12.9968,13.2600,13.0224,13.1430,13.2614,13.1567,NaN,NaN,NaN,NaN,13.168700,4
2022-06-20,SEK,JPY,13.2910,13.367,13.2176,13.2973,13.2600,13.0224,13.1430,13.2614,13.1567,NaN,NaN,NaN,13.190133,5
2022-06-21,SEK,JPY,13.2962,13.536,13.2962,13.5350,13.2973,13.2600,13.0224,13.1430,13.2614,13.1567,NaN,13.239400,13.239400,6
2022-06-22,SEK,JPY,13.5141,13.531,13.3585,13.4720,13.5350,13.2973,13.2600,13.0224,13.1430,13.2614,13.1567,13.284443,13.268475,7
2022-06-23,SEK,JPY,13.4775,13.495,13.1860,13.2195,13.4720,13.5350,13.2973,13.2600,13.0224,13.1430,13.2614,13.278457,13.263033,8


In [80]:
sekgbp = app.get_currency_exchange_daily('SEK', 'GBP')
sekgbp1 = sekgbp[0]
sekgbp1['to symbol'] = 'GBP'
sekgbp1['from symbol'] = 'SEK'

sekgbp1.rename(columns={'1. open': 'open'}, inplace=True)
sekgbp1.rename(columns={'2. high': 'high'}, inplace=True)
sekgbp1.rename(columns={'3. low': 'low'}, inplace=True)
sekgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekgbp1.index)
sekgbp1 = sekgbp1.sort_index()

sekgbp1['time'] = np.arange(len(sekgbp1.index))

sekgbp1['lag_1'] = sekgbp1['close'].shift(1)
sekgbp1['lag_2'] = sekgbp1['close'].shift(2)
sekgbp1['lag_3'] = sekgbp1['close'].shift(3)
sekgbp1['lag_4'] = sekgbp1['close'].shift(4)
sekgbp1['lag_5'] = sekgbp1['close'].shift(5)
sekgbp1['lag_6'] = sekgbp1['close'].shift(6)
sekgbp1['lag_7'] = sekgbp1['close'].shift(7)

sekgbp1['rolling_mean'] = sekgbp1['close'].rolling(window=7).mean()
sekgbp1['expanding_mean'] = sekgbp1['close'].expanding(2).mean()

sekgbp1 = sekgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekgbp1])

sekgbp1.to_pickle('sekgbp1.pkl')

sekgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,GBP,0.08096,0.08098,0.08045,0.08056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,GBP,0.08053,0.08169,0.08041,0.08145,0.08056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.081005,1
2022-06-15,SEK,GBP,0.08150,0.08185,0.08021,0.08044,0.08145,0.08056,NaN,NaN,NaN,NaN,NaN,NaN,0.080817,2
2022-06-16,SEK,GBP,0.08045,0.08065,0.07925,0.07963,0.08044,0.08145,0.08056,NaN,NaN,NaN,NaN,NaN,0.080520,3
2022-06-17,SEK,GBP,0.07965,0.08018,0.07946,0.08014,0.07963,0.08044,0.08145,0.08056,NaN,NaN,NaN,NaN,0.080444,4
2022-06-20,SEK,GBP,0.08030,0.08062,0.08005,0.08046,0.08014,0.07963,0.08044,0.08145,0.08056,NaN,NaN,NaN,0.080447,5
2022-06-21,SEK,GBP,0.08043,0.08067,0.08031,0.08042,0.08046,0.08014,0.07963,0.08044,0.08145,0.08056,NaN,0.080443,0.080443,6
2022-06-22,SEK,GBP,0.08041,0.08068,0.08007,0.08051,0.08042,0.08046,0.08014,0.07963,0.08044,0.08145,0.08056,0.080436,0.080451,7
2022-06-23,SEK,GBP,0.08053,0.08066,0.07975,0.07983,0.08051,0.08042,0.08046,0.08014,0.07963,0.08044,0.08145,0.080204,0.080382,8


In [81]:
sekaud = app.get_currency_exchange_daily('SEK', 'AUD')
sekaud1 = sekaud[0]
sekaud1['to symbol'] = 'AUD'
sekaud1['from symbol'] = 'SEK'

sekaud1.rename(columns={'1. open': 'open'}, inplace=True)
sekaud1.rename(columns={'2. high': 'high'}, inplace=True)
sekaud1.rename(columns={'3. low': 'low'}, inplace=True)
sekaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekaud1.index)
sekaud1 = sekaud1.sort_index()

sekaud1['time'] = np.arange(len(sekaud1.index))

sekaud1['lag_1'] = sekaud1['close'].shift(1)
sekaud1['lag_2'] = sekaud1['close'].shift(2)
sekaud1['lag_3'] = sekaud1['close'].shift(3)
sekaud1['lag_4'] = sekaud1['close'].shift(4)
sekaud1['lag_5'] = sekaud1['close'].shift(5)
sekaud1['lag_6'] = sekaud1['close'].shift(6)
sekaud1['lag_7'] = sekaud1['close'].shift(7)

sekaud1['rolling_mean'] = sekaud1['close'].rolling(window=7).mean()
sekaud1['expanding_mean'] = sekaud1['close'].expanding(2).mean()

sekaud1 = sekaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekaud1])

sekaud1.to_pickle('sekaud1.pkl')

sekaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,AUD,0.14205,0.14222,0.14101,0.14149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,AUD,0.14147,0.14293,0.14115,0.14256,0.14149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.142025,1
2022-06-15,SEK,AUD,0.14260,0.14279,0.13982,0.14017,0.14256,0.14149,NaN,NaN,NaN,NaN,NaN,NaN,0.141407,2
2022-06-16,SEK,AUD,0.14018,0.14070,0.13919,0.13996,0.14017,0.14256,0.14149,NaN,NaN,NaN,NaN,NaN,0.141045,3
2022-06-17,SEK,AUD,0.13999,0.14184,0.13971,0.14169,0.13996,0.14017,0.14256,0.14149,NaN,NaN,NaN,NaN,0.141174,4
2022-06-20,SEK,AUD,0.14185,0.14214,0.14077,0.14209,0.14169,0.13996,0.14017,0.14256,0.14149,NaN,NaN,NaN,0.141327,5
2022-06-21,SEK,AUD,0.14200,0.14280,0.14168,0.14209,0.14209,0.14169,0.13996,0.14017,0.14256,0.14149,NaN,0.141436,0.141436,6
2022-06-22,SEK,AUD,0.14203,0.14329,0.14185,0.14300,0.14209,0.14209,0.14169,0.13996,0.14017,0.14256,0.14149,0.141651,0.141631,7
2022-06-23,SEK,AUD,0.14301,0.14376,0.14191,0.14231,0.14300,0.14209,0.14209,0.14169,0.13996,0.14017,0.14256,0.141616,0.141707,8


In [82]:
sekchf = app.get_currency_exchange_daily('SEK', 'CHF')
sekchf1 = sekchf[0]
sekchf1['to symbol'] = 'CHF'
sekchf1['from symbol'] = 'SEK'

sekchf1.rename(columns={'1. open': 'open'}, inplace=True)
sekchf1.rename(columns={'2. high': 'high'}, inplace=True)
sekchf1.rename(columns={'3. low': 'low'}, inplace=True)
sekchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekchf1.index)
sekchf1 = sekchf1.sort_index()

sekchf1['time'] = np.arange(len(sekchf1.index))

sekchf1['lag_1'] = sekchf1['close'].shift(1)
sekchf1['lag_2'] = sekchf1['close'].shift(2)
sekchf1['lag_3'] = sekchf1['close'].shift(3)
sekchf1['lag_4'] = sekchf1['close'].shift(4)
sekchf1['lag_5'] = sekchf1['close'].shift(5)
sekchf1['lag_6'] = sekchf1['close'].shift(6)
sekchf1['lag_7'] = sekchf1['close'].shift(7)

sekchf1['rolling_mean'] = sekchf1['close'].rolling(window=7).mean()
sekchf1['expanding_mean'] = sekchf1['close'].expanding(2).mean()

sekchf1 = sekchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekchf1])

sekchf1.to_pickle('sekchf1.pkl')

sekchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,CHF,0.09830,0.09838,0.0971,0.09770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,CHF,0.09720,0.09837,0.0969,0.09814,0.09770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097920,1
2022-06-15,SEK,CHF,0.09760,0.09871,0.0968,0.09757,0.09814,0.09770,NaN,NaN,NaN,NaN,NaN,NaN,0.097803,2
2022-06-16,SEK,CHF,0.09720,0.09794,0.0941,0.09490,0.09757,0.09814,0.09770,NaN,NaN,NaN,NaN,NaN,0.097077,3
2022-06-17,SEK,CHF,0.09480,0.09555,0.0940,0.09482,0.09490,0.09757,0.09814,0.09770,NaN,NaN,NaN,NaN,0.096626,4
2022-06-20,SEK,CHF,0.09470,0.09576,0.0943,0.09548,0.09482,0.09490,0.09757,0.09814,0.09770,NaN,NaN,NaN,0.096435,5
2022-06-21,SEK,CHF,0.09549,0.09617,0.0950,0.09557,0.09548,0.09482,0.09490,0.09757,0.09814,0.09770,NaN,0.096311,0.096311,6
2022-06-22,SEK,CHF,0.09510,0.09560,0.0943,0.09460,0.09557,0.09548,0.09482,0.09490,0.09757,0.09814,0.09770,0.095869,0.096098,7
2022-06-23,SEK,CHF,0.09470,0.09526,0.0934,0.09431,0.09460,0.09557,0.09548,0.09482,0.09490,0.09757,0.09814,0.095321,0.095899,8


In [83]:
sekcad = app.get_currency_exchange_daily('SEK', 'CAD')
sekcad1 = sekcad[0]
sekcad1['to symbol'] = 'CAD'
sekcad1['from symbol'] = 'SEK'

sekcad1.rename(columns={'1. open': 'open'}, inplace=True)
sekcad1.rename(columns={'2. high': 'high'}, inplace=True)
sekcad1.rename(columns={'3. low': 'low'}, inplace=True)
sekcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekcad1.index)
sekcad1 = sekcad1.sort_index()

sekcad1['time'] = np.arange(len(sekcad1.index))

sekcad1['lag_1'] = sekcad1['close'].shift(1)
sekcad1['lag_2'] = sekcad1['close'].shift(2)
sekcad1['lag_3'] = sekcad1['close'].shift(3)
sekcad1['lag_4'] = sekcad1['close'].shift(4)
sekcad1['lag_5'] = sekcad1['close'].shift(5)
sekcad1['lag_6'] = sekcad1['close'].shift(6)
sekcad1['lag_7'] = sekcad1['close'].shift(7)

sekcad1['rolling_mean'] = sekcad1['close'].rolling(window=7).mean()
sekcad1['expanding_mean'] = sekcad1['close'].expanding(2).mean()

sekcad1 = sekcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekcad1])

sekcad1.to_pickle('sekcad1.pkl')

sekcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,CAD,0.1269,0.1269,0.1252,0.1255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,CAD,0.1254,0.1265,0.1253,0.1261,0.1255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125800,1
2022-06-15,SEK,CAD,0.1260,0.1272,0.1251,0.1256,0.1261,0.1255,NaN,NaN,NaN,NaN,NaN,NaN,0.125733,2
2022-06-16,SEK,CAD,0.1256,0.1268,0.1246,0.1268,0.1256,0.1261,0.1255,NaN,NaN,NaN,NaN,NaN,0.126000,3
2022-06-17,SEK,CAD,0.1268,0.1272,0.1262,0.1270,0.1268,0.1256,0.1261,0.1255,NaN,NaN,NaN,NaN,0.126200,4
2022-06-20,SEK,CAD,0.1273,0.1277,0.1267,0.1273,0.1270,0.1268,0.1256,0.1261,0.1255,NaN,NaN,NaN,0.126383,5
2022-06-21,SEK,CAD,0.1273,0.1277,0.1268,0.1270,0.1273,0.1270,0.1268,0.1256,0.1261,0.1255,NaN,0.126471,0.126471,6
2022-06-22,SEK,CAD,0.1269,0.1277,0.1264,0.1273,0.1270,0.1273,0.1270,0.1268,0.1256,0.1261,0.1255,0.126729,0.126575,7
2022-06-23,SEK,CAD,0.1273,0.1275,0.1260,0.1266,0.1273,0.1270,0.1273,0.1270,0.1268,0.1256,0.1261,0.126800,0.126578,8


In [84]:
sekhkd = app.get_currency_exchange_daily('SEK', 'HKD')
sekhkd1 = sekhkd[0]
sekhkd1['to symbol'] = 'HKD'
sekhkd1['from symbol'] = 'SEK'

sekhkd1.rename(columns={'1. open': 'open'}, inplace=True)
sekhkd1.rename(columns={'2. high': 'high'}, inplace=True)
sekhkd1.rename(columns={'3. low': 'low'}, inplace=True)
sekhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(sekhkd1.index)
sekhkd1 = sekhkd1.sort_index()

sekhkd1['time'] = np.arange(len(sekhkd1.index))

sekhkd1['lag_1'] = sekhkd1['close'].shift(1)
sekhkd1['lag_2'] = sekhkd1['close'].shift(2)
sekhkd1['lag_3'] = sekhkd1['close'].shift(3)
sekhkd1['lag_4'] = sekhkd1['close'].shift(4)
sekhkd1['lag_5'] = sekhkd1['close'].shift(5)
sekhkd1['lag_6'] = sekhkd1['close'].shift(6)
sekhkd1['lag_7'] = sekhkd1['close'].shift(7)

sekhkd1['rolling_mean'] = sekhkd1['close'].rolling(window=7).mean()
sekhkd1['expanding_mean'] = sekhkd1['close'].expanding(2).mean()

sekhkd1 = sekhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, sekhkd1])

sekhkd1.to_pickle('sekhkd1.pkl')

sekhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,HKD,0.7822,0.7822,0.7668,0.7672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,HKD,0.7673,0.7732,0.7653,0.7671,0.7672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.767150,1
2022-06-15,SEK,HKD,0.7674,0.7742,0.7598,0.7685,0.7671,0.7672,NaN,NaN,NaN,NaN,NaN,NaN,0.767600,2
2022-06-16,SEK,HKD,0.7686,0.7743,0.7593,0.7721,0.7685,0.7671,0.7672,NaN,NaN,NaN,NaN,NaN,0.768725,3
2022-06-17,SEK,HKD,0.7722,0.7725,0.7640,0.7689,0.7721,0.7685,0.7671,0.7672,NaN,NaN,NaN,NaN,0.768760,4
2022-06-20,SEK,HKD,0.7699,0.7749,0.7677,0.7735,0.7689,0.7721,0.7685,0.7671,0.7672,NaN,NaN,NaN,0.769550,5
2022-06-21,SEK,HKD,0.7732,0.7794,0.7727,0.7752,0.7735,0.7689,0.7721,0.7685,0.7671,0.7672,NaN,0.770357,0.770357,6
2022-06-22,SEK,HKD,0.7748,0.7787,0.7680,0.7751,0.7752,0.7735,0.7689,0.7721,0.7685,0.7671,0.7672,0.771486,0.770950,7
2022-06-23,SEK,HKD,0.7753,0.7761,0.7663,0.7682,0.7751,0.7752,0.7735,0.7689,0.7721,0.7685,0.7671,0.771643,0.770644,8


In [85]:
seknzd = app.get_currency_exchange_daily('SEK', 'NZD')
seknzd1 = seknzd[0]
seknzd1['to symbol'] = 'NZD'
seknzd1['from symbol'] = 'SEK'

seknzd1.rename(columns={'1. open': 'open'}, inplace=True)
seknzd1.rename(columns={'2. high': 'high'}, inplace=True)
seknzd1.rename(columns={'3. low': 'low'}, inplace=True)
seknzd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(seknzd1.index)
seknzd1 = seknzd1.sort_index()

seknzd1['time'] = np.arange(len(seknzd1.index))

seknzd1['lag_1'] = seknzd1['close'].shift(1)
seknzd1['lag_2'] = seknzd1['close'].shift(2)
seknzd1['lag_3'] = seknzd1['close'].shift(3)
seknzd1['lag_4'] = seknzd1['close'].shift(4)
seknzd1['lag_5'] = seknzd1['close'].shift(5)
seknzd1['lag_6'] = seknzd1['close'].shift(6)
seknzd1['lag_7'] = seknzd1['close'].shift(7)

seknzd1['rolling_mean'] = seknzd1['close'].rolling(window=7).mean()
seknzd1['expanding_mean'] = seknzd1['close'].expanding(2).mean()

seknzd1 = seknzd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, seknzd1])

seknzd1.to_pickle('seknzd1.pkl')

seknzd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,SEK,NZD,0.1526,0.1526,0.1513,0.1516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,SEK,NZD,0.1515,0.1529,0.1513,0.1527,0.1516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.152150,1
2022-06-15,SEK,NZD,0.1527,0.1536,0.1506,0.1513,0.1527,0.1516,NaN,NaN,NaN,NaN,NaN,NaN,0.151867,2
2022-06-16,SEK,NZD,0.1512,0.1518,0.1495,0.1501,0.1513,0.1527,0.1516,NaN,NaN,NaN,NaN,NaN,0.151425,3
2022-06-17,SEK,NZD,0.1502,0.1511,0.1497,0.1507,0.1501,0.1513,0.1527,0.1516,NaN,NaN,NaN,NaN,0.151280,4
2022-06-20,SEK,NZD,0.1509,0.1511,0.1498,0.1511,0.1507,0.1501,0.1513,0.1527,0.1516,NaN,NaN,NaN,0.151250,5
2022-06-21,SEK,NZD,0.1510,0.1518,0.1509,0.1514,0.1511,0.1507,0.1501,0.1513,0.1527,0.1516,NaN,0.151271,0.151271,6
2022-06-22,SEK,NZD,0.1514,0.1529,0.1514,0.1526,0.1514,0.1511,0.1507,0.1501,0.1513,0.1527,0.1516,0.151414,0.151438,7
2022-06-23,SEK,NZD,0.1526,0.1532,0.1509,0.1513,0.1526,0.1514,0.1511,0.1507,0.1501,0.1513,0.1527,0.151214,0.151422,8


In [86]:
nzdusd = app.get_currency_exchange_daily('NZD', 'USD')
nzdusd1 = nzdusd[0]
nzdusd1['to symbol'] = 'USD'
nzdusd1['from symbol'] = 'NZD'

nzdusd1.rename(columns={'1. open': 'open'}, inplace=True)
nzdusd1.rename(columns={'2. high': 'high'}, inplace=True)
nzdusd1.rename(columns={'3. low': 'low'}, inplace=True)
nzdusd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdusd1.index)
nzdusd1 = nzdusd1.sort_index()

nzdusd1['time'] = np.arange(len(nzdusd1.index))

nzdusd1['lag_1'] = nzdusd1['close'].shift(1)
nzdusd1['lag_2'] = nzdusd1['close'].shift(2)
nzdusd1['lag_3'] = nzdusd1['close'].shift(3)
nzdusd1['lag_4'] = nzdusd1['close'].shift(4)
nzdusd1['lag_5'] = nzdusd1['close'].shift(5)
nzdusd1['lag_6'] = nzdusd1['close'].shift(6)
nzdusd1['lag_7'] = nzdusd1['close'].shift(7)

nzdusd1['rolling_mean'] = nzdusd1['close'].rolling(window=7).mean()
nzdusd1['expanding_mean'] = nzdusd1['close'].expanding(2).mean()

nzdusd1 = nzdusd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdusd1])

nzdusd1.to_pickle('nzdusd1.pkl')

nzdusd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,USD,0.63446,0.63503,0.62440,0.62606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,USD,0.62610,0.62934,0.61940,0.62141,0.62606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.623735,1
2022-06-15,NZD,USD,0.62164,0.63131,0.62120,0.62840,0.62141,0.62606,NaN,NaN,NaN,NaN,NaN,NaN,0.625290,2
2022-06-16,NZD,USD,0.62848,0.63957,0.62310,0.63596,0.62840,0.62141,0.62606,NaN,NaN,NaN,NaN,NaN,0.627957,3
2022-06-17,NZD,USD,0.63617,0.63704,0.62670,0.63100,0.63596,0.62840,0.62141,0.62606,NaN,NaN,NaN,NaN,0.628566,4
2022-06-20,NZD,USD,0.63097,0.63640,0.63015,0.63288,0.63100,0.63596,0.62840,0.62141,0.62606,NaN,NaN,NaN,0.629285,5
2022-06-21,NZD,USD,0.63246,0.63634,0.63160,0.63311,0.63288,0.63100,0.63596,0.62840,0.62141,0.62606,NaN,0.629831,0.629831,6
2022-06-22,NZD,USD,0.63328,0.63328,0.62410,0.62850,0.63311,0.63288,0.63100,0.63596,0.62840,0.62141,0.62606,0.630180,0.629665,7
2022-06-23,NZD,USD,0.62861,0.63107,0.62450,0.62756,0.62850,0.63311,0.63288,0.63100,0.63596,0.62840,0.62141,0.631059,0.629431,8


In [87]:
nzdeur = app.get_currency_exchange_daily('NZD', 'EUR')
nzdeur1 = nzdeur[0]
nzdeur1['to symbol'] = 'EUR'
nzdeur1['from symbol'] = 'NZD'

nzdeur1.rename(columns={'1. open': 'open'}, inplace=True)
nzdeur1.rename(columns={'2. high': 'high'}, inplace=True)
nzdeur1.rename(columns={'3. low': 'low'}, inplace=True)
nzdeur1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdeur1.index)
nzdeur1 = nzdeur1.sort_index()

nzdeur1['time'] = np.arange(len(nzdeur1.index))

nzdeur1['lag_1'] = nzdeur1['close'].shift(1)
nzdeur1['lag_2'] = nzdeur1['close'].shift(2)
nzdeur1['lag_3'] = nzdeur1['close'].shift(3)
nzdeur1['lag_4'] = nzdeur1['close'].shift(4)
nzdeur1['lag_5'] = nzdeur1['close'].shift(5)
nzdeur1['lag_6'] = nzdeur1['close'].shift(6)
nzdeur1['lag_7'] = nzdeur1['close'].shift(7)

nzdeur1['rolling_mean'] = nzdeur1['close'].rolling(window=7).mean()
nzdeur1['expanding_mean'] = nzdeur1['close'].expanding(2).mean()

nzdeur1 = nzdeur1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdeur1])

nzdeur1.to_pickle('nzdeur1.pkl')

nzdeur1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,EUR,0.60326,0.60532,0.59960,0.60087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,EUR,0.60118,0.60273,0.59510,0.59632,0.60087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.598595,1
2022-06-15,NZD,EUR,0.59635,0.60356,0.59290,0.60120,0.59632,0.60087,NaN,NaN,NaN,NaN,NaN,NaN,0.599463,2
2022-06-16,NZD,EUR,0.60105,0.60417,0.59910,0.60220,0.60120,0.59632,0.60087,NaN,NaN,NaN,NaN,NaN,0.600147,3
2022-06-17,NZD,EUR,0.60135,0.60372,0.59920,0.60080,0.60220,0.60120,0.59632,0.60087,NaN,NaN,NaN,NaN,0.600278,4
2022-06-20,NZD,EUR,0.60109,0.60441,0.60030,0.60180,0.60080,0.60220,0.60120,0.59632,0.60087,NaN,NaN,NaN,0.600532,5
2022-06-21,NZD,EUR,0.60109,0.60266,0.59850,0.60060,0.60180,0.60080,0.60220,0.60120,0.59632,0.60087,NaN,0.600541,0.600541,6
2022-06-22,NZD,EUR,0.60063,0.60084,0.59303,0.59459,0.60060,0.60180,0.60080,0.60220,0.60120,0.59632,0.60087,0.599644,0.599797,7
2022-06-23,NZD,EUR,0.59423,0.59831,0.59160,0.59630,0.59459,0.60060,0.60180,0.60080,0.60220,0.60120,0.59632,0.599641,0.599409,8


In [88]:
nzdjpy = app.get_currency_exchange_daily('NZD', 'JPY')
nzdjpy1 = nzdjpy[0]
nzdjpy1['to symbol'] = 'JPY'
nzdjpy1['from symbol'] = 'NZD'

nzdjpy1.rename(columns={'1. open': 'open'}, inplace=True)
nzdjpy1.rename(columns={'2. high': 'high'}, inplace=True)
nzdjpy1.rename(columns={'3. low': 'low'}, inplace=True)
nzdjpy1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdjpy1.index)
nzdjpy1 = nzdjpy1.sort_index()

nzdjpy1['time'] = np.arange(len(nzdjpy1.index))

nzdjpy1['lag_1'] = nzdjpy1['close'].shift(1)
nzdjpy1['lag_2'] = nzdjpy1['close'].shift(2)
nzdjpy1['lag_3'] = nzdjpy1['close'].shift(3)
nzdjpy1['lag_4'] = nzdjpy1['close'].shift(4)
nzdjpy1['lag_5'] = nzdjpy1['close'].shift(5)
nzdjpy1['lag_6'] = nzdjpy1['close'].shift(6)
nzdjpy1['lag_7'] = nzdjpy1['close'].shift(7)

nzdjpy1['rolling_mean'] = nzdjpy1['close'].rolling(window=7).mean()
nzdjpy1['expanding_mean'] = nzdjpy1['close'].expanding(2).mean()

nzdjpy1 = nzdjpy1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdjpy1])

nzdjpy1.to_pickle('nzdjpy1.pkl')

nzdjpy1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,JPY,85.287,85.728,83.732,84.161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,JPY,84.157,84.720,83.603,84.268,84.161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.214500,1
2022-06-15,NZD,JPY,84.262,84.411,83.697,84.103,84.268,84.161,NaN,NaN,NaN,NaN,NaN,NaN,84.177333,2
2022-06-16,NZD,JPY,84.105,84.718,82.963,84.136,84.103,84.268,84.161,NaN,NaN,NaN,NaN,NaN,84.167000,3
2022-06-17,NZD,JPY,84.160,85.495,83.949,85.211,84.136,84.103,84.268,84.161,NaN,NaN,NaN,NaN,84.375800,4
2022-06-20,NZD,JPY,85.211,85.851,85.005,85.492,85.211,84.136,84.103,84.268,84.161,NaN,NaN,NaN,84.561833,5
2022-06-21,NZD,JPY,85.487,86.574,85.363,86.511,85.492,85.211,84.136,84.103,84.268,84.161,NaN,84.840286,84.840286,6
2022-06-22,NZD,JPY,86.496,86.558,84.857,85.555,86.511,85.492,85.211,84.136,84.103,84.268,84.161,85.039429,84.929625,7
2022-06-23,NZD,JPY,85.543,85.613,84.373,84.685,85.555,86.511,85.492,85.211,84.136,84.103,84.268,85.099000,84.902444,8


In [89]:
nzdgbp = app.get_currency_exchange_daily('NZD', 'GBP')
nzdgbp1 = nzdgbp[0]
nzdgbp1['to symbol'] = 'GBP'
nzdgbp1['from symbol'] = 'NZD'

nzdgbp1.rename(columns={'1. open': 'open'}, inplace=True)
nzdgbp1.rename(columns={'2. high': 'high'}, inplace=True)
nzdgbp1.rename(columns={'3. low': 'low'}, inplace=True)
nzdgbp1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdgbp1.index)
nzdgbp1 = nzdgbp1.sort_index()

nzdgbp1['time'] = np.arange(len(nzdgbp1.index))

nzdgbp1['lag_1'] = nzdgbp1['close'].shift(1)
nzdgbp1['lag_2'] = nzdgbp1['close'].shift(2)
nzdgbp1['lag_3'] = nzdgbp1['close'].shift(3)
nzdgbp1['lag_4'] = nzdgbp1['close'].shift(4)
nzdgbp1['lag_5'] = nzdgbp1['close'].shift(5)
nzdgbp1['lag_6'] = nzdgbp1['close'].shift(6)
nzdgbp1['lag_7'] = nzdgbp1['close'].shift(7)

nzdgbp1['rolling_mean'] = nzdgbp1['close'].rolling(window=7).mean()
nzdgbp1['expanding_mean'] = nzdgbp1['close'].expanding(2).mean()

nzdgbp1 = nzdgbp1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdgbp1])

nzdgbp1.to_pickle('nzdgbp1.pkl')

nzdgbp1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,GBP,0.51487,0.51773,0.51450,0.51558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,GBP,0.51568,0.51907,0.51370,0.51759,0.51558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516585,1
2022-06-15,NZD,GBP,0.51760,0.51858,0.51540,0.51590,0.51759,0.51558,NaN,NaN,NaN,NaN,NaN,NaN,0.516357,2
2022-06-16,NZD,GBP,0.51626,0.51913,0.51220,0.51460,0.51590,0.51759,0.51558,NaN,NaN,NaN,NaN,NaN,0.515918,3
2022-06-17,NZD,GBP,0.51430,0.51658,0.51310,0.51600,0.51460,0.51590,0.51759,0.51558,NaN,NaN,NaN,NaN,0.515934,4
2022-06-20,NZD,GBP,0.51610,0.51946,0.51590,0.51650,0.51600,0.51460,0.51590,0.51759,0.51558,NaN,NaN,NaN,0.516028,5
2022-06-21,NZD,GBP,0.51679,0.51738,0.51490,0.51520,0.51650,0.51600,0.51460,0.51590,0.51759,0.51558,NaN,0.515910,0.515910,6
2022-06-22,NZD,GBP,0.51525,0.51556,0.50920,0.51210,0.51520,0.51650,0.51600,0.51460,0.51590,0.51759,0.51558,0.515413,0.515434,7
2022-06-23,NZD,GBP,0.51180,0.51381,0.51010,0.51170,0.51210,0.51520,0.51650,0.51600,0.51460,0.51590,0.51759,0.514571,0.515019,8


In [90]:
nzdaud = app.get_currency_exchange_daily('NZD', 'AUD')
nzdaud1 = nzdaud[0]
nzdaud1['to symbol'] = 'AUD'
nzdaud1['from symbol'] = 'NZD'

nzdaud1.rename(columns={'1. open': 'open'}, inplace=True)
nzdaud1.rename(columns={'2. high': 'high'}, inplace=True)
nzdaud1.rename(columns={'3. low': 'low'}, inplace=True)
nzdaud1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdaud1.index)
nzdaud1 = nzdaud1.sort_index()

nzdaud1['time'] = np.arange(len(nzdaud1.index))

nzdaud1['lag_1'] = nzdaud1['close'].shift(1)
nzdaud1['lag_2'] = nzdaud1['close'].shift(2)
nzdaud1['lag_3'] = nzdaud1['close'].shift(3)
nzdaud1['lag_4'] = nzdaud1['close'].shift(4)
nzdaud1['lag_5'] = nzdaud1['close'].shift(5)
nzdaud1['lag_6'] = nzdaud1['close'].shift(6)
nzdaud1['lag_7'] = nzdaud1['close'].shift(7)

nzdaud1['rolling_mean'] = nzdaud1['close'].rolling(window=7).mean()
nzdaud1['expanding_mean'] = nzdaud1['close'].expanding(2).mean()

nzdaud1 = nzdaud1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdaud1])

nzdaud1.to_pickle('nzdaud1.pkl')

nzdaud1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,AUD,0.90161,0.90440,0.90050,0.90283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,AUD,0.90281,0.90610,0.90090,0.90360,0.90283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.903215,1
2022-06-15,NZD,AUD,0.90375,0.90441,0.89509,0.89607,0.90360,0.90283,NaN,NaN,NaN,NaN,NaN,NaN,0.900833,2
2022-06-16,NZD,AUD,0.89634,0.90586,0.89510,0.90121,0.89607,0.90360,0.90283,NaN,NaN,NaN,NaN,NaN,0.900927,3
2022-06-17,NZD,AUD,0.90044,0.91015,0.90044,0.91010,0.90121,0.89607,0.90360,0.90283,NaN,NaN,NaN,NaN,0.902762,4
2022-06-20,NZD,AUD,0.90899,0.91121,0.90800,0.90950,0.91010,0.90121,0.89607,0.90360,0.90283,NaN,NaN,NaN,0.903885,5
2022-06-21,NZD,AUD,0.90907,0.91170,0.90703,0.90767,0.90950,0.91010,0.90121,0.89607,0.90360,0.90283,NaN,0.904426,0.904426,6
2022-06-22,NZD,AUD,0.90771,0.90823,0.90440,0.90699,0.90767,0.90950,0.91010,0.90121,0.89607,0.90360,0.90283,0.905020,0.904746,7
2022-06-23,NZD,AUD,0.90663,0.91135,0.90610,0.90948,0.90699,0.90767,0.90950,0.91010,0.90121,0.89607,0.90360,0.905860,0.905272,8


In [91]:
nzdchf = app.get_currency_exchange_daily('NZD', 'CHF')
nzdchf1 = nzdchf[0]
nzdchf1['to symbol'] = 'CHF'
nzdchf1['from symbol'] = 'NZD'

nzdchf1.rename(columns={'1. open': 'open'}, inplace=True)
nzdchf1.rename(columns={'2. high': 'high'}, inplace=True)
nzdchf1.rename(columns={'3. low': 'low'}, inplace=True)
nzdchf1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdchf1.index)
nzdchf1 = nzdchf1.sort_index()

nzdchf1['time'] = np.arange(len(nzdchf1.index))

nzdchf1['lag_1'] = nzdchf1['close'].shift(1)
nzdchf1['lag_2'] = nzdchf1['close'].shift(2)
nzdchf1['lag_3'] = nzdchf1['close'].shift(3)
nzdchf1['lag_4'] = nzdchf1['close'].shift(4)
nzdchf1['lag_5'] = nzdchf1['close'].shift(5)
nzdchf1['lag_6'] = nzdchf1['close'].shift(6)
nzdchf1['lag_7'] = nzdchf1['close'].shift(7)

nzdchf1['rolling_mean'] = nzdchf1['close'].rolling(window=7).mean()
nzdchf1['expanding_mean'] = nzdchf1['close'].expanding(2).mean()

nzdchf1 = nzdchf1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdchf1])

nzdchf1.to_pickle('nzdchf1.pkl')

nzdchf1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,CHF,0.62679,0.62815,0.62250,0.62439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,CHF,0.62430,0.62644,0.61780,0.62259,0.62439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.623490,1
2022-06-15,NZD,CHF,0.62261,0.62765,0.62089,0.62461,0.62259,0.62439,NaN,NaN,NaN,NaN,NaN,NaN,0.623863,2
2022-06-16,NZD,CHF,0.62466,0.62744,0.60956,0.61436,0.62461,0.62259,0.62439,NaN,NaN,NaN,NaN,NaN,0.621487,3
2022-06-17,NZD,CHF,0.61474,0.61583,0.60781,0.61210,0.61436,0.62461,0.62259,0.62439,NaN,NaN,NaN,NaN,0.619610,4
2022-06-20,NZD,CHF,0.61201,0.61469,0.60925,0.61201,0.61210,0.61436,0.62461,0.62259,0.62439,NaN,NaN,NaN,0.618343,5
2022-06-21,NZD,CHF,0.61156,0.61501,0.60938,0.61101,0.61201,0.61210,0.61436,0.62461,0.62259,0.62439,NaN,0.617296,0.617296,6
2022-06-22,NZD,CHF,0.61154,0.61256,0.60090,0.60376,0.61101,0.61201,0.61210,0.61436,0.62461,0.62259,0.62439,0.614349,0.615604,7
2022-06-23,NZD,CHF,0.60397,0.60636,0.60108,0.60324,0.60376,0.61101,0.61201,0.61210,0.61436,0.62461,0.62259,0.611584,0.614230,8


In [92]:
nzdcad = app.get_currency_exchange_daily('NZD', 'CAD')
nzdcad1 = nzdcad[0]
nzdcad1['to symbol'] = 'CAD'
nzdcad1['from symbol'] = 'NZD'

nzdcad1.rename(columns={'1. open': 'open'}, inplace=True)
nzdcad1.rename(columns={'2. high': 'high'}, inplace=True)
nzdcad1.rename(columns={'3. low': 'low'}, inplace=True)
nzdcad1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdcad1.index)
nzdcad1 = nzdcad1.sort_index()

nzdcad1['time'] = np.arange(len(nzdcad1.index))

nzdcad1['lag_1'] = nzdcad1['close'].shift(1)
nzdcad1['lag_2'] = nzdcad1['close'].shift(2)
nzdcad1['lag_3'] = nzdcad1['close'].shift(3)
nzdcad1['lag_4'] = nzdcad1['close'].shift(4)
nzdcad1['lag_5'] = nzdcad1['close'].shift(5)
nzdcad1['lag_6'] = nzdcad1['close'].shift(6)
nzdcad1['lag_7'] = nzdcad1['close'].shift(7)

nzdcad1['rolling_mean'] = nzdcad1['close'].rolling(window=7).mean()
nzdcad1['expanding_mean'] = nzdcad1['close'].expanding(2).mean()

nzdcad1 = nzdcad1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdcad1])

nzdcad1.to_pickle('nzdcad1.pkl')

nzdcad1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,CAD,0.81086,0.81313,0.80502,0.80743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,CAD,0.80741,0.81049,0.80275,0.80504,0.80743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.806235,1
2022-06-15,NZD,CAD,0.80535,0.81318,0.80386,0.80977,0.80504,0.80743,NaN,NaN,NaN,NaN,NaN,NaN,0.807413,2
2022-06-16,NZD,CAD,0.80967,0.82557,0.80711,0.82353,0.80977,0.80504,0.80743,NaN,NaN,NaN,NaN,NaN,0.811442,3
2022-06-17,NZD,CAD,0.82384,0.82469,0.81916,0.82230,0.82353,0.80977,0.80504,0.80743,NaN,NaN,NaN,NaN,0.813614,4
2022-06-20,NZD,CAD,0.82184,0.82683,0.82062,0.82148,0.82230,0.82353,0.80977,0.80504,0.80743,NaN,NaN,NaN,0.814925,5
2022-06-21,NZD,CAD,0.82128,0.82267,0.81662,0.81775,0.82148,0.82230,0.82353,0.80977,0.80504,0.80743,NaN,0.815329,0.815329,6
2022-06-22,NZD,CAD,0.81793,0.81812,0.81049,0.81386,0.81775,0.82148,0.82230,0.82353,0.80977,0.80504,0.80743,0.816247,0.815145,7
2022-06-23,NZD,CAD,0.81372,0.81698,0.81054,0.81564,0.81386,0.81775,0.82148,0.82230,0.82353,0.80977,0.80504,0.817761,0.815200,8


In [93]:
nzdhkd = app.get_currency_exchange_daily('NZD', 'HKD')
nzdhkd1 = nzdhkd[0]
nzdhkd1['to symbol'] = 'HKD'
nzdhkd1['from symbol'] = 'NZD'

nzdhkd1.rename(columns={'1. open': 'open'}, inplace=True)
nzdhkd1.rename(columns={'2. high': 'high'}, inplace=True)
nzdhkd1.rename(columns={'3. low': 'low'}, inplace=True)
nzdhkd1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdhkd1.index)
nzdhkd1 = nzdhkd1.sort_index()

nzdhkd1['time'] = np.arange(len(nzdhkd1.index))

nzdhkd1['lag_1'] = nzdhkd1['close'].shift(1)
nzdhkd1['lag_2'] = nzdhkd1['close'].shift(2)
nzdhkd1['lag_3'] = nzdhkd1['close'].shift(3)
nzdhkd1['lag_4'] = nzdhkd1['close'].shift(4)
nzdhkd1['lag_5'] = nzdhkd1['close'].shift(5)
nzdhkd1['lag_6'] = nzdhkd1['close'].shift(6)
nzdhkd1['lag_7'] = nzdhkd1['close'].shift(7)

nzdhkd1['rolling_mean'] = nzdhkd1['close'].rolling(window=7).mean()
nzdhkd1['expanding_mean'] = nzdhkd1['close'].expanding(2).mean()

nzdhkd1 = nzdhkd1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdhkd1])

nzdhkd1.to_pickle('nzdhkd1.pkl')

nzdhkd1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,HKD,4.9789,4.9828,4.9041,4.9147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,HKD,4.9170,4.9374,4.8637,4.8787,4.9147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.896700,1
2022-06-15,NZD,HKD,4.8794,4.9532,4.8785,4.9321,4.8787,4.9147,NaN,NaN,NaN,NaN,NaN,NaN,4.908500,2
2022-06-16,NZD,HKD,4.9339,5.0168,4.8921,4.9935,4.9321,4.8787,4.9147,NaN,NaN,NaN,NaN,NaN,4.929750,3
2022-06-17,NZD,HKD,4.9905,4.9970,4.9217,4.9545,4.9935,4.9321,4.8787,4.9147,NaN,NaN,NaN,NaN,4.934700,4
2022-06-20,NZD,HKD,4.9530,4.9938,4.9497,4.9695,4.9545,4.9935,4.9321,4.8787,4.9147,NaN,NaN,NaN,4.940500,5
2022-06-21,NZD,HKD,4.9694,4.9927,4.9584,4.9700,4.9695,4.9545,4.9935,4.9321,4.8787,4.9147,NaN,4.944714,4.944714,6
2022-06-22,NZD,HKD,4.9687,4.9687,4.9007,4.9341,4.9700,4.9695,4.9545,4.9935,4.9321,4.8787,4.9147,4.947486,4.943388,7
2022-06-23,NZD,HKD,4.9329,4.9509,4.9032,4.9275,4.9341,4.9700,4.9695,4.9545,4.9935,4.9321,4.8787,4.954457,4.941622,8


In [94]:
nzdsek = app.get_currency_exchange_daily('NZD', 'SEK')
nzdsek1 = nzdsek[0]
nzdsek1['to symbol'] = 'SEK'
nzdsek1['from symbol'] = 'NZD'

nzdsek1.rename(columns={'1. open': 'open'}, inplace=True)
nzdsek1.rename(columns={'2. high': 'high'}, inplace=True)
nzdsek1.rename(columns={'3. low': 'low'}, inplace=True)
nzdsek1.rename(columns={'4. close': 'close'}, inplace=True)

pd.to_datetime(nzdsek1.index)
nzdsek1 = nzdsek1.sort_index()

nzdsek1['time'] = np.arange(len(nzdsek1.index))

nzdsek1['lag_1'] = nzdsek1['close'].shift(1)
nzdsek1['lag_2'] = nzdsek1['close'].shift(2)
nzdsek1['lag_3'] = nzdsek1['close'].shift(3)
nzdsek1['lag_4'] = nzdsek1['close'].shift(4)
nzdsek1['lag_5'] = nzdsek1['close'].shift(5)
nzdsek1['lag_6'] = nzdsek1['close'].shift(6)
nzdsek1['lag_7'] = nzdsek1['close'].shift(7)

nzdsek1['rolling_mean'] = nzdsek1['close'].rolling(window=7).mean()
nzdsek1['expanding_mean'] = nzdsek1['close'].expanding(2).mean()

nzdsek1 = nzdsek1[['from symbol', 'to symbol', 'open', 'high', 'low', 'close', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'rolling_mean', 'expanding_mean', 'time']]

all_pairs = pd.concat([all_pairs, nzdsek1])

nzdsek1.to_pickle('nzdsek1.pkl')

nzdsek1.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,NZD,SEK,6.3390,6.3950,6.3372,6.3686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,NZD,SEK,6.3692,6.3910,6.2783,6.3278,6.3686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.348200,1
2022-06-15,NZD,SEK,6.3275,6.4262,6.3001,6.3768,6.3278,6.3686,NaN,NaN,NaN,NaN,NaN,NaN,6.357733,2
2022-06-16,NZD,SEK,6.3764,6.4714,6.3561,6.4045,6.3768,6.3278,6.3686,NaN,NaN,NaN,NaN,NaN,6.369425,3
2022-06-17,NZD,SEK,6.4105,6.4586,6.4015,6.4193,6.4045,6.3768,6.3278,6.3686,NaN,NaN,NaN,NaN,6.379400,4
2022-06-20,NZD,SEK,6.3940,6.4579,6.3813,6.4007,6.4193,6.4045,6.3768,6.3278,6.3686,NaN,NaN,NaN,6.382950,5
2022-06-21,NZD,SEK,6.3940,6.4094,6.3451,6.3765,6.4007,6.4193,6.4045,6.3768,6.3278,6.3686,NaN,6.382029,6.382029,6
2022-06-22,NZD,SEK,6.3764,6.3898,6.3017,6.3095,6.3765,6.4007,6.4193,6.4045,6.3768,6.3278,6.3686,6.373586,6.372962,7
2022-06-23,NZD,SEK,6.3200,6.4114,6.3125,6.3780,6.3095,6.3765,6.4007,6.4193,6.4045,6.3768,6.3278,6.380757,6.373522,8


View all_pairs dataframe:

In [95]:
all_pairs

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time
date,,,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.960050,1
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,0.959033,2
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,0.956025,3
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,0.955340,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-24,NZD,SEK,6.4340,6.4685,6.2850,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.2585,6.3119,6.323357,6.411394,95
2022-10-25,NZD,SEK,6.3243,6.3800,6.2690,6.2852,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.2585,6.327171,6.410093,96
2022-10-26,NZD,SEK,6.2921,6.3517,6.2853,6.3005,6.2852,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.330543,6.408974,97


Save to pickle:

In [96]:
all_pairs.to_pickle('all_pairs.pkl')

# Part Three
Obtain and clean sentiment data

Create empty dataframe to concatenate to

In [97]:
all_sentiment = pd.DataFrame()

* Request from API
* Get json

In [98]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:USD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

The data that was returned is very layered:
* Convert to list
* Pop unneeded items

In [99]:
listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

('relevance_score_definition',
 '0 < x <= 1, with a higher score indicating higher relevance.')

Flatten list twice (I found this worked the best):

In [100]:
flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

Explode 'ticker sentiment' column:

In [101]:
dftest = dftest.explode('ticker_sentiment')

* Delete unneeded columns
* Reset index

In [102]:
del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

Use a for loop to access dictionary inside 'ticker sentiment' column:
* Even though the API returns data for the specified ticker, it also returns duplicate rows of the same data for other tickers that the same news article also applies to
* I want to keep the data as organized as possible and do not want to deal with messy duplicates later after requesting from the API for other tickers
* The for loop checks if the ticker equals the specified currency - if not, the row is dropped

In [103]:
drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:USD':
                drop_values.append(k)

* Create new dataframe from list
* Sort values by overall sentiment score

In [104]:
usdforexsentiment = dftest.drop(index=drop_values)
usdforexsentiment = usdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

* Add a column for symbol
* Set index to symbol

In [105]:
usdforexsentiment['symbol'] = 'USD'
usdforexsentiment.set_index('symbol', inplace=True)

* Concatenate to all_sentiment dataframe
* Check individual dataframe head to confirm everything worked

In [106]:
all_sentiment = pd.concat([all_sentiment, usdforexsentiment])

usdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
USD,Aon ( AON ) Q3 2022 Earnings Call Transcript,https://www.fool.com/earnings/call-transcripts...,20221028T183037,[Motley Fool Transcribing],0.332498,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Why DexCom Stock Is Soaring Today,https://www.fool.com/investing/2022/10/28/why-...,20221028T155908,[Keith Speights],0.315872,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,This Analyst Expects Solid October Sales Repor...,https://www.benzinga.com/analyst-ratings/analy...,20221028T181129,[Shivani Kumaresan],0.307520,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,CRITEO REPORTS SOLID THIRD QUARTER 2022 RESULTS,https://www.prnewswire.com/news-releases/crite...,20221028T110000,[Criteo S.A.],0.272255,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Criteo ( CRTO ) Q3 2022 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221028T183032,[Motley Fool Transcribing],0.271256,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."


Save individual dataframe to pickle:

In [107]:
usdforexsentiment.to_pickle('usdforexsentiment.pkl')

* Find mean of overall sentiment score

In [108]:
mean = usdforexsentiment['overall_sentiment_score'].mean()

* Create empty column in all_pairs dataframe

In [109]:
all_pairs['sentiment_score'] = ''

* Use a for loop to add sentiment score mean where relevant

In [110]:
for symbol in range(len(all_pairs['to symbol'])):
    all_pairs['sentiment_score'] = all_pairs['to symbol'].apply(lambda x: mean if x == 'USD' else '')

View with empty column:

In [111]:
all_pairs.head()

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time,sentiment_score
date,,,,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.960050,1,
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,0.959033,2,
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,0.956025,3,
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,0.955340,4,


In [112]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:EUR&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:EUR':
                drop_values.append(k)

eurforexsentiment = dftest.drop(index=drop_values)
eurforexsentiment = eurforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

eurforexsentiment['symbol'] = 'EUR'
eurforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, eurforexsentiment])

eurforexsentiment.to_pickle('eurforexsentiment.pkl')

eurmean = eurforexsentiment['overall_sentiment_score'].mean()

eurforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
EUR,Tradeweb Markets Inc ( TW ) Q3 2022 Earnings...,https://www.fool.com/earnings/call-transcripts...,20221027T233039,[Motley Fool Transcribing],0.361996,Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,United States Steel ( X ) Q3 2022 Earnings C...,https://www.fool.com/earnings/call-transcripts...,20221028T193049,[Motley Fool Transcribing],0.330452,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,Anheuser-Busch InBev NV ( BUD ) Q3 2022 Earn...,https://www.fool.com/earnings/call-transcripts...,20221027T210044,[Motley Fool Transcribing],0.330223,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,Mastercard ( MA ) Q3 2022 Earnings Call Tran...,https://www.fool.com/earnings/call-transcripts...,20221027T193016,[Motley Fool Transcribing],0.299494,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."
EUR,"Sanofi Q3 Profit Down, Sales Climb; Lifts FY22...",https://markets.businessinsider.com/news/stock...,20221028T060218,[],0.294378,Somewhat-Bullish,"{'ticker': 'FOREX:EUR', 'relevance_score': '0...."


In [113]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:JPY&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:JPY':
                drop_values.append(k)

jpyforexsentiment = dftest.drop(index=drop_values)
jpyforexsentiment = jpyforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

jpyforexsentiment['symbol'] = 'JPY'
jpyforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, jpyforexsentiment])

jpyforexsentiment.to_pickle('jpyforexsentiment.pkl')

jpymean = jpyforexsentiment['overall_sentiment_score'].mean()
    
jpyforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
JPY,Caterpillar ( CAT ) Q3 Earnings & Sales Top ...,https://www.zacks.com/stock/news/1998818/cater...,20221027T152300,[Zacks Investment Research],0.328319,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,"The U.S. Dollar Is Back On Top, And That Impro...",https://www.forbes.com/sites/johntobey/2022/10...,20221027T043815,[John S. Tobey],0.311861,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,Asia's most influential Web3 business forum br...,https://www.prnewswire.com/news-releases/asias...,20221027T071500,[C-Cubed],0.296489,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,FUJIFILM Diosynth Biotechnologies Enters Into ...,https://www.benzinga.com/pressreleases/22/10/g...,20221027T050500,[Globe Newswire],0.294901,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."
JPY,CRITEO REPORTS SOLID THIRD QUARTER 2022 RESULTS,https://www.prnewswire.com/news-releases/crite...,20221028T110000,[Criteo S.A.],0.272255,Somewhat-Bullish,"{'ticker': 'FOREX:JPY', 'relevance_score': '0...."


In [114]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:GBP&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:GBP':
                drop_values.append(k)

gbpforexsentiment = dftest.drop(index=drop_values)
gbpforexsentiment = gbpforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

gbpforexsentiment['symbol'] = 'GBP'
gbpforexsentiment.set_index('symbol', inplace=True)
                            
all_sentiment = pd.concat([all_sentiment, gbpforexsentiment])
                            
gbpforexsentiment.to_pickle('gbpforexsentiment.pkl')

gbpmean = gbpforexsentiment['overall_sentiment_score'].mean()
    
gbpforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
GBP,Tradeweb Markets Inc ( TW ) Q3 2022 Earnings...,https://www.fool.com/earnings/call-transcripts...,20221027T233039,[Motley Fool Transcribing],0.361996,Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,Rollins ( ROL ) Q3 2022 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221026T200057,[Motley Fool Transcribing],0.345087,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,"ServiceNow, Inc. ( NOW ) Q3 2022 Earnings Ca...",https://www.fool.com/earnings/call-transcripts...,20221027T033026,[Motley Fool Transcribing],0.334421,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,A revival in innovative manufacturing is playi...,https://www.marketwatch.com/story/a-revival-in...,20221026T145000,"[, Asutosh Padhi, David Ebenstein]",0.291532,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."
GBP,CRITEO REPORTS SOLID THIRD QUARTER 2022 RESULTS,https://www.prnewswire.com/news-releases/crite...,20221028T110000,[Criteo S.A.],0.272255,Somewhat-Bullish,"{'ticker': 'FOREX:GBP', 'relevance_score': '0...."


In [115]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:AUD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:AUD':
                drop_values.append(k)

audforexsentiment = dftest.drop(index=drop_values)
audforexsentiment = audforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

audforexsentiment['symbol'] = 'AUD'
audforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, audforexsentiment])

audforexsentiment.to_pickle('audforexsentiment.pkl')

audmean = audforexsentiment['overall_sentiment_score'].mean()
    
audforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
AUD,Rollins ( ROL ) Q3 2022 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221026T200057,[Motley Fool Transcribing],0.345087,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Caterpillar ( CAT ) Q3 Earnings & Sales Top ...,https://www.zacks.com/stock/news/1998818/cater...,20221027T152300,[Zacks Investment Research],0.328319,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Australia and Singapore strike agreement to ac...,https://apnews.com/article/business-australia-...,20221018T074735,[],0.305811,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Vail Resorts ( MTN ) Q4 2022 Earnings Call T...,https://www.fool.com/earnings/call-transcripts...,20220929T000032,[Motley Fool Transcribing],0.286237,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."
AUD,Neogen ( NEOG ) Q1 2023 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221002T163037,[Motley Fool Transcribing],0.283554,Somewhat-Bullish,"{'ticker': 'FOREX:AUD', 'relevance_score': '0...."


In [116]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:CHF&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:CHF':
                drop_values.append(k)

chfforexsentiment = dftest.drop(index=drop_values)
chfforexsentiment = chfforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

chfforexsentiment['symbol'] = 'CHF'
chfforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, chfforexsentiment])

chfforexsentiment.to_pickle('chfforexsentiment.pkl')

chfmean = chfforexsentiment['overall_sentiment_score'].mean()
    
chfforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
CHF,Nestlé reports nine-month sales for 2022,https://www.globenewswire.com/news-release/202...,20221019T051500,[Nestlé S.A.],0.462795,Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Worldline - Q3 2022 revenue - Press Release,https://www.globenewswire.com/news-release/202...,20221025T050000,[WORLDLINE SA],0.381400,Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Virginia Wines Deserve A Place At The Table,https://www.forbes.com/sites/lanabortolot/2022...,20221027T232726,[Lana Bortolot],0.364977,Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Ornellaia Vendemmia d'Artista 2019: Online Auc...,https://www.benzinga.com/pressreleases/22/10/n...,20221006T182000,[PRNewswire],0.362025,Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."
CHF,Swiss utility Axpo increases syndicated credit...,https://www.reuters.com/markets/europe/swiss-u...,20221011T052800,[Reuters],0.272921,Somewhat-Bullish,"{'ticker': 'FOREX:CHF', 'relevance_score': '0...."


In [117]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:CAD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:CAD':
                drop_values.append(k)

cadforexsentiment = dftest.drop(index=drop_values)
cadforexsentiment = cadforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

cadforexsentiment['symbol'] = 'CAD'
cadforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, cadforexsentiment])

cadforexsentiment.to_pickle('cadforexsentiment.pkl')

cadmean = cadforexsentiment['overall_sentiment_score'].mean()
    
cadforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
CAD,Rollins ( ROL ) Q3 2022 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221026T200057,[Motley Fool Transcribing],0.345087,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,STAR DIAMOND CORPORATION PROVIDES UPDATE ON RI...,https://investingnews.com/star-diamond-corpora...,20221021T221431,[],0.305733,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,Fortis Inc. Releases Third Quarter 2022 Result...,https://www.globenewswire.com/news-release/202...,20221028T100000,[Fortis],0.272889,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,Polaris Industries ( PII ) Q3 2022 Earnings ...,https://www.fool.com/earnings/call-transcripts...,20221025T200047,[Motley Fool Transcribing],0.268741,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."
CAD,Wintrust Financial Corporation Reports Third Q...,https://www.globenewswire.com/news-release/202...,20221018T202500,[Wintrust Financial Corporation],0.253265,Somewhat-Bullish,"{'ticker': 'FOREX:CAD', 'relevance_score': '0...."


In [118]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:HKD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:HKD':
                drop_values.append(k)

hkdforexsentiment = dftest.drop(index=drop_values)
hkdforexsentiment = hkdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

hkdforexsentiment['symbol'] = 'HKD'
hkdforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, hkdforexsentiment])

hkdforexsentiment.to_pickle('hkdforexsentiment.pkl')

hkdmean = hkdforexsentiment['overall_sentiment_score'].mean()
    
hkdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
HKD,"2023 is 'year of hope' for Hong Kong, finance ...",https://www.scmp.com/business/banking-finance/...,20221025T114425,[Mia Castagnone],0.272902,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,Futu Holdings Ltd ( FUTU ) Q2 2022 Earnings ...,https://www.fool.com/earnings/call-transcripts...,20220830T150025,[Motley Fool Transcribing],0.235365,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,US dollar strength lets Hong Kong investors go...,https://www.scmp.com/comment/opinion/article/3...,20220913T080020,[Neal Kimberley],0.232905,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,Exclusive: HKMA trials to find suitable use fo...,https://www.scmp.com/business/banking-finance/...,20220929T003022,"[Enoch Yiu, Peggy Sito]",0.216266,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."
HKD,Hong Kong equities beat US stock picks in inve...,https://www.scmp.com/business/markets/article/...,20220908T082540,[Enoch Yiu],0.213054,Somewhat-Bullish,"{'ticker': 'FOREX:HKD', 'relevance_score': '0...."


In [119]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:SEK&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:SEK':
                drop_values.append(k)

sekforexsentiment = dftest.drop(index=drop_values)
sekforexsentiment = sekforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

sekforexsentiment['symbol'] = 'SEK'
sekforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, sekforexsentiment])

sekforexsentiment.to_pickle('sekforexsentiment.pkl')

sekmean = sekforexsentiment['overall_sentiment_score'].mean()
    
sekforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
SEK,Kvika banki hf.: Announcement regarding increa...,https://www.globenewswire.com/news-release/202...,20220922T162900,[Kvika banki hf.],0.378631,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Kvika banki hf.: Announcement regarding increa...,https://www.globenewswire.com/news-release/202...,20220818T173000,[Kvika banki hf.],0.374410,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,IFS continues to deliver strong 2022 financial...,https://www.prnewswire.com/news-releases/ifs-c...,20221027T082100,[IFS],0.365162,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,IFS continues to deliver strong 2022 financial...,https://www.prnewswire.com/news-releases/ifs-c...,20221027T082300,[IFS],0.365162,Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."
SEK,Worldline - H1 2022 results,https://www.globenewswire.com/news-release/202...,20220727T050000,[WORLDLINE SA],0.345460,Somewhat-Bullish,"{'ticker': 'FOREX:SEK', 'relevance_score': '0...."


In [120]:
url2 = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=forex:NZD&from_date:20220901T0000&apikey={}'.format(ALPHA_VANTAGE_API_KEY)
r2 = requests.get(url2)
data2 = r2.json()

listsent = list(data2.items())

listsent.pop(0)
listsent.pop(0)
listsent.pop(0)

flat_ls = []
for i in listsent:
    for j in i:
        flat_ls.append(j)
 
flat_ls.pop(0)

flat_ls2 = []
for i in flat_ls:
    for j in i:
        flat_ls2.append(j)
 
dftest = pd.DataFrame(flat_ls2)

dftest = dftest.explode('ticker_sentiment')

del dftest['summary']
del dftest['banner_image']
del dftest['category_within_source']
del dftest['source_domain']
del dftest['source']
del dftest['topics']

dftest = dftest.reset_index(inplace = False, drop = True)

drop_values = []
for k, v in dftest['ticker_sentiment'].iteritems():
    for j, i in v.items():
        if j == 'ticker':
            if i != 'FOREX:NZD':
                drop_values.append(k)

nzdforexsentiment = dftest.drop(index=drop_values)
nzdforexsentiment = nzdforexsentiment.sort_values(by='overall_sentiment_score', ascending=False)

nzdforexsentiment['symbol'] = 'NZD'
nzdforexsentiment.set_index('symbol', inplace=True)

all_sentiment = pd.concat([all_sentiment, nzdforexsentiment])

nzdforexsentiment.to_pickle('nzdforexsentiment.pkl')

nzdmean = nzdforexsentiment['overall_sentiment_score'].mean()
    
nzdforexsentiment.head()

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
NZD,EverCommerce Inc. ( EVCM ) Q2 2022 Earnings ...,https://www.fool.com/earnings/call-transcripts...,20220809T090025,[Motley Fool Transcribing],0.340220,Somewhat-Bullish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,New Zealand Central Bank Serves Up 4th Straigh...,https://www.benzinga.com/markets/asia/22/08/28...,20220817T112913,[Bhavik Nair],0.127383,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Rupee remains best performer against other glo...,https://www.financialexpress.com/market/rupee-...,20220911T035021,[],0.053979,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Speculators cut long U.S. dollar bets in lates...,https://www.reuters.com/markets/us/speculators...,20220812T193900,[Reuters],0.052987,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Major currencies hold steady ahead of Fed minu...,https://www.reuters.com/markets/us/major-curre...,20220817T015100,[Alun John],0.052572,Neutral,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."


View all_sentiment dataframe:

In [121]:
all_sentiment

,title,url,time_published,authors,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
symbol,,,,,,,
USD,Aon ( AON ) Q3 2022 Earnings Call Transcript,https://www.fool.com/earnings/call-transcripts...,20221028T183037,[Motley Fool Transcribing],0.332498,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Why DexCom Stock Is Soaring Today,https://www.fool.com/investing/2022/10/28/why-...,20221028T155908,[Keith Speights],0.315872,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,This Analyst Expects Solid October Sales Repor...,https://www.benzinga.com/analyst-ratings/analy...,20221028T181129,[Shivani Kumaresan],0.307520,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,CRITEO REPORTS SOLID THIRD QUARTER 2022 RESULTS,https://www.prnewswire.com/news-releases/crite...,20221028T110000,[Criteo S.A.],0.272255,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
USD,Criteo ( CRTO ) Q3 2022 Earnings Call Transc...,https://www.fool.com/earnings/call-transcripts...,20221028T183032,[Motley Fool Transcribing],0.271256,Somewhat-Bullish,"{'ticker': 'FOREX:USD', 'relevance_score': '0...."
...,...,...,...,...,...,...,...
NZD,Dollar towers over peers as markets bet on lar...,https://www.reuters.com/markets/europe/dollar-...,20220615T012800,[Alun John],-0.179154,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Euro on thin ice ahead of U.S. labour data,https://www.reuters.com/markets/us/euro-thin-i...,20220708T020400,[Tom Westbrook],-0.180598,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."
NZD,Japanese yen rises as investors seek havens af...,https://www.aljazeera.com/economy/2022/7/8/jap...,20220708T045733,[],-0.204529,Somewhat-Bearish,"{'ticker': 'FOREX:NZD', 'relevance_score': '0...."


Save to pickle:

In [122]:
all_sentiment.to_pickle('all_sentiment.pkl')

* Use a for loop to apply sentiment score mean to sentiment score column in all_pairs:

In [125]:
for symbol in range(len(all_pairs['to symbol'])):
    all_pairs.loc[(all_pairs['to symbol'] == 'USD'), 'sentiment_score'] = mean
    all_pairs.loc[(all_pairs['to symbol'] == 'EUR'), 'sentiment_score'] = eurmean
    all_pairs.loc[(all_pairs['to symbol'] == 'JPY'), 'sentiment_score'] = jpymean
    all_pairs.loc[(all_pairs['to symbol'] == 'GBP'), 'sentiment_score'] = gbpmean
    all_pairs.loc[(all_pairs['to symbol'] == 'AUD'), 'sentiment_score'] = audmean
    all_pairs.loc[(all_pairs['to symbol'] == 'CHF'), 'sentiment_score'] = chfmean
    all_pairs.loc[(all_pairs['to symbol'] == 'CAD'), 'sentiment_score'] = cadmean
    all_pairs.loc[(all_pairs['to symbol'] == 'HKD'), 'sentiment_score'] = hkdmean
    all_pairs.loc[(all_pairs['to symbol'] == 'SEK'), 'sentiment_score'] = sekmean
    all_pairs.loc[(all_pairs['to symbol'] == 'NZD'), 'sentiment_score'] = nzdmean

View all_pairs:

In [126]:
all_pairs.head(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time,sentiment_score
date,,,,,,,,,,,,,,,,,
2022-06-13,USD,EUR,0.9507,0.9612,0.9503,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.11509
2022-06-14,USD,EUR,0.9604,0.9613,0.9537,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.960050,1,0.11509
2022-06-15,USD,EUR,0.9597,0.9648,0.9516,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,NaN,0.959033,2,0.11509
2022-06-16,USD,EUR,0.9571,0.9631,0.9432,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,NaN,0.956025,3,0.11509
2022-06-17,USD,EUR,0.9473,0.9570,0.9467,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,NaN,0.955340,4,0.11509
2022-06-20,USD,EUR,0.9528,0.9544,0.9482,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,NaN,NaN,0.954617,5,0.11509
2022-06-21,USD,EUR,0.9510,0.9513,0.9448,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,NaN,0.953786,0.953786,6,0.11509
2022-06-22,USD,EUR,0.9489,0.9547,0.9426,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.9604,0.951757,0.952838,7,0.11509
2022-06-23,USD,EUR,0.9460,0.9536,0.9449,0.9499,0.9462,0.9488,0.9510,0.9526,0.9470,0.9570,0.9597,0.950357,0.952511,8,0.11509


In [127]:
all_pairs.tail(10)

,from symbol,to symbol,open,high,low,close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean,expanding_mean,time,sentiment_score
date,,,,,,,,,,,,,,,,,
2022-10-17,NZD,SEK,6.2649,6.3248,6.2372,6.2769,6.2585,6.3119,6.3418,6.3329,6.2762,6.2916,6.2962,6.298543,6.415008,90,0.1545
2022-10-18,NZD,SEK,6.2769,6.3354,6.2769,6.2840,6.2769,6.2585,6.3119,6.3418,6.3329,6.2762,6.2916,6.297457,6.413584,91,0.1545
2022-10-19,NZD,SEK,6.2860,6.3646,6.2860,6.3352,6.2840,6.2769,6.2585,6.3119,6.3418,6.3329,6.2762,6.305886,6.412741,92,0.1545
2022-10-20,NZD,SEK,6.3390,6.4149,6.3135,6.3897,6.3352,6.2840,6.2769,6.2585,6.3119,6.3418,6.3329,6.314000,6.412496,93,0.1545
2022-10-21,NZD,SEK,6.3849,6.4595,6.3703,6.3951,6.3897,6.3352,6.2840,6.2769,6.2585,6.3119,6.3418,6.321614,6.412313,94,0.1545
2022-10-24,NZD,SEK,6.4340,6.4685,6.2850,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.2585,6.3119,6.323357,6.411394,95,0.1545
2022-10-25,NZD,SEK,6.3243,6.3800,6.2690,6.2852,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.2585,6.327171,6.410093,96,0.1545
2022-10-26,NZD,SEK,6.2921,6.3517,6.2853,6.3005,6.2852,6.3241,6.3951,6.3897,6.3352,6.2840,6.2769,6.330543,6.408974,97,0.1545
2022-10-27,NZD,SEK,6.3024,6.4091,6.2454,6.3798,6.3005,6.2852,6.3241,6.3951,6.3897,6.3352,6.2840,6.344229,6.408680,98,0.1545
